# 大模型开发教程

## LangChain 入门篇
### 聊天模型

#### DeepSeek

#### 配置环境

In [1]:
%pip install -qU langchain-deepseek langchain
# 注意：您可能需要重新启动内核才能使用更新后的包。

Note: you may need to restart the kernel to use updated packages.


##### 配置API

In [1]:
import getpass
import os

# os.environ["LANGSMITH_TRACING"] = "true"
if not os.getenv("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("输入您的 DeepSeek API 密钥：")

输入您的 DeepSeek API 密钥： ········


##### 实例化

In [3]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

##### 调用

In [4]:
messages = [
    (
        "system",
        "你是一个乐于助人的助手，负责将英文翻译成中文。请翻译用户的句子。",
    ),
    ("human", "我喜欢编程。"),
]
ai_msg = llm.invoke(messages)
ai_msg.content

'I like programming.'

##### 提示模板

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "你是一个乐于助人的助手，负责将{input_language}翻译成{output_language}。",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "英文",
        "output_language": "中文",
        "input": "我喜欢编程。",
    }
)

AIMessage(content='I enjoy programming.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 20}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '4c9b2e6b-3927-4e83-b0bc-45ee1ee797ba', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--37b32572-b2db-4ab1-b141-7c3419cd81f8-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 简单示例

In [6]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate

def get_completion(content, model="deepseek-chat"):
    llm = ChatDeepSeek(
        model=model,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        # 其他参数...
    )
    prompt = ChatPromptTemplate(
        [
            (
                "system",
                "你是一个乐于助人的助手，负责将{input_language}翻译成{output_language}。",
            ),
            ("human", f"{content}"),
        ]
    )

    chain = prompt | llm
    response = chain.invoke(
        {
            "input_language": "中文",
            "output_language": "英文",
            "input": f"{content}",
        }
    )

    return response.content

get_completion("她的糖是谁？")

'Who is her sugar?  \n\n(Note: The phrase "她的糖" could have different meanings depending on context. If it refers to a pet name or slang (e.g., "sugar" as a term of endearment), the translation would fit. If it\'s literal (e.g., asking about someone\'s sugar ownership), a more precise translation might be needed, such as "Whose sugar is this?" or "Who does this sugar belong to?")'

### 模型

#### 是什么？

在 Langchain 中，“模型”指的是对各种大型语言模型（LLMs）或聊天模型（Chat Models）的**抽象和封装**。Langchain 提供了一个统一的接口，让你能够方便地与不同的 AI 服务提供商（如 OpenAI, Google, Anthropic 等）进行交互，而无需关心底层具体的 API 调用细节。

#### 作用

- 执行核心任务： 模型是真正进行文本生成、理解、分类、摘要等工作的“大脑”。
- 提供统一接口： 无论你使用的是哪家公司的哪种模型，Langchain 都试图让你用相似的代码结构来调用它们。

#### 类型
- **LLMs (Language Models)**： 传统的文本输入、文本输出模型。你给它一个字符串，它返回一个字符串。例如，早期的 text-davinci-003。在 Langchain 中通常对应 LLM 类型的类。
- **Chat Models (聊天模型)**： 设计用于对话场景的模型。它们接收一个**消息列表**（这些消息有不同的角色，如系统、用户、AI），然后返回一个 AI 角色的消息。例如，GPT-4, Claude, Gemini。在 Langchain 中通常对应 ChatModel 类型的类。

#### 模型使用

##### 步骤1：创建ChatOpenAI实例

In [7]:
# 导入 ChatDeepSeek，这是 LangChain 对 DeepSeek API 访问的抽象
from langchain_deepseek import ChatDeepSeek
# 要控制 LLM 生成文本的随机性和创造性，请设置 temperature = 0.0
chat = ChatDeepSeek(temperature=0.0,model="deepseek-chat")

##### 步骤2：创建提示模板实例

In [8]:
# 模板字符串，用于指定目标风格，拥有两个输入变量："style" 和 "text"
template_string = """将由三重反引号分隔的文本 \\
翻译成 {style} 风格。 \\
文本: ```{text}```
"""
# 构建一个 ChatPromptTemplate 实例，用于模板复用
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

##### 步骤3：定义翻译风格与待翻译文本，作为输入变量传入提示模板

In [9]:
# 风格
customer_style = """美式英语 \\
语气平静且尊重
"""

# 文本
customer_email = """
Arrr, I be fuming that me blender lid \\
flew off and splattered me kitchen walls \\
with smoothie! And to make matters worse, \\
the warranty don't cover the cost of \\
cleaning up me kitchen. I need yer help \\
right now, matey!
"""

# 将风格和文本作为输入变量传入提示模板
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

##### 步骤4：调用LLM翻译成指定风格，并打印结果


In [10]:
customer_response = chat(customer_messages)
print(customer_response.content)

C:\Users\14798\AppData\Local\Temp\ipykernel_26280\2211690849.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


Here’s the translated text in a calm and respectful tone, adapted to American English:

```
I’m quite frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn’t cover the cost of cleaning up the mess. I’d really appreciate your help with this right away.  
```  

This version keeps the urgency and concern while maintaining a polite and composed tone. Let me know if you'd like any further adjustments!


### 提示
#### 是什么？

“提示”是你发送给模型输入的**文本或消息列表**。它告诉模型你的需求是什么，提供必要的上下文信息，并指导模型生成期望的输出。
在 Langchain 中，提示通常是动态生成的，而不是固定不变的。你会使用**提示模板 (Prompt Templates)**。

#### 作用
- **指示模型行为**： 明确告诉模型它应该扮演什么角色，要完成什么任务（如写一封邮件、总结一段文字、回答一个问题）。
- **提供上下文**： 包含模型需要理解和处理的相关信息（如用户的具体要求、待处理的文本内容、历史对话记录）。
- **定义输出要求**： 引导模型以特定的格式或风格生成回复。

#### 类型

- **PromptTemplate (字符串模板)**： 用于 LLMs，基于一个包含占位符的字符串。
- **ChatPromptTemplate (聊天模板)**： 用于 Chat Models，基于一个包含多个消息模板（每个模板可以有不同的角色和内容，含占位符）的列表。
你可以做什么？

定义包含变量的提示模板（如 "{subject} 的邮件内容：{content}"）。
将实际的数据（如 subject="会议通知", content="明天下午三点开会..."）填充到模板中，生成最终发送给模型的具体提示文本或消息列表。
管理复杂的提示结构，包括添加前缀、后缀、少量示例等。

#### 举个例子

你有一个写邮件的功能，提示模板可以是 "主题：{subject}\\n收件人：{recipient}\\n正文：{body}"。当用户输入主题、收件人和正文后，你用实际内容填充这个模板，生成完整的邮件文本作为提示发送给模型。

### 输出解析
#### 是什么？

“输出解析”是处理模型原始文本输出的工具。LLMs 和 Chat Models 生成的最终结果通常是纯文本字符串。输出解析器的任务是将这个纯文本转换成更结构化、更方便程序处理的数据格式（如 Python 的字典、列表、布尔值，或者一个自定义的数据对象）。
作用：

结构化输出： 将无格式或半结构化的模型文本输出转换成程序可以直接使用的结构化数据。
验证输出格式： 检查模型生成的文本是否符合预期的格式（例如，是否是有效的 JSON）。
提供指令： 很多输出解析器会生成一段“格式指令”，你可以将其添加到提示中，告诉模型应该以何种格式生成输出，以便解析器能够成功处理。
#### 类型

StrOutputParser: 最简单的，直接返回原始字符串。
JsonOutputParser: 解析 JSON 格式的输出。
CommaSeparatedListOutputParser: 解析逗号分隔的列表。
PydanticOutputParser: 将输出解析成 Pydantic 数据模型对象。
还有其他许多用于不同数据类型的解析器。
#### 你可以做什么？

指定你期望的模型输出格式。
使用相应的解析器将模型的文本输出转换为 Python 对象。
结合提示，引导模型生成易于解析的格式。

#### 举个例子

你让模型从一段用户评论中提取“产品名称”和“评价星级”，并要求它以 JSON 格式输出，如 {"product": "手机", "rating": 5}。模型返回一个字符串 '{"product": "手机", "rating": 5}'。这时，你可以使用 JsonOutputParser 将这个字符串解析成 Python 字典 {'product': '手机', 'rating': 5}，方便后续的代码处理。

#### 如何使用

##### 步骤1：指定返回的JSON的格式规范


In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# 礼物规范
gift_schema = ResponseSchema(name="gift",
                             description="该商品是否是作为礼物为他人购买的？\
                             如果是，回答 True，\
                             如果否或未知，回答 False。")

# 送货天数规范
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="产品送达需要多少天？\
                                      如果未找到此信息，\
                                      输出 -1。")
# 价格值规范
price_value_schema = ResponseSchema(name="price_value",
                                    description="提取任何关于价值或价格的句子，\
                                    并将它们输出为逗号分隔的 Python 列表。")

##### 步骤2：创建解析器实例，获取格式指令

格式指令用于让LLM生成指定的内容格式，以便解析器可以解析，打印得到其内容如下：

In [12]:
# 将格式规范放入列表中
response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
# 构建 StructuredOutputParser 实例
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 获取将发送给 LLM 的格式指令
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // 该商品是否是作为礼物为他人购买的？                             如果是，回答 True，                             如果否或未知，回答 False。
	"delivery_days": string  // 产品送达需要多少天？                                      如果未找到此信息，                                      输出 -1。
	"price_value": string  // 提取任何关于价值或价格的句子，                                    并将它们输出为逗号分隔的 Python 列表。
}
```


##### 步骤3：创建提示模板实例，将文本和格式指令作为输入变量传入

In [13]:
# 提示
review_template_2 = """\\
对于以下文本，提取以下信息：

gift: 该商品是否是作为礼物为他人购买的？如果是，回答 True，如果否或未知，回答 False。

delivery_days: 产品送达需要多少天？如果未找到此信息，输出 -1。

price_value: 提取任何关于价值或价格的句子，并将它们输出为逗号分隔的 Python 列表。

text: {text}

{format_instructions}
"""

customer_review = """
我最近购买了发光小部件，对此非常满意！
我买这个是作为朋友生日礼物送给她的，她非常喜欢。
物流速度快得惊人，仅用了2天就送到了，比我预期的要快得多。
考虑到质量和功能，我认为价格非常合理，实际上是物超所值。
强烈推荐这款产品！
""" # (这里对示例 customer_review 也做了翻译以方便理解)

# 构建 ChatPromptTemplate 实例，用于模板复用
prompt = ChatPromptTemplate.from_template(template=review_template_2)
# 将文本和格式指令作为输入变量传入
messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

##### 步骤4：调用LLM解析文本，并打印结果

In [14]:
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "我认为价格非常合理，实际上是物超所值"
}
```


##### 步骤5：将结果解析为字典类型，并提取与送货天数相关联的值

提取到的值如下：

In [15]:
output_dict = output_parser.parse(response.content)
print(output_dict.get('delivery_days'))

2


### 记忆

由于语言模型本身具有无状态的特性，在与它们互动时，通常无法记住之前的对话历史。这意味着每个请求或API调用都是独立的处理单元，这给构建需要连贯上下文的流畅对话应用带来了挑战。

为了解决这一问题，一些框架（如LangChain）提供了丰富的记忆存储管理策略。

#### 核心组件

##### Memory 组件的核心作用 (Core Role of the Memory Component)

LangChain 中的 Memory 类及其各种实现，其根本目标是在用户与 LLM 的整个交互周期中维持状态 。它负责管理和存储对话历史，充当应用程序会话中 LLM 的“短期记忆”或“长期记忆”。这个组件是实现链或代理状态化功能的核心。

##### ChatMessages：记忆的基础单元 ( ChatMessages: The Fundamental Unit of Memory)

用户与 LLM 之间的交互信息被捕获并结构化为 ChatMessages 对象 。记忆的管理过程，包括信息的摄取、捕获、转换和知识提取，都围绕着处理 ChatMessages 序列展开 。通常，一个 ChatMessage 包含消息内容本身以及发送者类型（如 'human' 代表用户，'ai' 代表模型）。

将交互统一标准化为 ChatMessages 格式是一个关键的设计决策。这种标准化提供了一个清晰、一致的数据结构，使得 LangChain 的记忆系统具有高度的模块化和可扩展性。不同的记忆模块可以预期接收和产生这种格式的数据，而链和其它组件也知道如何消费这种结构。这使得开发者可以根据需要轻松地替换不同的记忆类型，甚至创建自定义的记忆实现，而不会破坏应用程序的整体流程，极大地促进了灵活性和代码复用。

##### 记忆的工作流程 (Memory Workflow)
记忆组件的基本工作流程可以概括为：捕获新的交互信息（通常是用户输入和模型的响应），根据所选的记忆策略将其存储起来，然后在处理下一次交互时，检索相关的历史信息（例如最近的几条消息、整个对话的摘要等），并将这些信息整合到发送给 LLM 的提示 (Prompt) 中 。这个“捕获 -> 存储 -> 检索 -> 利用”的循环是记忆模块融入 LangChain 应用请求-响应周期的核心机制。检索到的历史信息有效地为 LLM 提供了必要的上下文，使其能够生成更相关、更连贯的响应。

#### 主要的记忆类型与实现

| 记忆类型 (Memory Type)          | 描述 (Description)                                                                 |
| :------------------------------ | :--------------------------------------------------------------------------------- |
| **ConversationBufferMemory** | 简单地存储所有历史消息的完整对话。                                                 |
| **ConversationBufferWindowMemory**| 存储最近 K 条消息，即只保留最新的 N 轮对话。                                      |
| **ConversationTokenBufferMemory** | 存储最近的消息，但根据设定的最大 Token 数限制进行截断。                              |
| **ConversationSummaryMemory** | 不存储原始消息，而是随着对话的进行，对历史消息进行总结。                             |
| **ConversationSummaryBufferMemory**| 结合窗口和总结：保留最近消息（窗口），对更早的对话内容进行总结（Buffer）。             |
| **ConversationKGMemory** | 通过提取对话中的实体和关系，构建和维护一个知识图谱来记忆对话中的实体和关系。                       |

#### ConversationBufferMemory
ConversationBufferMemory 的工作方式非常直接：它就像一个无底的容器，完整地存储每一次用户输入和AI输出的文本历史。想象它就是一个简单的聊天记录滚动条，从头到尾记录下所有的对话内容。

##### 步骤 1：搭建基础对话链

首先，我们需要引入 LangChain 的相关模块，并创建一个语言模型（LLM）实例、一个 ConversationBufferMemory 实例，然后将它们连接起来形成一个对话链（ConversationChain）。

In [16]:
# 导入 ChatDeepSeek，这是 LangChain 对 DeepSeek API 访问的抽象
from langchain_deepseek import ChatDeepSeek
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 初始化语言模型。temperature=0.0 通常用于需要确定性输出的场景。
llm = ChatDeepSeek(temperature=0.0,model="deepseek-chat")

# 创建 ConversationBufferMemory 记忆实例
memory = ConversationBufferMemory()

# 将 LLM 和 memory 关联到 ConversationChain
# verbose=True 会打印出链的内部处理过程，方便理解
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

C:\Users\14798\AppData\Local\Temp\ipykernel_26280\1840314544.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\14798\AppData\Local\Temp\ipykernel_26280\1840314544.py:14: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


##### 步骤 2：进行多轮对话

创建好 conversation 链后，我们就可以使用其 .predict() 方法与AI进行交互了。每当你调用 .predict() 发送一条消息，ConversationBufferMemory 就会自动记录下你的输入和AI的回应。

In [17]:
# 进行第一轮对话
print(conversation.predict(input="Hi, my name is Andrew"))

# 进行第二轮对话
print(conversation.predict(input="What is 1+1?"))

# 进行第三轮对话
print(conversation.predict(input="What is my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.
Hi Andrew! It's great to meet you. How's your day going so far? I'd love to hear what's on your mind—whether it's a question, a fun topic you want to discuss, or just a casual chat. I'm here for it all! 😊


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hi Andrew! It's great to meet you. How's your day going so far?

此外，你也可以直接通过 memory.save_context 方法向记忆中手动添加对话记录。这在你希望在对话开始前预设一些历史信息，或者在某些特殊流程中手动管理记忆时非常有用。



In [18]:
# 手动向 memory 添加一条对话记录
memory.save_context(
    {"input": "Not much, just hanging"},
    {"output": "Cool"}
)

当 verbose=True 时，你会看到 LangChain 在每次调用 LLM 之前，是如何构建完整的 Prompt 的。这个 Prompt 中会包含一个名为 Current conversation 的部分，这里就是 ConversationBufferMemory 提供的全部历史记录。例如，在问“What is my name?”之前，Prompt 中就包含了前两轮完整的对话历史。正是因为 LLM 接收到了这些历史信息，它才能顺利地回答出你的名字，让你感觉AI“记住”你了。

（AI 的实际回答 Your name is Andrew, as you mentioned earlier. 是 predict 方法的返回值，显示在链运行结束之后。）



##### 步骤 3：查看记忆内容

如果你想检查 ConversationBufferMemory 中当前存储了哪些对话历史，可以直接访问它的 .buffer 属性，或者使用 .load_memory_variables({}) 方法。

通过 .buffer 属性：

In [19]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hi Andrew! It's great to meet you. How's your day going so far? I'd love to hear what's on your mind—whether it's a question, a fun topic you want to discuss, or just a casual chat. I'm here for it all! 😊
Human: What is 1+1?
AI: That's an easy one! 1 + 1 equals **2**. It's one of the first math facts most people learn—right up there with "2 + 2 = 4" and "5 + 5 = 10." 😊  

Need help with anything more advanced (or just want to chat about numbers)? I'm happy to dive deeper!
Human: What is my name?
AI: You told me earlier that your name is **Andrew**! 😊 Unless you'd like me to call you something else—I'm happy to adjust. Let me know!  

(And don’t worry, I don’t forget names unless you *want* me to. Human memory might be tricky, but AI has your back!)
Human: Not much, just hanging
AI: Cool


输出会是包含所有历史对话的字符串。

通过 .load_memory_variables({}) 方法：

In [20]:
print(memory.load_memory_variables({}))

{'history': 'Human: Hi, my name is Andrew\nAI: Hi Andrew! It\'s great to meet you. How\'s your day going so far? I\'d love to hear what\'s on your mind—whether it\'s a question, a fun topic you want to discuss, or just a casual chat. I\'m here for it all! 😊\nHuman: What is 1+1?\nAI: That\'s an easy one! 1 + 1 equals **2**. It\'s one of the first math facts most people learn—right up there with "2 + 2 = 4" and "5 + 5 = 10." 😊  \n\nNeed help with anything more advanced (or just want to chat about numbers)? I\'m happy to dive deeper!\nHuman: What is my name?\nAI: You told me earlier that your name is **Andrew**! 😊 Unless you\'d like me to call you something else—I\'m happy to adjust. Let me know!  \n\n(And don’t worry, I don’t forget names unless you *want* me to. Human memory might be tricky, but AI has your back!)\nHuman: Not much, just hanging\nAI: Cool'}


它会返回一个字典，其中包含了记忆变量（默认键是 history）及其对应的历史字符串。


#### ConversationBufferMemory 的局限性与更高级的记忆策略
ConversationBufferMemory 因其简单性而易于理解和使用，尤其适合于快速演示或处理预期会很短的对话。然而，这种“照单全收”的记忆方式，在面对长时间、多轮次的对话时会暴露出明显的不足：

Prompt 过长： 随着对话轮数不断增加，记忆中积累的历史文本会越来越长，导致每次发送给 LLM 的 Prompt 变得庞大。
成本激增： 绝大多数 LLM 的 API 调用是根据输入和输出的 Token 数量收费的。Prompt 越长，Token 数越多，成本也就越高。长对话会导致费用快速上涨。
超出上下文窗口： LLM 都有一个处理输入的 Token 上限，即“上下文窗口”。无限增长的对话历史很容易超出这个限制，导致模型无法处理，甚至引发错误。
为了应对这些挑战，LangChain 提供了多种更智能、更高效的记忆策略。它们不再仅仅是简单地记录一切，而是采用不同的方法来管理、精简或概括对话历史，以平衡记忆的需求和 LLM 的处理能力及成本。这些高级记忆类型包括但不限于：

ConversationBufferWindowMemory：只保留最近 N 轮对话，通过固定窗口大小来控制记忆长度。
ConversationTokenBufferMemory：根据设定的最大 Token 数量来保留对话，超出限制时从最旧的开始丢弃。
ConversationSummaryMemory / ConversationSummaryBufferMemory：不保留全部原始对话，而是调用 LLM 对旧的对话进行总结，只保留总结文本和（可选的）最新几轮对话。

### 链

在 LangChain 中，“链”（Chain）是一个至关重要的概念。它不仅仅是将大型语言模型（LLM）与一个简单的提示（Prompt）结合起来，更强大的在于，我们可以通过组合多个链，让 AI 能够按照特定的流程对文本或其他数据执行一系列复杂的操作。链是构建复杂工作流和应用程序的基础。

LangChain 提供了多种不同类型的链，用于处理不同的场景和需求：

- LLM链 (LLMChain)：最基础的链，用于连接 LLM 和提示。
- 简单顺序链 (SimpleSequentialChain)：将多个链按顺序串联，前一个链的单个输出作为后一个链的单个输入。
- 常规顺序链 (SequentialChain)：比简单顺序链更灵活，支持链之间传递多个输入和多个输出。
- 路由链 (RouterChain)：根据初始输入的内容，将任务动态地分配给不同的子链去处理。

#### 前期准备
先加载一个稍后会在示例中用到的数据集（假设数据已保存在 Data.csv 文件中）：

```csv
ProductID,Rating,Date,CustomerName,Review
101,5,2023-10-26,Alice Smith,"This product is amazing! Highly recommend."
102,4,2023-10-27,Bob Johnson,"Good value for money, slightly slow shipping."
103,5,2023-10-28,Charlie Brown,"Very durable and well-made."
104,1,2023-10-29,Diana Prince,"Did not work as advertised."
105,4,2023-10-30,Ethan Hunt,"Seems good so far, easy to set up."
106,2,2023-10-31,Fiona Gallagher,"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
107,5,2023-11-01,George Costanza,"Simply the best! Will buy again."
108,3,2023-11-02,Hannah Montana,"It's alright, met basic expectations."
109,4,2023-11-03,Ivy Wong,"性价比很高，包装很仔细。"
110,1,2023-11-04,Jack Ryan,"完全に期待外れでした。"
```

In [23]:
import pandas as pd

# 加载数据
df = pd.read_csv('Data.csv')

# 显示前几行数据
print(df.head())

EmptyDataError: No columns to parse from file

#### LLM链 (LLMChain)

LLMChain 是 LangChain 中最基础但功能强大的链类型。它的核心作用就是将一个语言模型（LLM）和一个提示模板（PromptTemplate）绑定在一起。它是构建所有更复杂链的基础单元。

下面是一个简单的例子：

##### 步骤 1：初始化语言模型和提示模板

首先创建一个聊天模型实例和一个提示模板。这个提示模板包含一个可变参数 {product}，用于接收产品名称。

In [24]:
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# 初始化聊天模型。这里使用一个较高的 temperature 值，以便获得更具创造性的公司名称。
llm = ChatDeepSeek(temperature=0.0,model="deepseek-chat")

# 定义提示模板。它接受一个变量 'product'。
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

##### 步骤 2：创建 LLM链 并运行
将初始化好的 LLM 和 Prompt 组合成一个 LLMChain 实例。然后，我们可以直接向链提供 {product} 变量的值，调用 .run() 方法来执行链。

In [25]:
# 创建 LLMChain 实例，将 LLM 和 prompt 关联
chain = LLMChain(llm=llm, prompt=prompt)

# 定义产品名称
product = "Queen Size Sheet Set"

# 运行链，传入产品名称
print(chain.run(product))

C:\Users\14798\AppData\Local\Temp\ipykernel_26280\3767528.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\14798\AppData\Local\Temp\ipykernel_26280\3767528.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run(product))


A great name for a company that makes **Queen Size Sheet Sets** should evoke comfort, luxury, and quality while clearly indicating the product. Here are some strong naming ideas:  

### **1. Elegant & Luxurious Names:**  
- **RoyalRest Linens**  
- **Queenly Comfort**  
- **RegalSlumber**  
- **VelvetDream Sheets**  
- **LuxeLinen Co.**  

### **2. Simple & Clear Names:**  
- **QueenSheets**  
- **PerfectQueen Linens**  
- **CozyQueen Bedding**  
- **SheenSheets**  
- **SoftQueen**  

### **3. Nature-Inspired Names (for organic/bamboo sheets):**  
- **BreezeSheets**  
- **CloudRest Linens**  
- **SilkSlumber**  
- **Moonlight Bedding**  
- **PetalSoft Sheets**  

### **4. Modern & Minimalist Names:**  
- **Duvè (play on "duvet")**  
- **Slumber & Co.**  
- **Sheen**  
- **LüxLinens**  
- **NüSleep**  

### **5. Playful & Memorable Names:**  
- **Sheet Happens** (fun & cheeky)  
- **DreamWeave**  
- **SnugQueen**  
- **The Sheet Society**  
- **BeddyBye**  

Would you like a name that l

当运行 chain.run(product) 时，LLMChain 会在内部用 "Queen Size Sheet Set" 填充提示模板，生成完整的 Prompt，然后将这个 Prompt 发送给 ChatOpenAI 模型。模型处理后返回结果，chain.run() 就会输出这个结果。

这就是 LLMChain 的基本用法：接收输入变量，格式化 Prompt，调用 LLM，返回 LLM 的输出。

#### 简单顺序链 (SimpleSequentialChain)
当你的任务需要按顺序执行多个步骤，并且每一步（子链）都只接收前一步的一个输出作为自己的唯一输入，同时也只产生一个输出时，SimpleSequentialChain 是一个非常便捷的选择。它将一系列子链像管道一样串联起来。

示例：我们来构建一个简单的流程：先根据产品名称生成公司名称，然后根据公司名称生成一段描述。

##### 步骤 1：初始化语言模型

In [26]:
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

# 初始化语言模型，用于后面的子链
llm = ChatDeepSeek(temperature=0.0,model="deepseek-chat")

##### 步骤 2：创建第一个子链 (生成公司名称)

这是一个 LLMChain，它接收产品名称 {product}，输出一个公司名称。

In [27]:
# 定义第一个提示模板：根据产品生成公司名称
first_prompt = ChatPromptTemplate.from_template(
    "描述一个生产{product}的公司最好的名称是什么？"
)

# 创建第一个 LLMChain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

##### 步骤 3：创建第二个子链 (生成公司描述)
这是另一个 LLMChain，它接收公司名称 {company_name}，输出一段描述。请注意，这里的输入变量 {company_name} 必须与前一个链的输出变量相匹配（尽管在 SimpleSequentialChain 中不需要显式指定输出键）。

In [28]:
# 定义第二个提示模板：根据公司名称生成描述
second_prompt = ChatPromptTemplate.from_template(
    "为以下公司写一个 20 字的描述：{company_name}"
)

# 创建第二个 LLMChain
chain_two = LLMChain(llm=llm, prompt=second_prompt)

##### 步骤 4：组合子链并运行 SimpleSequentialChain
我们将 chain_one 和 chain_two 按照执行顺序放入一个列表中，然后用这个列表创建 SimpleSequentialChain。前一个链 (chain_one) 的输出会自动作为后一个链 (chain_two) 的输入。

当 verbose=True 时，运行过程会清晰地展示。

In [29]:
# 将两个子链按顺序组合成 SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True # 设为 True 可以看到链的执行过程
                                            )

# 定义初始输入（产品名称）
product = "Queen Size Sheet Set"

# 运行整个 SimpleSequentialChain，只提供第一个链的输入
print(overall_simple_chain.run(product))



> Entering new SimpleSequentialChain chain...
为一家生产Queen Size床品套件的公司命名时，名称应体现舒适、优雅、品质和睡眠体验，同时易于记忆和品牌传播。以下是几个精心设计的建议，涵盖不同风格和定位：

---

### **1. 经典优雅型**
- **RoyalSlumber**  
  （皇家安眠）  
  传递高端与奢华感，暗示产品如皇室般精致。

- **VelvetDream Linens**  
  （天鹅绒之梦家纺）  
  强调柔软触感和梦幻睡眠体验。

---

### **2. 自然舒适型**
- **CloudNine Bedding**  
  （九霄云床品）  
  比喻床品如云端般轻盈舒适，适合追求极致放松的消费者。

- **BambooBreeze Sheets**  
  （竹风床单）  
  若使用环保材质（如竹纤维），突出自然与可持续理念。

---

### **3. 现代极简型**
- **Haven Textiles**  
  （避风港家纺）  
  简洁有力，暗示家是舒适庇护所。

- **LuxeLoom**  
  （奢织工坊）  
  结合“奢华”与“织造”，突出工艺与品质。

---

### **4. 创意趣味型**
- **Queen & Quilt**  
  （女王与被）  
  双关语，既指Queen Size床品，又带俏皮感。

- **SnoozeCraft**  
  （酣眠工坊）  
  将“睡眠”与“手工艺”结合，强调定制化设计。

---

### **5. 地域文化型**
- **Tuscany Linen Co.**  
  （托斯卡纳亚麻坊）  
  借用意大利高端家居文化，传递欧式优雅。

- **NordicNest Bedding**  
  （北欧巢床品）  
  契合斯堪的纳维亚风格的简约与功能性。

---

### **命名核心要素：**
- **关键词联想**：使用如Slumber（安眠）、Linen（亚麻）、Haven（港湾）等词汇。  
- **材质或工艺**：如Loom（织机）、Weave（编织）体现专业性。  
- **情感共鸣**：通过Dream、Cloud等词触发舒适感。  
- **易读易记**：

可以看到，SimpleSequentialChain 自动将第一个链的输出 "RoyalSlumber" 传递给了第二个链，然后由第二个链生成了最终的描述。这种链适用于简单、线性的工作流。

#### 常规顺序链 (SequentialChain)
与 SimpleSequentialChain 只能处理单个输入和输出不同，SequentialChain 提供了更大的灵活性，它允许链之间传递多个输入变量和产生多个输出变量。这对于构建更复杂、分支更多的工作流至关重要。

在使用 SequentialChain 时，你需要为每个子链指定 output_key，以便后续的链或最终输出能引用这个结果。同时，在创建 SequentialChain 时，需要明确声明整个链的 input_variables（整个流程最初需要的输入）和 output_variables（整个流程最终返回的所有输出）。


假设我们有一个用户评论，需要经过以下处理：

- 将评论翻译成英文。
- 用一句话总结英文评论。
- 检测原始评论的语言。
- 结合总结和检测到的语言，用该语言回复用户。
这个流程需要从原始评论产生多个中间结果（英文翻译、总结、语言），并在最后一步结合其中的两个中间结果（总结、语言）生成最终回复。这正是 SequentialChain 的典型应用场景。


##### 步骤 1：初始化语言模型

In [30]:
from langchain_deepseek import ChatDeepSeek
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# 初始化语言模型，用于后面的子链
llm = ChatDeepSeek(temperature=0.9,model="deepseek-chat")

##### 步骤 2：创建需要依次执行的子链

我们为每一步创建一个 LLMChain。关键在于，为每个链指定 output_key，给它的输出结果一个名字。

In [31]:
# 第一个链：翻译评论到英文
first_prompt = ChatPromptTemplate.from_template(
    "将以下评论翻译成英文："
    "\\n\\n{Review}" # 接收原始评论作为输入
)
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review" # 将输出结果命名为 "English_Review"
                    )

# 第二个链：总结英文评论
second_prompt = ChatPromptTemplate.from_template(
    "你能用一句话总结以下评论吗："
    "\\n\\n{English_Review}" # 接收上一个链的输出 "English_Review" 作为输入
)
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary" # 将输出结果命名为 "summary"
                    )

# 第三个链：检测原始评论的语言
third_prompt = ChatPromptTemplate.from_template(
    "以下评论是什么语言：\\n\\n{Review}" # 再次接收原始评论 {Review} 作为输入
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language" # 将输出结果命名为 "language"
                      )

# 第四个链：根据总结和检测到的语言生成回复
fourth_prompt = ChatPromptTemplate.from_template(
    "根据以下摘要，用指定的语言写一个后续回复："
    "\\n\\n摘要: {summary}\\n\\n语言: {language}" # 接收来自 chain_two 的 "summary" 和来自 chain_three 的 "language" 作为输入
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message" # 将输出结果命名为 "followup_message"
                     )

请注意，在 SequentialChain 中，一个链不仅可以接收前一个链的输出，还可以接收整个流程最初的输入变量，甚至可以接收来自它“侧面”的、在它之前执行的其他链的输出（只要这些链在 SequentialChain 定义的列表中位于它前面）。通过 output_key 和后续链的输入变量名匹配，实现了这种多路径的数据流动。

##### 步骤 3：组合链并定义输入/输出

创建 SequentialChain 实例时，需要传入子链列表、整个流程的输入变量名称列表 (input_variables)，以及最终需要作为整个链输出的变量名称列表 (output_variables)。

In [32]:
# 将所有子链按执行顺序组合
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four], # 子链列表
    input_variables=["Review"], # 整个链最初需要的输入变量
    # 整个链最终需要返回的输出变量。注意，即使是中间结果（如 English_Review）也可以包含在这里返回。
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True # 设置为 True 查看执行详情
)

重要提示： 在定义 input_variables 和 output_variables 时，变量名称必须与你的 Prompt 模板中使用的变量名以及子链的 output_key 精确匹配。确保列表中的变量名和链的执行顺序能正确地将数据从输入传递到中间链，最终汇聚到输出。

##### 步骤 4：运行 SequentialChain

现在，我们选择一条评论作为输入，运行整个 overall_chain。

In [33]:
# 从数据集中选择一条评论作为输入
review = df.Review[5]

# 运行整个链，传入原始评论
print(overall_chain(review)) # 注意这里调用的是整个链本身，传入字典或单个值取决于 input_variables 数量

NameError: name 'df' is not defined

当 verbose=True 时，会看到链的执行过程（SequentialChain 的 verbose 输出相对简洁）。最终的输出是一个字典，包含了你在 output_variables 中指定的键及其对应的值。

可以看到，最终输出包含了英文翻译、摘要和根据检测到的法语生成的回复。SequentialChain 成功地协调了多个子链的输入和输出。

#### 路由链 (RouterChain)
路由链（RouterChain）用于处理这样一种场景：你有多个专门用于处理不同类型输入的子链，并且希望系统能根据用户输入的具体内容，智能地选择最合适的那个子链来执行。路由链的核心在于其包含一个“路由器”（Router），这个路由器负责分析输入并决定将任务“路由”到哪个“目标链”（Destination Chain）。

示例：假设我们有一个问答系统，能够回答物理、数学、历史或计算机科学的问题。我们可以为每个主题创建一个专门的子链。当用户提问时，路由链会判断问题属于哪个领域，然后将问题发送给对应的领域专家子链。

##### 步骤 1：定义针对不同领域的提示模板

为每个主题领域创建一个 Prompt 模板，并赋予它一个特定“专家”的 persona。

In [34]:
# 第一个提示：物理学教授人设
physics_template = """你是一位非常聪明的物理学教授。 \\
你擅长以简洁易懂的方式回答物理学问题。 \\
当你不知道问题的答案时，你会承认你不知道。

这是一个问题：
{input}"""

# 第二个提示：数学家人设
math_template = """你是一位非常优秀的数学家。 \\
你擅长回答数学问题。 \\
你之所以如此优秀，是因为你能够将难题分解成各个组成部分，\\
回答这些部分，然后将它们组合起来回答更广泛的问题。

这是一个问题：
{input}"""

# 第三个提示：历史学家人设
history_template = """你是一位非常优秀的历史学家。 \\
你对各个历史时期的人物、事件和背景有着出色的知识和理解。 \\
你具备思考、反思、辩论、讨论和评价过去的能力。 \\
你尊重历史证据，并能利用它来支持你的解释和判断。

这是一个问题：
{input}"""

# 第四个提示：计算机科学家人设
computerscience_template = """ 你是一位成功的计算机科学家。\\
你对创造力、协作、前瞻性思维、自信、强大的解决问题能力、\\
对理论和算法的理解以及出色的沟通技巧充满热情。 \\
你擅长回答编程问题。 \\
你之所以如此优秀，是因为你知道如何通过描述机器易于解释的\\
命令式步骤来解决问题，并且你知道如何选择在时间复杂度和空间复杂度之间\\
取得良好平衡的解决方案。

这是一个问题：
{input}"""

##### 步骤 2：为每个提示模板提供元信息

创建一个列表，包含每个目标领域的元信息（名称、描述和对应的 Prompt 模板）。路由链会使用这些信息来理解每个子链是做什么的。

In [35]:
# 收集每个子链的元信息
prompt_infos = [
    {
        "name": "physics", # 子链的名称
        "description": "适合回答物理学问题", # 子链的描述，用于路由器判断
        "prompt_template": physics_template # 对应的提示模板
    },
    {
        "name": "math",
        "description": "适合回答数学问题",
        "prompt_template": math_template
    },
    {
        "name": "History", # 注意大小写 'History'
        "description": "适合回答历史问题",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "适合回答计算机科学问题",
        "prompt_template": computerscience_template
    }
]

##### 步骤 3：导入所需模块并初始化 LLM

路由链的实现需要一些特定的模块，尤其是 MultiPromptChain（顶层路由链）、LLMRouterChain（使用 LLM 进行路由决策的链）和 RouterOutputParser（解析 LLM 路由器输出的工具）。

In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain # 子链仍然是 LLMChain


# 初始化语言模型，用于后面的子链
llm = ChatDeepSeek(temperature=0.9,model="deepseek-chat")

##### 步骤 4：创建目标子链

根据步骤 1 和 2 中定义的 Prompt 模板和元信息，为每个领域创建一个实际的 LLMChain 实例。这些就是路由器可能将输入发送到的“目标链”。

In [37]:
destination_chains = {}
# 格式化目标链的描述，用于路由器的 Prompt
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# 为每个领域创建一个 LLMChain，并存储在字典中
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain # 字典的键是链的名称

##### 步骤 5：创建默认链 (Fallback)

通常，我们会设置一个默认链。当路由器无法确定哪个目标链最适合处理输入时，或者输入不属于任何已知领域时，就会fallback到这个默认链。

In [38]:
# 定义默认链的 Prompt
default_prompt = ChatPromptTemplate.from_template("{input}")

# 创建默认 LLMChain
default_chain = LLMChain(llm=llm, prompt=default_prompt)

##### 步骤 6：定义路由器的 Prompt 模板

这是路由链的核心部分。它是一个特殊的 Prompt 模板，用于指导 LLM 充当路由决策者。这个模板会向 LLM 提供所有目标链的名称和描述，要求 LLM 根据用户输入，决定将任务路由到哪个链（或默认链），并可以视情况修改输入传递给子链。模板中包含一个特殊的格式要求（期望 LLM 输出 JSON 格式），以便后续的 RouterOutputParser 能够正确解析。


In [39]:
# 路由器的 Prompt 模板
MULTI_PROMPT_ROUTER_TEMPLATE = """给定一个原始文本输入给语言模型，请选择最适合该输入的模型提示。 \\
你将获得可用提示的名称以及该提示最适合什么场景的描述。 \\
如果你认为修改原始输入最终会带来更好的语言模型响应，你也可以修改它。

<< 格式化 >>
返回一个 markdown 代码片段，其中包含格式化为如下所示的 JSON 对象：
\\`\\`\\`json
{{{{
    "destination": string \\ 要使用的提示名称或 "DEFAULT"
    "next_inputs": string \\ 可能是原始输入的修改版本
}}}}
\\`\\`\\`

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，则可以是 "DEFAULT"。
记住：如果你认为不需要任何修改，"next_inputs" 可以就是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得包含 ```json)>>"""

##### 步骤 7：构建路由链
将路由器的 Prompt 模板、用于路由决策的 LLM 以及 RouterOutputParser 组合起来，创建 LLMRouterChain。


In [40]:
# 格式化路由器的 Prompt，将目标链的描述填充进去
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# 创建路由器的 PromptTemplate 实例，并指定输出解析器
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"], # 路由器接收原始输入
    output_parser=RouterOutputParser(), # 用于解析 LLM 输出的 JSON
)

# 创建 LLM 路由器链
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

##### 步骤 8：创建顶层 MultiPromptChain

最后，将路由链、目标链字典和默认链组合到 MultiPromptChain 中。这就是我们将要调用的总链。

In [41]:
# 组合路由器链、目标链字典和默认链
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True # 设为 True 可以看到路由决策过程
                        )

C:\Users\14798\AppData\Local\Temp\ipykernel_26280\3664570637.py:2: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


##### 步骤 9：提问并观察路由效果

现在，我们可以向 chain 提出不同类型的问题，观察路由器如何将问题导向不同的子链。

提问物理问题：

In [42]:
print(chain.run("什么是black body radiation?"))



> Entering new MultiPromptChain chain...
physics: {'input': '什么是black body radiation?'}
> Finished chain.
**黑体辐射（Black Body Radiation）**是指一个**理想化物体（黑体）**在热平衡状态下吸收和发射的电磁辐射。以下是关键点：

1. **黑体的定义**：  
   - 黑体是能**100%吸收所有入射电磁辐射**的完美吸收体（无论波长或方向），同时也是**完美的发射体**。  
   - 现实中不存在理想黑体，但接近的例子包括恒星（如太阳）或实验室中的空腔辐射器。

2. **辐射特性**：  
   - 黑体辐射的**光谱（强度 vs 波长）仅取决于其温度**，与材质无关。  
   - 高温时辐射偏向短波（如可见光、紫外线），低温时偏向长波（如红外线）。

3. **经典理论的失败**：  
   - 19世纪末，经典物理（如瑞利-金斯定律）预测黑体在短波（紫外）区辐射能量会无限大，与实验矛盾，称为**“紫外灾难”**。

4. **普朗克的量子解释（1900年）**：  
   - 普朗克提出**能量量子化**假设：黑体辐射的能量以离散包（量子）形式发射，每份能量 \(E = h\nu\)（\(h\)为普朗克常数，\(\nu\)为频率）。  
   - 由此导出**普朗克定律**，完美拟合实验数据，奠定了量子力学的基础。

**公式示例**：  
普朗克辐射公式描述黑体辐射的光谱亮度：  
\[
B_\lambda(\lambda, T) = \frac{2hc^2}{\lambda^5} \frac{1}{e^{hc/(\lambda k_B T)} - 1}
\]  
（\(h\)：普朗克常数，\(k_B\)：玻尔兹曼常数，\(T\)：温度，\(\lambda\)：波长）

**意义**：黑体辐射研究直接引发了量子革命，改变了人类对能量本质的理解。**


提问数学问题：

In [43]:
print(chain.run("什么是 2 + 2"))



> Entering new MultiPromptChain chain...
math: {'input': '什么是 2 + 2'}
> Finished chain.
### 第一步：理解问题

首先，我需要明确问题的含义。"2 + 2" 是一个简单的加法表达式。我的目标是计算这个表达式的值。

### 第二步：回顾加法的定义

加法是数学中最基本的运算之一。在算术中，加法是将两个或多个数值合并在一起，得到它们的总和。对于自然数（即正整数），加法可以理解为数量的增加。

### 第三步：具体计算

现在，我需要计算 2 加 2。让我们用具体的例子来理解：

- 想象我有 2 个苹果。
- 然后我又得到了 2 个苹果。
- 现在，我总共有多少个苹果？

将这两个数量相加：
- 第一个 2：○○
- 第二个 2：○○

将它们合并：
○○ + ○○ = ○○○○

所以，总共有 4 个苹果。

### 第四步：验证

为了确保我的答案是正确的，我可以使用其他方法来验证：

1. **数数法**：
   - 从 2 开始，再数 2 个数字：2, 3, 4。所以结果是 4。

2. **数学事实**：
   - 基本的加法表中，2 + 2 = 4 是一个公认的事实。

### 第五步：考虑其他可能性

虽然在这个简单的加法中不太可能有其他解释，但为了全面性，我可以考虑：

- **不同数制**：如果在不同的数制（如二进制）中，2 + 2 的结果会不同。
  - 例如，在二进制中，2 表示为 "10"，所以 "10" + "10" = "100"（即十进制的 4）。所以结果仍然是 4。
  - 在三进制中，2 + 2 = 11（因为 3 进制的 "11" 是十进制的 4）。
  
  但通常在未指明的情况下，我们默认使用十进制，所以 2 + 2 = 4。

- **抽象代数**：在某些代数结构中，加法的定义可能不同。但在标准的算术中，加法是通常的数值相加。

### 第六步：总结

经过以上分析和验证，可以确定：

- 在标准的十进制算术中，2 + 2 = 4。
- 在其他数制中，可能需要转换，但通常在没有说明的情况下，默认为十进制。

### 最终答案

**2 + 2 = 4**


提问一个没有明确匹配领域的生物学问题：

In [44]:
print(chain.run("Why does every cell in our body contain DNA?"))



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.
Every cell in our body contains DNA because DNA serves as the fundamental blueprint for life, encoding all the genetic instructions necessary for an organism's growth, development, functioning, and reproduction. Here’s why DNA is present in every cell:

### 1. **Genetic Instructions**:
   - DNA contains the genes that provide the instructions for building and maintaining the organism. These genes dictate everything from protein synthesis to cellular repair and metabolism.

### 2. **Cellular Function and Specialization**:
   - Even though cells specialize into different types (e.g., muscle cells, neurons, skin cells), they all originate from the same fertilized egg and share the same DNA. Different cell types activate or silence specific genes to perform their unique roles.

### 3. **Protein Synthesis**:
   - DNA is transcribed into RNA, which is then translated i

路由链通过这种方式，使得构建能够处理多种不同类型请求的智能系统成为可能，提高了系统的模块化和可扩展性。

### 基于文档的回答

大型语言模型（LLM）本身具备海量的通用知识，但对于特定领域、企业内部或最新的私有数据，它们无法直接获取。要让 LLM 能够回答关于这些自定义数据的问题，就需要结合**检索增强生成（RAG）**技术。LangChain 提供了构建这类系统的便捷工具。

以下将介绍如何使用 LangChain 的组件，从 CSV 文件中加载数据，

#### 生成测试数据
首先，需要一些用于测试的自定义数据。可以使用任意聊天机器人来生成符合特定格式的数据。

生成数据的提示词示例：

```
请随机生成30条商品介绍数据，以逗号分隔的csv格式输出，第一行是表头，输出：no,name,description，包含以下三列：
序号
商品名称
商品的详细描述
```

这个提示词会生成类似下面的 CSV 格式数据。将这些内容保存为一个名为 product.csv 的文件。

```csv
no,name,description
1,智能蓝牙音箱,一款音质出众、连接稳定的智能蓝牙音箱，支持语音助手，提供沉浸式听觉体验。
2,人体工学办公椅,专为长时间伏案工作设计，有效支撑腰部和背部，提高舒适度和工作效率。
3,多功能空气炸锅,采用高速空气循环技术，无需用油或少量用油即可烹饪美食，健康便捷。
4,超薄笔记本电脑,轻巧便携，性能强劲，满足日常办公、学习和娱乐需求，续航持久。
5,高清运动相机,记录精彩瞬间，支持4K视频录制和多种拍照模式，防水耐用。
6,智能手环,实时监测心率、睡眠、步数等健康数据，来电提醒和消息推送，是您的健康生活助手。
7,家用咖啡机,操作简单，快速制作香浓咖啡，享受美好的咖啡时光。
8,无线降噪耳机,主动降噪技术，有效隔绝外界噪音，提供纯净的音乐体验，佩戴舒适。
9,智能扫地机器人,规划式清扫，自动避障，APP远程控制，解放双手，享受洁净家居。
10,便携式投影仪,小巧机身，高清画质，随时随地打造您的专属家庭影院。
11,高速固态硬盘,显著提升电脑运行速度，读写性能强劲，数据存储更安全。
12,车载充电器,双USB接口，智能识别设备，快速充电，保障行车过程不断电。
13,旅行收纳套装,包含多种尺寸收纳袋，有效整理行李，让您的旅途更轻松。
14,保温保冷杯,采用双层真空绝缘技术，长效保温保冷，健康环保。
15,儿童益智积木,环保材质，色彩丰富，激发儿童创造力和空间想象力。
16,瑜伽垫,防滑耐磨，回弹性好，为您的瑜伽练习提供舒适支撑。
17,宠物自动喂食器,定时定量喂食，远程控制，让您的爱宠按时用餐，健康生活。
18,厨房多功能料理机,集榨汁、搅拌、研磨于一体，轻松制作各种美食。
19,LED护眼台灯,无可视频闪，光线柔和均匀，有效保护视力，多种亮度可调。
20,蓝牙游戏手柄,精准操控，舒适握感，提升游戏体验。
21,智能体重秤,监测体重、体脂、肌肉量等多项身体数据，全面了解健康状况。
22,车载吸尘器,小巧便携，吸力强劲，轻松清洁车内灰尘和杂物。
23,户外运动背包,大容量设计，透气舒适，适合徒步、登山等户外活动。
24,家用电烤箱,多种烘烤模式，精确控温，轻松制作面包、蛋糕、烤肉等美食。
25,无线鼠标,人体工学设计，握感舒适，定位精准，提高工作效率。
26,儿童智能手表,实时定位，一键通话，确保儿童安全，家长更放心。
27,植物补光灯,模拟自然阳光，促进植物生长，适合室内盆栽。
28,车载空气净化器,有效过滤车内PM2.5、甲醛等污染物，改善空气质量。
29,颈椎按摩仪,多模式按摩，舒缓颈部疲劳，放松身心。
30,手持挂烫机,快速除皱，小巧便携，居家旅行必备。
```

#### 构建问答链 (RetrievalQA)
有了自定义数据后，可以使用 LangChain 构建一个问答链，让 LLM 能够利用 product.csv 中的信息来回答问题。下面的代码演示了如何快速实现这一点。



In [45]:
%pip install docarray

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


##### 载入数据到 LangChain

为了让 LangChain 处理您的数据，需要使用数据载入器（Loader）将文件内容读取并转化为 LangChain 内部的标准格式：Document 对象列表。

说明： 每个 Document 对象通常包含 page_content 属性（文本内容）和 metadata 属性（数据的元信息，如来源、行号等）。LangChain 支持多种 Loader，用于载入不同格式（PDF, DOCX, JSON, HTML 等）或来自不同来源（数据库、网页、云存储）的数据。

In [47]:
# 导入 CSVLoader 模块
from langchain_community.document_loaders import CSVLoader

# 指定数据文件路径
file_path = 'product.csv'

# 创建一个 CSVLoader 实例
loader = CSVLoader(file_path=file_path, encoding="utf8")

# 调用 loader 的 load() 方法载入数据
docs = loader.load()

# 打印载入的数据，查看其格式（通常是 Document 对象列表）
print(docs[0]) # 可以取消注释查看第一个 Document 对象

RuntimeError: Error loading product.csv

##### 将文本转化为向量（创建 Embeddings 模型）
RAG 的核心之一是能够进行语义搜索，这需要将文本内容转化为数值表示，即向量（Vector）。这个过程称为嵌入（Embedding）。

In [48]:
# 导入 OllamaEmbeddings 模块
from langchain_community.embeddings import OllamaEmbeddings

# 配置 Ollama 服务地址和用于 Embedding 的模型名称
ollama_base_url = 'http://localhost:11434'
# 根据您在 Ollama 中安装的 Embedding 模型修改下面的名称
embedding_model_name = 'nomic-embed-text' # 示例名称，请替换为您实际安装的 Embedding 模型名称

# 创建一个 OllamaEmbeddings 实例
embeddings = OllamaEmbeddings(base_url=ollama_base_url, model=embedding_model_name)

# （可选）演示如何将一段文本转化为向量
test_embed = embeddings.embed_query("你好，我的名字是火眼9988")
print(len(test_embed)) # 打印向量维度
print(test_embed[:5]) # 打印向量的前几个数值

C:\Users\14798\AppData\Local\Temp\ipykernel_26280\1581447899.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(base_url=ollama_base_url, model=embedding_model_name)


ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000247D51958B0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

说明： embed_query 方法用于将查询文本转化为向量，而 embed_documents 方法用于将 Document 对象列表转化为向量列表。这些向量捕获了文本的语义信息。

##### 构建向量存储（存储嵌入向量）
为了能够高效地进行向量相似度搜索，需要一个地方来存储上述步骤中生成的文本向量及其对应的原始文本。这就是向量存储（Vector Store）的作用。

DocArrayInMemorySearch 是一个简单的内存向量存储，适合测试和小型应用。实际生产环境中通常使用独立的向量数据库产品，它们提供更好的性能、可扩展性和持久性。对于大型文档，通常在载入后、嵌入前还有一个 **文本分割（Text Splitting）** 步骤，将长文档切分成小片段，以适应向量搜索和 LLM 上下文窗口。VectorstoreIndexCreator 在内部会处理文本分割（如果需要）和嵌入、存储的完整流程。

In [49]:
# 导入 DocArrayInMemorySearch 模块
from langchain_community.vectorstores import DocArrayInMemorySearch

# 使用载入的 Document 对象列表 (docs) 和 Embedding 模型 (embeddings)
# 构建一个基于内存的向量存储
db = DocArrayInMemorySearch.from_documents(
    docs, # Document 对象列表
    embeddings # Embedding 模型
)

NameError: name 'docs' is not defined

##### 配置大型语言模型（LLM）
虽然 Embedding 模型用于理解文本含义并进行搜索，但最终的回答生成仍然需要一个强大的 LLM 来完成。



In [50]:
# 导入 ChatDeepSeek 模块
from langchain_deepseek import ChatDeepSeek

# 配置 DeepSeek Chat 模型名称
deepseek_model_name = 'deepseek-chat' # DeepSeek Chat 模型名称

# 创建一个 ChatDeepSeek 实例
# temperature=0.0 使模型回答更确定，减少随机性
chat = ChatDeepSeek(temperature=0.0, model=deepseek_model_name)

# （可选）演示直接调用 LLM
response = chat.invoke("你好，你是一个大型语言模型。")
print(response)

content='你好！是的，我是一个基于人工智能的大型语言模型，旨在通过自然语言与你交流，回答问题、提供建议或协助完成各种任务。虽然我没有真实的意识或情感，但我会尽力理解你的需求并给出有用的回应。如果有任何问题或需要帮助的地方，随时告诉我哦！ 😊  \n\n（比如：你想了解某个知识领域？需要写作灵感？还是解决具体问题？）' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 11, 'total_tokens': 91, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'd50b1397-a9ab-44ff-a60a-5f82968e4bcd', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--c1b23ef4-6490-42fe-bcc7-513c6f8e3631-0' usage_metadata={'input_tokens': 11, 'output_tokens': 80, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


#### 检索相关文档（使用 Retriever）
在实际问答时，第一步是根据用户的问题从向量存储中找出最相关的文档片段。Retriever 是 LangChain 提供的一个标准接口，用于执行这个检索任务。它可以基于向量搜索，也可以基于其他检索方法。

In [51]:
# 从之前创建的向量存储 (db) 中创建一个 Retriever 实例
retriever = db.as_retriever()

# （可选）演示使用 Retriever 进行检索
query_for_retrieval = "智能家用电器有哪些？"
retrieved_docs = retriever.invoke(query_for_retrieval) # 或者使用 retriever.get_relevant_documents(query_for_retrieval)
print(len(retrieved_docs)) # 打印找到的文档数量
print(retrieved_docs[0]) # 打印第一个检索到的文档

NameError: name 'db' is not defined

Retriever 接收用户的问题，并在内部调用向量存储的搜索功能，然后返回最相关的 Document 对象列表。

##### 将检索与问答结合（构建 RetrievalQA 链）
现在有了能够检索文档的 Retriever 和能够回答问题的 LLM（ChatDeepSeek 实例）。接下来需要将它们组合起来，形成一个完整的 RAG 问答流程。RetrievalQA 链就是用于实现这一目标的。

In [52]:
# 导入 RetrievalQA 模块
from langchain.chains import RetrievalQA

# 使用 RetrievalQA.from_chain_type 方法创建问答链
qa_chain = RetrievalQA.from_chain_type(
    llm=chat, # 传递 ChatDeepSeek 实例作为 LLM
    retriever=retriever, # 传递 Retriever 实例
    chain_type="stuff", # 指定链类型，这里使用 "stuff"
    verbose=True # 打印详细过程
)

# （可选）更简便的方法：使用 VectorstoreIndexCreator
# VectorstoreIndexCreator 整合了载入、分割、嵌入、存储到向量存储、创建 Retriever 的前 6 个步骤，并可以直接创建 RetrievalQA 链或作为 Retriever 使用。
# from langchain.indexes import VectorstoreIndexCreator
# index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch, embedding=embeddings).from_loaders([loader])
# # index.query(query, llm=chat) # VectorstoreIndexCreator 创建的对象可以直接调用 query 方法，它内部会创建并运行一个 RetrievalQA 链
# # 或者获取 Retriever 给 RetrievalQA 用: retriever_from_creator = index.vectorstore.as_retriever()

NameError: name 'retriever' is not defined

说明： 上面使用了 chain_type="stuff"。下面详细解释不同 chain_type 的含义。

##### RetrievalQA 链的 chain_type 参数详解

chain_type 决定了当 Retriever 找到多个文档片段时，RetrievalQA 链如何将这些片段组织起来传递给 LLM。

stuff (默认也是最简单)：
- 工作原理： 将所有检索到的文档片段的文本内容简单地拼接成一个长字符串，然后将这个长字符串作为上下文，与用户问题一起构建成一个 Prompt，一次性发送给 LLM。
- 优点： 只需要一次 LLM 调用，速度快，成本最低。LLM 可以一次性看到所有上下文，有助于理解文档之间的整体关联。
- 缺点： 如果检索到的文档很多，拼接后的文本可能超出 LLM 的上下文窗口限制，导致截断或错误。
- 适用场景： 检索到的文档数量和总长度较少，确定不会超过 LLM 的上下文窗口限制。

map_reduce：
- 工作原理： 分两步。首先（Map 阶段），将每个检索到的文档片段独立地与用户问题一起发送给 LLM，让 LLM 对每个片段生成一个初步回答。这个过程可以并行执行。然后（Reduce 阶段），将所有初步回答收集起来，再发送给 LLM 进行总结整合，生成最终的回答。
- 优点： 可以处理任意数量的文档，不受单个 LLM 上下文窗口限制。Map 阶段可并行，处理速度相对较快（对于大量文档）。
- 缺点： 需要多次 LLM 调用（至少是文档数量 + 1 次），成本较高。每个文档独立处理，可能丢失文档片段之间的关联性。
- 适用场景： 检索到的文档数量非常庞大，总长度远超 LLM 上下文窗口，且文档片段可以相对独立地提供信息。

refine：
- 工作原理： 逐步处理文档。首先，将第一个文档片段与用户问题发送给 LLM，生成一个初步回答。然后，将上一个回答和下一个文档片段一起发送给 LLM，让 LLM 基于新文档“精炼”或更新上一个回答。重复此过程直到所有文档处理完毕。
- 优点： 可以处理任意数量的文档。在生成回答时可以考虑文档之间的潜在关联，因为后续处理依赖于之前的回答和文档。
- 缺点： 需要多次 LLM 调用，成本较高。处理过程是顺序的，速度较慢。中间回答的质量会影响最终结果。
- 适用场景： 检索到的文档数量大，且文档之间有较强的关联性或顺序依赖性，需要逐步构建最终回答。
 
map_rerank：
- 工作原理： 将每个检索到的文档片段与用户问题一起发送给 LLM，让 LLM 不仅生成一个回答，还要为这个回答给出一个相关性得分。链根据 LLM 给出的得分对所有文档的回答进行排序，选择得分最高（或前几名）的回答作为最终输出。
- 优点： 速度较快（Map 阶段可并行）。通过打分机制，尝试选出最相关的回答，尤其适合文档片段包含独立答案的场景。
- 缺点： 多次 LLM 调用，成本较高。文档独立处理，可能丢失文档间关联。依赖 LLM 准确地生成得分。
- 适用场景： 检索文档量大，每个文档片段可能独立包含答案，需要快速找到最相关的答案。


##### 运行问答链获取回答
创建好 RetrievalQA 链后，就可以直接调用其 .run() 或 .invoke() 方法，传入用户的问题，链会自动执行前面配置的检索和问答流程。

In [57]:
# 定义用户要问的问题
query = "请列出带有智能功能且节能环保的电器，以 Markdown 格式输出，总结它们的功能描述。"

# 运行问答链，传入问题
response = qa_chain.run(query) # 或使用 await qa_chain.ainvoke(query) 对于异步

# 打印最终的回答
print(response)

NameError: name 'qa_chain' is not defined

说明： 当 verbose=True 时，运行 .run() 会打印出详细的日志，展示检索到的文档、构建的 Prompt 以及 LLM 的调用过程。最终打印的 response 就是 DeepSeek Chat 根据检索到的数据生成的回答。

通过以上步骤，成功地将您自己的数据（product.csv）与大型语言模型（DeepSeek Chat）连接起来，构建了一个简单的 RAG 问答系统。 **核心流程是：载入数据 -> 嵌入数据 -> 存储向量 -> 检索相关向量对应的文本 -> 将检索到的文本作为上下文发送给 LLM 进行问答。**  LangChain 的各种模块（Loader, Embeddings, Vector Store, Retriever, RetrievalQA Chain）使得构建这个流程变得标准化和便捷。理解不同 chain_type 的区别，有助于在面对不同数据量和文档特性时做出正确的选择。

### 评估

构建一个基于 LLM 的问答应用后，如何知道它回答得好不好？特别是在更换 LLM 模型、调整 Prompt、优化向量数据库或修改 RAG 流程后，怎么判断这些改动是带来了提升还是降低了效果？人工逐个检查大量问答对的结果是非常耗时耗力的。

LangChain 提供了一些工具，可以帮助我们自动化测试数据的生成和应用回答的评估。本节将介绍如何利用这些工具来评估您的问答应用。

#### 步骤 1：准备待评估的问答链

在进行评估之前，首先需要一个可以运行并产生回答的问答链。这里以前面构建的基于 CSV 数据的 RetrievalQA 链为例。

需要设置 LLM 模型（这里使用 Ollama 的 qwen2，但也可以替换为您自己的 DeepSeek 或其他模型）、Embedding 模型、数据加载器、向量存储和 Retriever，最后创建 RetrievalQA 链实例。

使用的模块： RetrievalQA, ChatOllama, CSVLoader, OllamaEmbeddings, DocArrayInMemorySearch, VectorstoreIndexCreator

作用： 搭建一个完整的 RAG 问答链。

常用参数回顾：
- `ChatOllama` (或其他 LLM 类): base_url, model, temperature 等。
- `CSVLoader`: file_path。
- `OllamaEmbeddings` (或其他 Embeddings 类): base_url, model 等。
- `DocArrayInMemorySearch` (或其他 Vector Store 类): `.from_documents(docs, embeddings)` 方法从文档和嵌入模型创建实例。
- `VectorstoreIndexCreator`: vectorstore_cls, embedding 参数指定向量存储和嵌入模型，`.from_loaders([loader]) `方法从 loader 创建索引。
- `RetrievalQA.from_chain_type()`: llm (LLM 实例), retriever (Retriever 实例), chain_type (字符串，如 "stuff"), verbose (bool), chain_type_kwargs (字典，传递给链类型的额外参数)。retriever 通常通过 `vectorstore.as_retriever()` 获取。

In [54]:
# 导入构建链所需的模块
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama # 这里使用 Ollama LLM 实例
from langchain_community.chat_models import ChatOllama # 也可以使用 ChatOllama
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

# 配置 Ollama 服务地址和模型名称
base_url = 'http://localhost:11434'
llm_model = 'qwen2' # 用于问答和生成测试集的 LLM 模型
embedding_model_name = 'nomic-embed-text' # 用于嵌入的 Ollama 模型名称
file_path = 'product.csv' # 数据文件路径

# # 1. 创建 LLM 实例 (用于问答链和后续评估)
# # 可以根据需要替换为 ChatDeepSeek 或其他模型实例
# llm = ChatOllama(base_url=base_url, model=llm_model)

# 2. 载入测试数据 (原始 Document 对象)
# 创建一个 CSVLoader 实例
loader = CSVLoader(file_path=file_path, encoding="utf8")

# 调用 loader 的 load() 方法载入数据
data = loader.load()

# 3. 创建嵌入模型
embeddings = OllamaEmbeddings(base_url=base_url, model=embedding_model_name)

# 4. 创建向量索引 (包括向量存储和 Retriever)
# VectorstoreIndexCreator 简化了流程
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, # 指定向量存储类型
    embedding=embeddings # 指定嵌入模型
).from_loaders([loader]) # 从 Loader 载入数据并创建索引

# 5. 从索引获取 Retriever
retriever = index.vectorstore.as_retriever() # VectorstoreIndexCreator 创建的对象有 vectorstore 属性

# 6. 创建待评估的问答链 RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # 问答使用的 LLM
    chain_type="stuff", # 链类型，这里使用 "stuff"
    retriever=retriever, # 检索器
    verbose=True, # 开启详细日志
    chain_type_kwargs={ # 传递给具体链类型的额外参数
        "document_separator": "<<<<>>>>>" # 为 "stuff" 链类型指定文档之间的分隔符
    }
)

RuntimeError: Error loading product.csv

说明： chain_type_kwargs 参数用于向所选的 chain_type 实现类传递特定的额外参数。在这里，为 stuff 类型链指定了 document_separator，用于分隔拼接在一起的文档内容。

#### 步骤 2：创建测试数据集
要评估问答链，需要一个包含问题和对应正确答案的测试数据集。这个数据集用来判断问答链给出的回答是否准确。

测试数据集通常是一个字典列表，每个字典包含 query (问题) 和 answer (正确答案)。

##### 步骤 2a：手动添加测试数据
对于少量关键或具有代表性的问答对，可以手动创建测试数据。

In [55]:
# 手动创建一些测试问答对
examples = [
    {
        "query": "高清投影仪支持高清视频播放吗？",
        "answer": "是" # 或者写更详细的“支持高清视频播放”
    },
    {
        "query": "哪一款产品能监测心率？",
        "answer": "智能手环"
    }
]
# print(examples) # 可以打印查看手动创建的测试数据

说明： 手动创建的测试数据质量通常较高，但数量有限，覆盖面可能不足。

##### 步骤 2b：利用 LLM 自动生成测试数据
对于大量原始文档，手动编写测试问答对效率很低。可以利用 LLM 根据原始文档自动生成测试问题和答案。

- 使用的模块： QAGenerateChain
- 作用： 根据提供的文档，使用 LLM 自动生成问题和答案对。
- 常用方法：
    - `.from_llm(llm)`: 使用一个 LLM 实例来创建测试集生成链。这个 LLM 将被用来“扮演老师”，根据文档生成问答。
    - `.apply_and_parse(inputs)`: 接收一个输入列表，对每个输入运行链，并解析 LLM 的输出。输入的格式通常是` [{"doc": Document}, {"doc": Document}, ...]`。

In [56]:
# 导入 QAGenerateChain 模块
from langchain.evaluation.qa import QAGenerateChain

# 创建一个测试集生成链，使用前面创建的 LLM 实例
example_gen_chain = QAGenerateChain.from_llm(llm)

# 从原始文档数据 (data) 中选择一部分 (例如前 5 条) 来生成测试集
# apply_and_parse 方法接收一个字典列表，每个字典必须有 "doc" 键，对应一个 Document 对象
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]] # 只对前 5 个文档生成测试数据
)

# 打印第一条自动生成的测试数据
print(new_examples[0])

NameError: name 'data' is not defined

说明： QAGenerateChain 内部有一个 Prompt，指示 LLM 如何根据文档生成问题和答案，并按照特定格式输出，方便后续解析。apply_and_parse 方法会自动调用 LLM 并尝试解析输出。

如果您想了解 QAGenerateChain 内部的 Prompt 和运行过程，可以开启 LangChain 的调试模式：

In [57]:
# 导入 langchain 模块
import langchain
# 设置调试模式为 True，会打印详细日志
langchain.debug = True

# 重新运行 example_gen_chain.apply_and_parse(...) 的代码，观察输出
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in data[:5]])
print(new_examples[0])

NameError: name 'data' is not defined

调试输出分析： 在调试输出中，您会看到 LangChain 构建了一个 Prompt，其中包含了作为“老师”生成问答的角色设定，以及一个示例的输出格式（QUESTION: ..., ANSWER: ...），然后将原始文档内容放入 Prompt，发送给 LLM。LLM 根据这个 Prompt 生成文本，QAGenerateChain 再解析出 query 和 answer。

##### 步骤 2c：合并测试数据
将手动创建的测试数据与 LLM 自动生成的测试数据合并，形成完整的测试数据集。

In [58]:
all_examples = examples + [ex['qa_pairs'] for ex in new_examples]
print(f"总共生成了 {len(all_examples)} 条测试数据。")
print(all_examples) # 打印完整的测试数据集

NameError: name 'new_examples' is not defined

说明： 这里的 new_examples 是 apply_and_parse 的输出列表，每个元素是一个字典，包含 qa_pairs 键，其值是另一个字典` {'query': ..., 'answer': ...}`。所以需要 `[ex['qa_pairs'] for ex in new_examples]` 来提取问答对列表。

#### 步骤 3：手动运行单个测试用例（了解链的执行）
在进行自动化评估之前，可以先手动选取一两条测试数据，运行问答链，并通过开启 verbose=True 或 langchain.debug = True 来观察链的执行过程。这有助于理解问答链如何处理特定问题和上下文。

In [59]:
# 选择第一条手动创建的测试数据
test_case_query = examples[0]["query"]
expected_answer = examples[0]["answer"]

# 运行问答链获取回答
response = qa_chain.run(test_case_query)

# 打印结果
print(f"问题: {test_case_query}")
print(f"期望的答案: {expected_answer}")
print(f"链的回答: {response}")

NameError: name 'qa_chain' is not defined

调试输出分析： 如果开启了调试模式，你会看到 RetrievalQA 链首先调用 Retriever 找到与 test_case_query 相关的文档，然后根据 chain_type (stuff) 将这些文档内容和问题组合成一个 Prompt，最后将 Prompt 发送给 LLM（ChatOllama）。日志会显示完整的 Prompt 内容和 LLM 返回的原始文本。

通过比对 链的回答 和 期望的答案，可以初步判断链的准确性。但对于大量测试数据，需要自动化评估。

#### 步骤 4：利用 LLM 自动评估结果
为了高效地评估大量测试数据，可以利用另一个 LLM 作为“裁判”，来判断问答链生成的回答是否与测试数据集中的正确答案一致或相符。

- 使用的模块： QAEvalChain
- 作用： 使用 LLM 自动评估一个问答链在给定问题和真实答案下的表现（生成的回答是否正确）。
- 常用方法：
    - `.from_llm(llm)`: 使用一个 LLM 实例来创建评估链。这个 LLM 将被用来“扮演裁判”，根据问题、真实答案和预测答案来打分。
    - `.evaluate(examples, predictions)`: 接收两个列表作为输入：
        - examples: 原始测试数据集，包含问题和真实答案（即 all_examples 列表）。
        - predictions: 待评估问答链对 examples 中问题的回答结果列表。这个列表通常通过调用待评估链的 .apply() 方法获得。apply() 方法接收一个包含 'query' 键的字典列表，并返回一个包含 'result' 键的字典列表（每个字典对应一个输入问题的回答）。

In [60]:
# 导入 QAEvalChain 模块
from langchain.evaluation.qa import QAEvalChain

# （可选）关闭调试模式，避免评估过程输出过多日志
langchain.debug = False

# 1. 使用待评估的问答链 (qa_chain) 对所有测试数据 (all_examples) 生成预测回答
# qa_chain.apply 接收一个包含 'query' 键的字典列表，返回包含 'result' 键的字典列表
# 注意：这里的 predictions 列表结构为 [{'result': '...'}, {'result': '...'}, ...]
predictions = qa_chain.apply(all_examples)

# 2. 创建评估链，使用一个 LLM 作为评估器
# 可以使用与问答链相同的 LLM，也可以使用不同的 LLM
eval_llm = ChatOllama(base_url=base_url, model=llm_model) # 或者使用 chat (ChatDeepSeek)
eval_chain = QAEvalChain.from_llm(eval_llm)

# 3. 运行评估链，获得评估结果
# evaluate 方法接收原始测试数据 (all_examples) 和待评估链的预测结果 (predictions)
# graded_outputs 列表结构为 [{'results': 'CORRECT'}, {'results': 'INCORRECT'}, ...]
graded_outputs = eval_chain.evaluate(
    all_examples, # 原始测试数据 (问题和真实答案)
    predictions # 问答链生成的预测回答
)

# 4. 遍历并打印评估结果
print("\n--- 评估结果 ---")
for i, eg in enumerate(all_examples): # 遍历原始测试数据
    print(f"Example {i}:")
    print("Question: " + eg['query']) # 原始问题
    print("Real Answer: " + eg['answer']) # 真实答案
    # 从 predictions 中获取对应索引的预测回答
    print("Predicted Answer: " + predictions[i].get('result', 'N/A')) # 从 predictions 获取预测答案
    # 从 graded_outputs 中获取对应索引的评估结果
    print("Predicted Grade: " + graded_outputs[i]['results']) # LLM 评估的等级
    print("-" * 20)

NameError: name 'qa_chain' is not defined

说明： QAEvalChain 内部也有一个 Prompt，它会将**问题、真实答案、待评估链给出的预测答案**一起发送给作为“裁判”的 LLM，要求 LLM 判断预测答案是否与真实答案一致，并通常输出“CORRECT”或“INCORRECT”等标记。

评估结果分析：
上面的打印输出会显示每个测试用例的问题、真实答案、问答链给出的预测答案，以及评估链 LLM 给出的判断结果（Predicted Grade）。

- `Predicted Grade: CORRECT`: LLM 裁判认为问答链的回答是正确的（与真实答案一致或意思相符）。
- `Predicted Grade: INCORRECT`: LLM 裁判认为问答链的回答是不正确的。
通过统计 CORRECT 的比例，可以大致衡量问答链在当前测试集上的准确率。如果准确率不高，可能需要检查：

- 检索效果： 问答链是否检索到了相关的文档？检索到的文档是否包含了正确答案所需的信息？（可以通过 verbose=True 或调试模式查看 RetrievalQA 链检索到的文档）
- Prompt 设计： 传递给 LLM 的 Prompt 是否清晰有效？
- LLM 能力： 所选的 LLM 模型是否足够强大，能够理解上下文并生成准确回答？
- 测试数据： 自动生成的测试数据是否准确？手动编写的测试数据是否合理？
#### 步骤 5：总结与思考
利用 LangChain 的 QAGenerateChain 和 QAEvalChain，可以有效地自动化 RAG 问答应用的测试数据生成和结果评估流程。这极大地提高了迭代和优化应用的效率，使得在修改 LLM 模型、参数或数据处理流程后，能够快速量化评估效果的变化。

需要注意的是，基于 LLM 的自动评估并非完美，评估 LLM 本身也存在一定误差。但它提供了一种快速、可扩展的方式来获取大量的评估结果，是 LLM 应用开发中非常实用的工具。

### 代理（Agent）：让 LLM 具备行动能力
大型语言模型（LLM）不仅擅长回答问题，更强大的能力在于它们可以作为**推理引擎**。通过提供外部信息或工具，LLM 能够根据这些信息进行分析、规划，并决定执行下一步的行动。LangChain 的代理（Agent）框架就是为了构建这种具备推理和行动能力的 AI 应用而设计的。

一个 Agent 可以被理解为一个智能体：

1. 接收需求： 获取用户的指令或问题（用户输入）。
2. 分析情境： 利用自己的推理能力，结合已有的背景信息和可用的工具列表，分析如何完成用户的需求。
3. 选择工具： 从它被赋予的“工具箱”中，选择最适合当前任务的工具。
4. 执行操作： 调用选定的工具，并可能提供必要的参数。
5. 处理结果： 接收工具的执行结果，可能需要再次使用 LLM 进行分析或总结。
6. 循环与完成： 根据工具结果决定是继续选择其他工具、调用 LLM 生成最终回复，还是结束任务。
这个过程通常遵循 ReAct 等思想，Agent 在“思考”（推理 Reasoning）和“行动”（Acting）之间循环。

#### 步骤 1：准备工作：选择 LLM 和工具
Agent 的核心是一个 LLM（作为大脑进行思考和决策）以及一系列工具（Agent 可以执行的动作）。

##### 步骤 1a：选择作为 Agent 大脑的 LLM
Agent 需要一个 LLM 来理解用户的指令，进行推理，并决定何时使用哪个工具。

- 使用的模块： ChatOpenAI (或其他 ChatModel 类，取决于您使用的模型)
- 作用： 提供 Agent 进行推理和生成回复的能力。Agent 的智能程度很大程度上取决于所使用的 LLM 的能力。
- 常用参数：
    - model (string): 指定使用的具体模型名称（如 gpt-4, claude-3-sonnet, moonshot-v1-8k 等）。
    - temperature (float): 控制模型输出的随机性，通常在 Agent 中设置为较低的值（如 0.0 到 0.5），以获得更稳定和可预测的决策行为。
    - 其他参数可能包括 API 密钥配置（通常通过环境变量 DEEPSEEK_API_KEY 配置）、超时设置等。

In [61]:
# 导入 ChatDeepSeek 模块
from langchain_deepseek import ChatDeepSeek

# 配置 DeepSeek Chat 模型名称
llm_model = 'deepseek-chat'

# 创建 LLM 实例
# 在 Agent 中，较低的 temperature 通常有助于获得更稳定的工具选择和推理
llm = ChatDeepSeek(temperature=0.3, model=llm_model)

##### 步骤 1b：选择并定义 Agent 可用的工具
工具是 Agent 能够执行的特定操作，例如搜索网页、读写文件、调用某个 API、执行计算等。每个工具都有一个名称、描述（告诉 LLM 工具是做什么的）和执行逻辑。

- 使用的模块： ListDirectoryTool (或其他 LangChain 内置工具类), 或自定义工具。
- 作用： 为 Agent 提供与外部环境交互或执行特定任务的能力。
- 常用方法： 工具类通常无需参数或只需要少量配置，然后实例化即可使用。

In [62]:
# 导入内置工具模块，这里使用文件列表工具
from langchain_community.tools.file_management import ListDirectoryTool
# 导入其他可能的工具示例
# from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
# from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# 创建工具实例。ListDirectoryTool 无需参数。
list_dir = ListDirectoryTool()

# 将工具放入一个列表中，这个列表就是 Agent 的“工具箱”
tools = [list_dir]

# （可选）如果使用 Wikipedia 工具，可能需要先创建它的 Wrapper
# api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
# wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
# tools = [list_dir, wikipedia_tool] # 将多个工具放入列表

tools

[ListDirectoryTool()]

说明： LangChain 提供了大量的内置工具，涵盖文件操作、网络搜索、数学计算、与其他服务的集成等。这些工具通常在 langchain_community.tools 模块下。您也可以通过实现一个简单的接口来创建自定义工具，让 Agent 调用您自己编写的函数或服务。

#### 步骤 2：创建 Agent 实例
将选定的 LLM 和工具列表组合起来，创建 Agent 实例。LangChain 提供了多种创建 Agent 的方法和预设类型，create_react_agent 是其中一个便捷函数，用于创建一个遵循 ReAct 模式的 Agent。这个函数来自 langgraph.prebuilt，需要额外安装 langgraph 库 (pip install langgraph)。

- 使用的模块： create_react_agent (来自 langgraph.prebuilt)
- 作用： 根据 LLM 和工具列表，构建一个可执行的 Agent。
- 常用参数：
    - llm (LLM 实例): 作为 Agent 大脑的 LLM 实例（在此例中是 ChatDeepSeek 实例）。
    - tools (List[Tool]): Agent 可用的工具列表。
    - messages_modifier (Callable, 可选): 用于修改发送给 LLM 的消息格式。
- 返回对象： 一个 Agent 执行器（通常是 Runnable 接口），可以直接调用它来运行 Agent。

In [63]:
%pip install langgraph

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [64]:
# 导入创建 Agent 的函数和 HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage # 用于构建用户输入消息

# 创建 Agent 执行器实例
# 将 ChatDeepSeek 实例 (llm) 和工具列表 (tools) 传入
agent_executor = create_react_agent(llm, tools)

说明： create_react_agent 基于 LangGraph 构建了一个简单的 Agent 流程图，负责接收用户消息，调用 LLM，判断 LLM 输出是工具调用还是最终回复，如果是工具调用则执行工具并将结果反馈给 LLM，循环直到 LLM 生成最终回复。

#### 步骤 3：调用 Agent 执行任务
创建好 Agent 执行器后，就可以通过调用其 `.invoke()` 或 `.run()` 方法，传入用户的指令或问题，让 Agent 开始工作。

- 使用的概念： Agent 执行器实例 (agent_executor)。
- 作用： 启动 Agent 的思考和行动流程。
- 常用方法：
    - `.invoke(input)`: 同步调用 Agent，传入输入，返回最终结果。
    - `.ainvoke(input)`: 异步调用 Agent。
    - 输入格式取决于 Agent 类型，create_react_agent 创建的 Agent 通常接收一个包含消息历史（键为 messages）的字典作为输入。用户的当前指令作为列表中的最后一个 HumanMessage。

In [65]:
# 定义用户指令，使用 HumanMessage 对象封装
user_command = "请找出当前文件夹下的 Python 文件"
agent_input = {"messages": [HumanMessage(content=user_command)]}

# 调用 Agent 执行器，传入用户指令
# invoke 方法会等待 Agent 运行完成并返回结果
response = agent_executor.invoke(agent_input)

# response 是一个字典，包含 Agent 运行的状态和最终结果
# 最终的用户可见回复通常在返回的 messages 列表的最后一个 HumanMessage 或 AIMessage 中
final_message = response['messages'][-1]

# 输出最终结果（通常是最后一个消息的 content）
print(final_message.content)

当前文件夹下没有 Python 文件（`.py` 后缀的文件），只有以下内容：

- `.idea`（可能是 PyCharm 或其他 IDE 的配置文件）
- `.ipynb_checkpoints`（Jupyter Notebook 的检查点文件夹）
- `Data.csv`（CSV 数据文件）
- `大模型开发教程.ipynb`（Jupyter Notebook 文件）

如果您需要查找其他文件夹或特定类型的文件，请告诉我！


说明： HumanMessage 是 LangChain 中表示用户消息的标准类。 Agent 的输入输出通常是 BaseMessage 对象（如 HumanMessage, AIMessage, ToolMessage 等）的列表，构成了对话历史。

#### 步骤 4：理解 Agent 的执行过程（调试）
Agent 的强大之处在于其多步推理和工具使用能力，但这也使得其内部执行过程不像简单链那样直接。开启调试模式可以清晰地看到 Agent 的每一步思考（调用 LLM）和行动（调用工具）。

In [66]:
# 导入 langchain 模块
import langchain
# 设置调试模式为 True
langchain.debug = True

# 重新运行 agent_executor.invoke(agent_input) 的代码，观察详细日志
response = agent_executor.invoke(agent_input)
print(response['messages'][-1].content)

[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:agent] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:agent > chain:call_model] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:agent > chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:agent > chain:RunnableSequence > chain:Prompt] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:agent > chain:RunnableSequence > chain:Prompt] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:LangGraph > chain:agent > chain:RunnableSequence > llm:ChatDeepSeek] Entering LLM run with input:
{
  "prompts": [
    "Human: 请找出当前文件夹下的 Python 文件"
  ]
}
[llm/end] [chain:LangGraph > chain:agent > chain:RunnableSequence > llm:ChatDeepSeek] [6.69s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_

开启调试后，当运行 agent_executor.invoke() 时，控制台会打印大量的日志。观察这些日志中的关键部分，类似流程大致：
这个思考（LLM）-> 行动（工具）-> 观察（工具结果）-> 再思考（LLM）-> 最终回复的循环，是 Agent 执行复杂任务的核心流程。调试模式让这个过程变得透明可追踪。

#### 步骤 5：探索更多 Agent 工具
ListDirectoryTool 只是一个简单示例。LangChain 生态提供了丰富的工具库，可以赋予 Agent 各种能力：

- 文件操作： 读取文件、写入文件等。
- 网络搜索： 搜索 Google, Bing 等，获取实时信息。
- 数学计算： 使用计算器执行精确计算。
- API 调用： 调用任意 RESTful API 与外部服务交互。
- 数据库操作： 查询、修改数据库。
- 特定服务集成： 集成 Jira, GitHub, Shell 等。

可以在 langchain_community.tools 模块中找到许多预设工具。更重要的是，您可以通过实现一个简单的 BaseTool 类或使用 @tool 装饰器来编写自定义工具，让 Agent 能够执行您业务所需的任何操作。这极大地扩展了 Agent 的能力边界。

## LangChain 进阶篇

In [229]:
# %pip install langchain langchain-core langchain-community
# %pip install langchain-openai
# %pip install -U langchain-deepseek

# # 示例：安装 FAISS (CPU 版本)
# %pip install faiss-cpu

# # 示例：安装 Chroma
# %pip install langchain-chroma

# # 示例：安装 Pinecone
# %pip install langchain-pinecone

# # 示例：加载 PDF
# %pip install pypdf

# # 示例：加载网页
# %pip install beautifulsoup4

### RAG 流程

#### RAG 核心概念
RAG (Retrieval-Augmented Generation) 是一种**旨在通过结合外部数据源来增强 LLM 知识的技术** 。其**核心价值在于克服 LLM 固有的局限性**。LLM 的知识被限制在它们的训练数据中，这些数据可能过时，并且肯定不包含用户的私人信息或最新事件 。直接对 LLM 进行微调 (Fine-tuning) 以注入新知识通常成本高昂，且不一定适合需要精确事实检索的任务 。RAG 提供了一种更经济高效的方式，在 LLM 生成回答时动态地为其提供相关的外部信息。这个过程将 LLM 的推理能力与特定、实时的信息检索相结合，从而生成更准确、更相关、更值得信赖的回答，并**有效减少模型产生幻觉**的可能性 。   

这种架构的必要性源于 LLM 的内在限制。因为 LLM 无法自行访问训练数据之外的信息 ，所以需要一个外部机制 (RAG) 在推理时提供这些信息。为了让 RAG 能够高效、准确地 检索 到所需信息，这些外部数据必须首先经过预处理和索引。这个预处理和索引过程包括加载 (Load)、切分 (Split) 和存储 (Store) 数据。因此，整个 RAG 工作流——从数据准备到最终生成——构成了一个完整的解决方案，其根本目的是突破 LLM 的知识边界，使其能够处理涉及特定、私有或实时数据的任务。

#### 标准 RAG 工作流：索引与检索/生成
典型的 RAG 应用包含两个主要阶段 ：   

- **索引 (Indexing) - 通常离线进行**: 这个阶段的目标是准备好个人数据，以便后续能够高效地检索。

    - 加载 (Load): 使用 LangChain 的 DocumentLoaders 从各种来源（如 PDF 文件、文本文件、网页、数据库等）加载原始数据。
    - 切分 (Split): 使用 TextSplitters 将加载的长文档分割成更小、更易于管理的文本块 (chunks)。这对于后续的向量嵌入和检索至关重要，因为大多数 LLM 的上下文窗口大小有限，而且在较小的文本块上进行语义搜索通常更有效。
    - 存储 (Store):
        - 嵌入 (Embed): 使用 EmbeddingModel (词嵌入模型) 将每个文本块转换成一个数值向量 (vector embedding)。这个向量捕捉了文本块的语义含义。
        - 索引 (Index): 将生成的文本块及其对应的向量存储在 VectorStore (向量数据库) 中。向量数据库会对这些向量进行索引，以支持快速的相似性搜索。
- **检索与生成 (Retrieval and Generation) - 运行时进行**: 当用户提出查询时，系统执行以下步骤：

    - 检索 (Retrieve): 使用 Retriever 组件。首先，用户的查询也会被转换成一个向量。然后，Retriever 在向量数据库中搜索与查询向量最相似（即语义上最相关）的文本块向量，并取回对应的文本块。
    - 生成 (Generate): 将用户的原始查询和检索到的相关文本块（作为上下文）一起组合成一个结构化的 prompt (提示)。然后，将这个 prompt 发送给 LLM (例如 DeepSeek)。LLM 利用提供的上下文信息来生成对用户查询的回答。

这个两阶段流程使得 LLM 能够基于提供的外部个人数据进行回答，而不是仅仅依赖其内部的通用知识。

### 模块一：加载个人文档 (Document Loading)

#### 连接数据源：LangChain 的文档加载器
访问个人数据的第一步是将其加载到 LangChain 应用中。DocumentLoaders 是 LangChain 提供的标准接口，用于从各种来源加载数据 。加载后的数据通常表示为 Document 对象列表。每个 Document 对象包含两个主要部分：page_content (文档内容的字符串) 和 metadata (一个包含任意元数据（如来源、页码等）的字典) 。LangChain 提供了大量的内置加载器集成（超过 160 种 ），可以大致分为文件加载器、网页加载器和目录加载器等类别 。

#### 加载方式

##### PDF文件

PDF 是存储报告、文章和笔记的常用格式。可以使用 PyPDFLoader  或 PDFPlumberLoader  等加载器。

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "./note.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()  # load_and_split() 方法可以同时加载和切分

print(f"从 PDF '{pdf_path}' 加载了 {len(documents)} 页。")
# 注意：PyPDFLoader 加载的 Document 元数据通常包含页码 'page'
if documents:
    print(f"第一页元数据: {documents[0].metadata}")

# 查看第一页的内容
documents[0].page_content[0:500]

从 PDF './note.pdf' 加载了 361 页。
第一页元数据: {'producer': 'Typora, Electron', 'creator': 'Typora', 'creationdate': '20220831083928', 'moddate': '2022-08-31T08:39:29+00:00', 'title': '', 'subject': '', 'author': '', 'source': './note.pdf', 'total_pages': 361, 'page': 0, 'page_label': '1'}


'消息中间件笔记  \nMQ概述  \n分布式架构的演进过程  \n概述  \n分布式微服务架构的发展，主要经历了四个阶段：单一应用架构、垂直应用架构、分布 \n式架构和弹性  SOA 架构。 \n单体架构（All In One）  \n当网站流量很小时，只需一个应用，将所有功能都部署在一起，以减少部署节点和成 \n本。此时，用于简化增删改查工作量的数据访问框架 (ORM) 是关键。 \n适用于小型网站，小型管理系统，将所有功能都部署到一个功能里，简单易用。'

langchain.schema.document.Document类型包含两个属性：
- page_content：包含该文档页面的内容。
- meta_data：为文档页面相关的描述性数据。

##### 加载文本文件 (.txt,.md 等)
对于纯文本或 Markdown 文件，可以使用 TextLoader 。

In [3]:
from langchain_community.document_loaders import TextLoader

txt_path = "./note.txt"
loader = TextLoader(txt_path, encoding='utf-8') # 指定编码很重要
documents = loader.load()

print(f"从文本文件 '{txt_path}' 加载了 {len(documents)} 个文档。")
if documents:
    print(f"文档元数据: {documents[0].metadata}") # 通常只包含 'source'

documents[0].page_content[0:500]

从文本文件 './note.txt' 加载了 1 个文档。
文档元数据: {'source': './note.txt'}


'123'

##### 加载网页内容
如果个人数据存储在博客、个人网站或 Notion 公开页面等，可以使用 WebBaseLoader 。bs_kwargs 参数允许使用 BeautifulSoup 的 SoupStrainer 来精确提取特定 HTML 元素的内容，避免加载导航栏、页脚等无关信息 。 

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

url = "https://github.com/mlabonne/llm-course/blob/main/README.md"


header = {'User-Agent': 'python-requests/2.27.1', 
          'Accept-Encoding': 'gzip, deflate, br', 
          'Accept': '*/*',
          'Connection': 'keep-alive'}
loader = WebBaseLoader(web_path=url,header_template=header)

# 调用 WebBaseLoader Class 的函数 load对文件进行加载
documents = loader.load()

print(f"从 URL '{url}' 加载了 {len(documents)} 个文档。")
if documents:
    print(f"文档元数据: {documents[0].metadata}")
    documents[0].page_content[0:500]

USER_AGENT environment variable not set, consider setting it to identify your requests.


从 URL 'https://github.com/mlabonne/llm-course/blob/main/README.md' 加载了 1 个文档。
文档元数据: {'source': 'https://github.com/mlabonne/llm-course/blob/main/README.md', 'title': 'llm-course/README.md at main · mlabonne/llm-course · GitHub', 'description': 'Course to get into Large Language Models (LLMs) with roadmaps and Colab notebooks. - llm-course/README.md at main · mlabonne/llm-course', 'language': 'en'}


##### 加载目录
LangChain 也提供了从整个目录加载文件的方式，例如 DirectoryLoader，它可以配置使用特定的加载器来处理目录中的不同文件类型 。这对于批量处理存储在文件夹中的笔记或文档非常有用。

In [5]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader("./")

### 模块二：为 LLM 准备数据 (Document Splitting)

#### 为何需要文档分割？

在构建基于大型语言模型（LLM）的应用，尤其是检索增强生成（Retrieval-Augmented Generation, RAG）系统时，文档分割是一个至关重要且不可或缺的预处理步骤 。LLM 虽然强大，但其处理能力并非无限。将原始文档分割成更小的、语义相关的块（Chunks）是有效利用 LLM 的基础 。   

文档分割的必要性主要源于以下几个方面：

- **克服模型上下文窗口限制 (Overcoming Model Limitations)**: 大多数 LLM 都有其可以一次性处理的最大文本长度限制，即“上下文窗口”（Context Window）。对于超出此限制的长文档，直接输入模型是不可行的。分割能将长文档分解为适合模型处理的片段 。   
- **提高检索精度和效率 (Enhancing Retrieval Precision & Efficiency)**: 在 RAG 应用中，系统需要根据用户查询从大量文档中检索最相关的信息 。将文档分割成更小、更聚焦的块，可以提高检索的相关性，使得检索系统能够更精确地定位到包含答案的特定部分，而不是返回冗长且可能包含不相关信息的整个文档 。同时，处理和索引更小的块也通常更高效 。   
- **优化表示质量 (Improving Representation Quality)**: 对于长文档，生成单一的嵌入向量（Embedding）来代表整个文档的语义可能会导致信息丢失或表示质量下降，因为模型难以将所有细微差别压缩到一个向量中 。将文档分割成语义上相对独立的块，可以为每个块生成更精确、更具代表性的嵌入向量 。   
- **适应不同长度的文档 (Handling Non-uniform Document Lengths)**: 现实世界中的文档集通常包含长度差异巨大的文档。分割可以将这些不同长度的文档标准化为大小更一致的处理单元 。   
- **优化计算资源 (Optimizing Computational Resources)**: 处理较小的文本块通常更节省内存，并且更容易实现并行处理 。   
因此，选择合适的文档分割策略并精细调整其参数，对于构建高效、准确的 LLM 应用（尤其是 RAG 系统）至关重要 。LangChain 提供了丰富的文本分割器（Text Splitters）来满足不同的需求 。   


#### 通用分割参数：chunk_size 与 chunk_overlap

在 LangChain 的多种分割器中，chunk_size 和 chunk_overlap 是两个最核心、最常用的参数，它们共同决定了分割后块的大小和块之间的关系 。   

- chunk_size (**块大小**): 这个参数定义了每个分割后块（Chunk）的最大长度 。需要注意的是，**长度的衡量单位取决于所使用的分割器**。例如，CharacterTextSplitter 通常以字符数计算 ，而 TokenTextSplitter 则以 Token 数量计算 。选择合适的 chunk_size 非常重要，它需要平衡多个因素：   
    - **模型上下文窗口**: 块大小必须小于或等于下游 LLM 或嵌入模型的最大上下文窗口限制 。   
    - **信息完整性**: 块太小可能会割裂语义单元（如一个完整的段落或观点），导致上下文丢失 。   
    - **检索相关性**: 块太大可能包含过多不相关信息，降低检索精度 。   
    - **计算成本**: 块越小，数量越多，可能增加嵌入和索引的计算与存储成本 。   
- chunk_overlap (**块重叠**): 这个参数指定了连续块之间重叠的字符（或 Token）数量 。设置重叠的主要目的是**在块的边界处保持上下文的连续性** 。例如，如果一个重要的句子恰好被分割点分开，重叠部分可以确保这个句子的完整信息同时存在于两个相邻的块中，从而减少因分割带来的信息损失 。   

**参数选择的权衡**:
选择 chunk_size 和 chunk_overlap 的值是一个需要权衡和实验的过程。较大的 chunk_size 可以包含更完整的上下文，但可能超过模型限制或引入噪声；较小的 chunk_size 更精确，但可能丢失上下文。适度的 chunk_overlap 有助于保持连续性，但会增加总处理数据量和存储冗余。通常建议从一个合理的默认值开始（例如，chunk_size 设为 500-1000，chunk_overlap 设为 chunk_size 的 10%-20%），然后根据具体应用场景、文档特性和下游任务的效果进行调整和优化 。可视化工具如 Chunkviz 可以帮助理解不同参数设置下的分割效果 。

#### LangChain 中的主要文档分割策略

LangChain 提供了多种文档分割策略，以适应不同的文档类型和应用需求。这些策略大致可以分为基于长度、基于文本递归结构、基于文档特定结构和基于语义的分割 。   

##### 基于长度的分割 (Length-Based Splitting)
这是最直观的策略，通过设定块的大小限制来分割文本 。    

###### 字符分割 (CharacterTextSplitter)

**工作机制**: 这是最基础的分割器之一。它根据用户指定的单个**分隔符 (separator)** 来分割文本。默认分隔符是 \n\n（双换行符），通常用于按段落分割 。分割后，它会尝试合并相邻的小片段，直到达到 chunk_size 上限 。   

**参数详解**:
- separator: 用于分割文本的字符（默认为 "\n\n"）。   
- chunk_size: 块的最大字符数 。   
- chunk_overlap: 块之间的重叠字符数 。   
- length_function: 计算块长度的函数（默认为 len，即字符数）。   

**行为特点**: 如果文本中不存在指定的分隔符，或者两个分隔符之间的文本长度本身就超过了 chunk_size，CharacterTextSplitter **不会**强行在中间断开，而是会产生一个大于 chunk_size 的块 。它优先保证以分隔符进行分割，而不是严格遵守 chunk_size。  

**优缺点**: 优点是简单、快速、计算开销小 ；缺点是比较“朴素”，可能在不合适的地方（如句子中间）进行分割（如果分隔符选择不当或文本缺乏规律分隔符），忽略文本的语义结构 。   

**适用场景**: 适用于结构简单、段落分隔清晰的纯文本文档，或者对分割精度要求不高、追求速度的场景 。

In [10]:
from langchain_text_splitters import CharacterTextSplitter

some_text = "这是第一段。\n\n这是第二段，它比较长，可能会超过预设的块大小。\n\n这是第三段。"
# 假设 chunk_size 较小，比如 20
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=20,
    chunk_overlap=5,
    length_function=len,
)
docs = text_splitter.create_documents([some_text])
# 输出可能类似于：
# [Document(page_content='这是第一段。'),
#  Document(page_content='这是第二段，它比较长，可能会超过预设的块大小。'), # 注意：这个块可能大于 chunk_size
#  Document(page_content='这是第三段。')]
print(docs)

Created a chunk of size 23, which is longer than the specified 20


[Document(metadata={}, page_content='这是第一段。'), Document(metadata={}, page_content='这是第二段，它比较长，可能会超过预设的块大小。'), Document(metadata={}, page_content='这是第三段。')]


###### Token分割 (TokenTextSplitter)

**工作机制:** 这种分割器采用与 LLM 更接近的方式工作。它首先使用一个分词器（Tokenizer，如 tiktoken ）将文本分解为 Token，然后根据指定的 chunk_size（以 Token 数量计）来分割 Token 序列，最后再将每个块的 Token 转换回文本 。   

**参数详解**:
- encoding_name (或传递 tokenizer 对象): 指定用于计算 Token 的编码器名称或分词器实例 。   
- chunk_size: 块的最大 Token 数 。   
- chunk_overlap: 块之间的重叠 Token 数 。

**优缺点**: 优点是能精确控制块的 Token 数量，与 LLM 的处理方式更一致，有助于避免超出模型 Token 限制 ；缺点是分割点可能出现在单词内部或不符合自然语言阅读习惯的地方，降低了块的可读性，且依赖外部 Tokenizer 库，计算开销略高于字符分割 。 

**适用场景**: 主要用于为 RAG 或其他需要精确控制输入 LLM 的 Token 数量的应用准备数据 。

In [11]:
from langchain_text_splitters import TokenTextSplitter

# 假设我们有一段文本
text = "LangChain makes building LLM applications easier. Token splitting aligns with model processing."
# 使用 tiktoken (例如 gpt-4 使用的 cl100k_base)
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", # 或者直接传递 tokenizer 实例
    chunk_size=10, # 按 Token 数量分割
    chunk_overlap=2
)
texts = text_splitter.split_text(text)
# 输出会是根据 Token 边界分割的文本片段
print(texts)

['LangChain makes building LLM applications easier. Token', '. Token splitting aligns with model processing.']


##### 基于文本递归结构的分割 (Recursive Splitting)

**工作机制**: 这是 LangChain **推荐的通用文本分割器** 。它采用一种更智能、更尊重文本结构的方法。它接收一个**分隔符列表 (separators)**，并按顺序尝试使用这些分隔符进行分割。首先尝试使用列表中的第一个分隔符（默认是 "\n\n"，段落分隔符）。如果分割后的块仍然大于 chunk_size，它会递归地对这些过大的块应用列表中的下一个分隔符（默认是 "\n"，句子分隔符），以此类推，直到块大小符合要求或用尽所有分隔符 。   

**默认分隔符列表**: `["\n\n", "\n", " ", ""]` 。这个顺序的设计思想是**尽可能保持语义相关性最强的单元（段落 -> 句子 -> 单词）的完整性** 。   

**参数详解**:
- separators: 一个字符串列表，定义了尝试分割的字符顺序 (默认为 `["\n\n", "\n", " ", ""]`) 。   
- chunk_size: 块的最大长度（默认以字符数计）。   
- chunk_overlap: 块之间的重叠字符数 。   
- length_function: 计算块长度的函数（默认为 len）。   
- keep_separator: 是否在分割后的块中保留分隔符本身（默认为 True）。

**优缺点**: 优点是能更好地保持文本的自然结构和语义连贯性，是处理通用文本的良好起点 ；缺点是块大小可能不如固定大小分割器那样均匀 。   

**适用场景**: 适用于大多数通用文本分割任务，特别是当希望分割结果尽可能尊重原文的段落和句子结构时 。

递归分割的工作逻辑: RecursiveCharacterTextSplitter 之所以被推荐，是因为其递归机制和默认分隔符列表体现了一种“自上而下”的分割哲学。它首先尝试在最大的逻辑单元（段落）处分割，只有在必要时才诉诸更小的单元（句子、词语）。这种方式相比于简单的字符或 Token 分割，更有可能**保留文本的局部语义结构**，这对于 RAG 等需要理解上下文的任务通常是有利的 。   

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 示例文本，包含段落和句子
long_text = "这是第一段。\n它包含多个句子。\n\n这是第二段。\n它也包含多个句子。\n这个句子特别长，可能需要根据空格或字符进一步分割。"

text_splitter = RecursiveCharacterTextSplitter(
    # 可以自定义分隔符列表
    # separators=["\n\n", "\n", "。", "，", " "],
    chunk_size=50, # 设置一个较小的块大小以观察递归效果
    chunk_overlap=10,
    length_function=len,
    keep_separator=True
)
docs = text_splitter.create_documents([long_text])
# 输出会是根据分隔符优先级和 chunk_size 分割的块列表
for doc in docs:
    print(doc.page_content)
    print("-" * 10)

这是第一段。
它包含多个句子。
----------
这是第二段。
它也包含多个句子。
这个句子特别长，可能需要根据空格或字符进一步分割。
----------


##### 基于文档特定结构的分割 (Structure-based Splitting)

对于具有明确结构的文件格式（如 Markdown, HTML, 代码），利用其固有结构进行分割通常能产生更符合逻辑、更具上下文意义的块 。LangChain 为此提供了专门的分割器。   

###### 分割 Markdown 文档 (MarkdownHeaderTextSplitter)

**工作机制**: 此分割器专门用于分割 Markdown 文档。它根据用户指定的 Markdown 标题级别（如 #, ##, ###）来切分文本 。每个块包含从一个指定级别的标题开始到下一个同级或更高级别标题之前的内容。   

**参数详解**
- headers_to_split_on: 一个元组列表，定义了要作为分割依据的标题级别及其对应的元数据键名。例如：`[("#", "Header 1"), ("##", "Header 2")]` 表示在 H1 和 H2 标题处分割，并将对应的标题文本存入元数据的 "Header 1" 和 "Header 2" 键下 。   
- strip_headers: 一个布尔值，控制是否从最终块的内容中移除作为分割依据的标题行（默认为 True，即移除）。   
元数据处理: 这是此分割器的核心优势之一。它会自动将块所属的标题层级信息（根据 headers_to_split_on 的定义）添加到每个块的 metadata 字典中 。这对于后续需要根据文档结构进行检索或过滤非常有用。

**元数据处理**: 这是此分割器的核心优势之一。它会自动将块所属的标题层级信息（根据 headers_to_split_on 的定义）添加到每个块的 metadata 字典中 。这对于后续需要根据文档结构进行检索或过滤非常有用。   

与其他分割器结合: MarkdownHeaderTextSplitter 非常适合作为第一阶段的分割器，先按逻辑结构（章节、小节）粗分，然后再使用 RecursiveCharacterTextSplitter 或其他分割器对得到的块进行细粒度分割，以满足 chunk_size 的要求，同时保留结构信息 。

In [14]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

markdown_document = """
# 第一章：引言

这是引言内容。

## 1.1 背景

这是背景介绍。

## 1.2 目标

这是目标说明。

# 第二章：方法

这是方法章节的开头。
"""

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = md_splitter.split_text(markdown_document)

# 打印分割结果，注意 metadata
for split in md_header_splits:
    print(f"Content: {split.page_content.strip()}")
    print(f"Metadata: {split.metadata}")
    print("-" * 20)

# 可以进一步结合 RecursiveCharacterTextSplitter 控制块大小
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
final_splits = text_splitter.split_documents(md_header_splits)
print(final_splits)

Content: 这是引言内容。
Metadata: {'Header 1': '第一章：引言'}
--------------------
Content: 这是背景介绍。
Metadata: {'Header 1': '第一章：引言', 'Header 2': '1.1 背景'}
--------------------
Content: 这是目标说明。
Metadata: {'Header 1': '第一章：引言', 'Header 2': '1.2 目标'}
--------------------
Content: 这是方法章节的开头。
Metadata: {'Header 1': '第二章：方法'}
--------------------
[Document(metadata={'Header 1': '第一章：引言'}, page_content='这是引言内容。'), Document(metadata={'Header 1': '第一章：引言', 'Header 2': '1.1 背景'}, page_content='这是背景介绍。'), Document(metadata={'Header 1': '第一章：引言', 'Header 2': '1.2 目标'}, page_content='这是目标说明。'), Document(metadata={'Header 1': '第二章：方法'}, page_content='这是方法章节的开头。')]


###### HTML分割(HTMLHeaderTextSplitter)

**工作机制**: 与 Markdown 版本类似，但它基于 HTML 的标题标签（如 `<h1>`, `<h2>`, `<h3>` 等）来分割文档内容 。  

**参数详解**:
- headers_to_split_on: 定义要分割的 HTML 标题标签及其元数据键名，例如 `[("h1", "Header 1"), ("h2", "Header 2")]`。  
- return_each_element: 布尔值，如果设为 True，则每个 HTML 元素（如 `<p>`, `<li>`）都会成为一个独立的块，并附带其所属的标题


**元数据**；如果为 False（默认），则同一标题下的连续元素会被合并到一个块中 。   
元数据处理: 同样地，它会将块所属的 HTML 标题层级信息添加到 metadata 中 。

In [15]:
from langchain_text_splitters import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <h1>主标题</h1>
    <p>这是一个段落。</p>
    <h2>章节一</h2>
    <p>章节一的内容。</p>
    <ul><li>列表项1</li><li>列表项2</li></ul>
    <h2>章节二</h2>
    <p>章节二的内容。</p>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)

# 打印分割结果
for split in html_header_splits:
    print(f"Content: {split.page_content.strip()}")
    print(f"Metadata: {split.metadata}")
    print("-" * 20)

# 同样可以结合 RecursiveCharacterTextSplitter 控制大小
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)
final_splits = text_splitter.split_documents(html_header_splits)
# print(final_splits)

Content: 主标题
Metadata: {'Header 1': '主标题'}
--------------------
Content: 这是一个段落。
Metadata: {'Header 1': '主标题'}
--------------------
Content: 章节一
Metadata: {'Header 1': '主标题', 'Header 2': '章节一'}
--------------------
Content: 章节一的内容。  
列表项1  
列表项2
Metadata: {'Header 1': '主标题', 'Header 2': '章节一'}
--------------------
Content: 章节二
Metadata: {'Header 1': '主标题', 'Header 2': '章节二'}
--------------------
Content: 章节二的内容。
Metadata: {'Header 1': '主标题', 'Header 2': '章节二'}
--------------------


###### 代码分割 (RecursiveCharacterTextSplitter.from_language)

**工作机制**: RecursiveCharacterTextSplitter 提供了一个便捷的类方法 `.from_language()`，专门用于分割各种编程语言的代码 。它会根据指定的编程语言，自动加载预定义的、适合该语言语法结构的分隔符列表（例如，优先按类、函数定义分割，然后是语句等）。   

**支持的语言**: LangChain 通过 Language 枚举类型内置了对多种主流编程和标记语言的支持 。   

**参数详解:**
- `language`: 必需参数，需要传入 `Language` 枚举中的一个值，例如 `Language.PYTHON` 或 `Language.JS` 。
- `chunk_size`, `chunk_overlap`: 与标准的 `RecursiveCharacterTextSplitter` 相同，分别控制块的最大长度和重叠长度。

**元数据的重要性**: 对于结构化分割器（Markdown, HTML, Code），其核心价值不仅在于按结构分割，更在于**将结构信息作为元数据附加到每个块上** 。这些元数据（如块所属的章节标题、函数名）对于下游的 RAG 应用极其有用。例如，可以在检索时利用元数据进行过滤（只检索特定章节的内容），或者在生成答案时告知 LLM 块的来源上下文，提高答案的准确性和可追溯性。   

> (其他结构化分割器，如 HTMLSectionSplitter, HTMLSemanticPreservingSplitter, RecursiveJsonSplitter 等提供了更专门化的功能，例如处理 HTML 的 `<section>` 标签、保留表格/列表完整性、或递归处理 JSON 结构，可以作为进阶学习方向 )   

In [19]:
# 1. 导入必要的库
# RecursiveCharacterTextSplitter 是用于递归分割文本的类
# Language 是一个枚举，用于指定文本的语言（这里是 Python）
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

# 2. 定义要分割的 Python 代码字符串
PYTHON_CODE = """
   def hello_world():
     print("Hello, World!")

   class MyClass:
     def __init__(self, name):
       self.name = name

     def greet(self):
       print(f"Hello, {self.name}!")

   # Call the function
   hello_world()
   obj = MyClass("LangChain")
   obj.greet()
   """

# 3. 创建 Python 代码分割器实例
# 使用 .from_language(language=Language.PYTHON, ...) 是关键
# 这告诉分割器要使用专门为 Python 语法优化的分隔符
# 例如：优先按空行('\n\n')、类定义('class ...'), 函数定义('def ...')分割
# chunk_size=60: 目标块大小设为 60 字符（这可能有点小）
# chunk_overlap=10: 块之间的重叠设为 10 字符
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=60, chunk_overlap=10
)

# 4. 使用分割器处理 Python 代码
# .create_documents() 方法接收代码字符串，返回一个 Document 对象列表
# 每个 Document 对象包含一块代码（在 page_content 属性中）
python_docs = python_splitter.create_documents([PYTHON_CODE]) # 注意：传入列表 [PYTHON_CODE] 或直接传入 PYTHON_CODE 通常都可以

# 5. (可选) 遍历并打印每个代码块的内容，用于检查结果
print(f"--- Splitting with chunk_size=60, chunk_overlap=10 ---")
for i, doc in enumerate(python_docs):
    print(f"Chunk {i+1}:")
    print(doc.page_content)
    print("-" * 20)

--- Splitting with chunk_size=60, chunk_overlap=10 ---
Chunk 1:
def hello_world():
     print("Hello, World!")
--------------------
Chunk 2:
class MyClass:
     def __init__(self, name):
--------------------
Chunk 3:
self.name = name
--------------------
Chunk 4:
def greet(self):
       print(f"Hello, {self.name}!")
--------------------
Chunk 5:
# Call the function
   hello_world()
--------------------
Chunk 6:
obj = MyClass("LangChain")
   obj.greet()
--------------------


##### 基于语义的分割 (Semantic Splitting)
与前面基于固定规则（长度、分隔符、结构）的方法不同，语义分割尝试**根据文本内容的含义**来确定分割点 。其核心思想是在文本语义发生显著变化的地方进行切分，以期得到语义上更内聚、更连贯的块 。   

###### 语义文档分割器（SemanticChunker）

**工作机制**: SemanticChunker 是 LangChain 中实现语义分割的主要工具（位于 langchain_experimental 包，表示其可能仍在发展和变化中）。其基本流程如下 ：
1. **句子分割**: 首先将输入文本分割成句子（可以使用正则表达式 sentence_split_regex 控制）。
2. **句子嵌入**: 使用用户提供的**嵌入模型 (Embeddings model)** 计算每个句子的向量表示。
3. **计算差异**: 计算相邻句子（或通过滑动窗口聚合的句子组 ）嵌入向量之间的距离或相似度差异。   
4. **确定断点 (Breakpoints)**: 根据选定的阈值类型 (breakpoint_threshold_type) 和阈值 (breakpoint_threshold_amount)，判断哪些句子间的语义差异足够大，可以视为一个语义单元的结束和下一个单元的开始。差异超过阈值的点即为分割点（断点）。
5. **合并成块**: 将位于两个断点之间的句子合并成一个块。
  
**嵌入模型需求**: SemanticChunker 的有效性**强依赖于所使用的嵌入模型**。必须在初始化时提供一个 LangChain 的 Embeddings 对象实例（如 OpenAIEmbeddings, HuggingFaceEmbeddings 等）。嵌入模型的质量直接影响句子语义表示的准确性，进而影响分割效果。   
**断点确定 (Breakpoint Determination)**: 通过比较相邻句子（或句子组）的嵌入向量来识别语义上的“跳跃点”或不连续处 。   
**阈值类型 (breakpoint_threshold_type)**: SemanticChunker 提供了几种不同的方法来确定分割阈值 ：
- percentile (默认值): 计算所有相邻句子（或组）嵌入差异值，并将差异值大于指定百分位数（默认为 95%）的位置作为分割点。例如，只有语义差异最大的前 5% 的地方会被分割 。   
- standard_deviation: 计算所有差异值的均值和标准差。将差异值大于“均值 + N * 标准差”（N 默认为 3）的位置作为分割点 。   
- interquartile: 计算差异值的四分位距 (IQR = Q3 - Q1)。将差异值大于“Q3 + N * IQR”（N 默认为 1.5）的位置作为分割点 。   
- gradient: 结合了百分位方法和嵌入距离变化率（梯度）的分析。它试图检测语义距离变化速率的显著改变，可能更适用于语义转换比较平滑、句子间关联度高的文本（如法律或医学文档）。   
  
**关键参数详解**:
- embeddings: 必需，一个 langchain_core.embeddings.Embeddings 的实例，用于生成句子嵌入 。可选用 LangChain 支持的多种嵌入模型 。   
- breakpoint_threshold_type: 上述四种阈值确定方法之一，字符串类型，可选值为 'percentile', 'standard_deviation', 'interquartile', 'gradient' 。   
- breakpoint_threshold_amount: 一个浮点数，其含义取决于 breakpoint_threshold_type。对于 percentile 和 gradient，是 0-100 的百分位数值（默认 95.0）；对于 standard_deviation，是标准差的倍数（默认 3.0）；对于 interquartile，是 IQR 的倍数（默认 1.5）。调整此参数可以控制分割的“激进”程度：值越高，分割越少，块越大。   
- min_chunk_size: 可选整数。用于设置块的最小尺寸。如果语义分割产生的块小于此值，可能会尝试与相邻块合并 。   
- number_of_chunks: 可选整数。尝试将文本分割成指定数量的块，这会影响阈值的动态调整 。   
- sentence_split_regex: 可选字符串。用于将文本分割成句子的正则表达式。
- buffer_size: 可选整数（默认 1）。在计算嵌入差异时，考虑当前句子与前面多少个句子的平均嵌入进行比较，用于平滑差异计算 。


**优缺点**:
- 优点：能够产生语义上更连贯、更符合内容逻辑的块，可能比基于固定规则的分割更能捕捉到文档的自然主题边界 。理论上这有助于提高 RAG 系统的检索相关性和生成质量 。   
- 缺点：计算成本显著更高，因为它需要在每个句子（或句子组）上运行嵌入模型 。分割速度远慢于其他方法。分割效果高度依赖所选嵌入模型的质量以及该模型对特定领域文本语义的理解能力 。参数（尤其是 breakpoint_threshold_amount）的选择可能需要针对不同数据集进行实验和调优，不如其他方法直观 。有时可能产生过大或过小的块 。

**适用场景**: 对块的语义连贯性有非常高要求的应用；处理主题切换明显但缺乏明确结构分隔符（如标题）的长篇文档；希望分割过程能更智能地适应文本内容本身，而不是依赖固定的长度或分隔符规则 。当计算资源充足且愿意投入时间进行参数调优时可以考虑。


**语义分割的深层含义**: 语义分割代表了文档分割技术从基于“形式”（字符数、结构标记）向基于“内容”（语义含义）的转变。这是一种更高级、更智能的策略。然而，它的有效性并非绝对保证，而是**强耦合于嵌入模型的性能**。如果嵌入模型无法准确捕捉文本的细微语义差异、领域特定术语或上下文依赖关系，那么基于嵌入距离的分割决策就可能出错，导致分割效果不佳，甚至可能不如简单的递归分割 。因此，选择一个在目标任务和数据领域表现优异的嵌入模型  是成功应用语义分割的前提。   


In [21]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings # 或者其他嵌入模型，如 HuggingFaceEmbeddings
import os

# 确保设置了 OpenAI API Key 环境变量
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# 示例文本
state_of_the_union = """
女士们先生们，晚上好。今晚，我们齐聚一堂，探讨国家的现状与未来。
经济方面，我们面临挑战，但也看到了增长的机遇。失业率正在下降，制造业正在回暖。
然而，通货膨胀仍然是一个需要解决的问题。医疗保健方面，我们致力于降低成本，扩大覆盖范围。
教育是国家未来的基石，我们将继续投资于学校和教师。外交政策上，我们坚持多边主义，与盟友紧密合作，应对全球性挑战。
气候变化是全人类的威胁，我们必须采取果断行动。总之，虽然挑战重重，但我们对未来充满信心。
"""

# 使用 OpenAI 嵌入模型
embeddings = OpenAIEmbeddings()

# 1. 使用默认的 percentile 阈值
semantic_splitter_percentile = SemanticChunker(
    embeddings, breakpoint_threshold_type="percentile" # breakpoint_threshold_amount 默认为 95.0
)
docs_percentile = semantic_splitter_percentile.create_documents([state_of_the_union])
print(f"Percentile 分割块数: {len(docs_percentile)}")
# for i, doc in enumerate(docs_percentile):
#     print(f"块 {i+1}:\n{doc.page_content}\n{'-'*20}")

# 2. 尝试使用 standard_deviation 阈值，设置更宽松的阈值（例如 2.0）以获得更多块
semantic_splitter_stddev = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=2.0 # 调整阈值
)
docs_stddev = semantic_splitter_stddev.create_documents([state_of_the_union])
print(f"Standard Deviation (2.0) 分割块数: {len(docs_stddev)}")
# for i, doc in enumerate(docs_stddev):
#     print(f"块 {i+1}:\n{doc.page_content}\n{'-'*20}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

#### 如何选择合适的分割策略 (最佳实践)

面对 LangChain 提供的多种文档分割选项，选择最适合特定需求的策略是一个关键决策。以下是一些需要考虑的因素和不同策略的比较，以帮助做出明智的选择。

##### 考虑因素 (Considerations)

在选择分割策略时，应综合考虑以下几个方面：

* **文档类型 (Document Type):** 文档是结构化的（如 Markdown, HTML, 代码）还是非结构化的纯文本？文档的语言是什么？不同的分割器对特定格式有优化（如 `MarkdownHeaderTextSplitter`, `HTMLHeaderTextSplitter`, `.from_language`）。
* **下游任务 (Downstream Task):** 分割后的块将用于什么目的？是 RAG 问答、文档摘要、信息提取还是其他任务？例如，RAG 通常受益于语义连贯性好的块，而摘要任务可能对块的大小和上下文完整性有不同要求。
* **块属性需求 (Chunk Properties):** 对分割后的块有哪些具体要求？是否需要严格控制块的大小（字符数或 Token 数）？语义连贯性有多重要？块之间的上下文保持（通过重叠）需求如何？
* **计算资源与时间 (Computational Resources & Time):** 可用的计算资源（CPU, GPU, 内存）和处理时间是否有限制？语义分割等需要嵌入计算的策略会消耗更多资源和时间。
* **嵌入模型 (Embedding Model):** 如果计划使用语义分割，所选嵌入模型的质量、训练数据、以及它对目标文档领域语言的理解能力如何？这将直接影响语义分割的效果。

##### 不同策略的比较 (Comparison of Strategies)

为了更清晰地对比各种策略，下表总结了 LangChain 中主要分割器的特点：

**Table: LangChain 文档分割策略对比**

| 分割器 (Splitter Class)                       | 核心机制 (Mechanism Summary)                             | 关键参数 (Key Parameters)                                                         | 优点 (Pros)                                                               | 缺点 (Cons)                                                                               | 典型用例 (Typical Use Cases)                                         |
| :-------------------------------------------- | :------------------------------------------------------- | :-------------------------------------------------------------------------------- | :------------------------------------------------------------------------ | :---------------------------------------------------------------------------------------- | :------------------------------------------------------------------- |
| `CharacterTextSplitter`                       | 基于指定的单个字符分隔符分割，然后合并片段至 chunk_size      | `separator`, `chunk_size` (字符), `chunk_overlap` (字符), `length_function`         | 简单、快速、计算开销小                                                    | 可能破坏语义结构，对无规律分隔符的文本效果差，块大小不严格遵守                           | 结构简单、段落分隔清晰的纯文本；快速原型验证；对语义连贯性要求不高     |
| `TokenTextSplitter`                           | 先分词 (Tokenize)，再按 Token 数量分割，最后转回文本       | `encoding_name`/`tokenizer`, `chunk_size` (Token), `chunk_overlap` (Token)        | 精确控制 Token 数，与 LLM 处理方式一致                                      | 可能在词中分割，可读性差，依赖 Tokenizer，速度稍慢                                    | 为 RAG 或 LLM 输入准备数据，严格控制 Token 数量                        |
| `RecursiveCharacterTextSplitter`              | 按分隔符列表递归分割，优先保持段落、句子完整性             | `separators`, `chunk_size` (字符), `chunk_overlap` (字符), `length_function`, `keep_separator` | 较好地保持语义连贯性和文本结构，通用性强，推荐起点                          | 块大小可能不均匀                                                                            | 大多数通用文本分割任务，希望保留原文结构                           |
| `MarkdownHeaderTextSplitter`                  | 基于 Markdown 标题 (#, ## 等) 分割，并添加标题元数据       | `headers_to_split_on`, `strip_headers`                                            | 保持 Markdown 文档逻辑结构，元数据利于下游任务                             | 不直接控制块大小（需结合其他分割器）                                                     | 处理 Markdown 文档，需要利用其章节结构信息                         |
| `HTMLHeaderTextSplitter`                      | 基于 HTML 标题 (h1, h2 等) 分割，并添加标题元数据        | `headers_to_split_on`, `return_each_element`                                      | 保持 HTML 文档逻辑结构，元数据利于下游任务                             | 不直接控制块大小（需结合其他分割器）                                                     | 处理 HTML 文档，需要利用其标题结构信息                           |
| `RecursiveCharacterTextSplitter.from_language` | 使用特定编程语言的预定义分隔符进行递归分割                 | `language`, `chunk_size`, `chunk_overlap`                                         | 智能识别代码结构（函数、类等），分割更符合代码逻辑                          | 仅适用于支持的语言                                                                        | 分割代码文件以进行分析或问答                                         |
| `SemanticChunker` (Experimental)              | 基于句子嵌入向量的语义相似度差异进行分割                 | `embeddings`, `breakpoint_threshold_type`, `breakpoint_threshold_amount`, `min_chunk_size`, `buffer_size`, `number_of_chunks`, `sentence_split_regex` | 产生语义上更连贯的块，更智能地适应内容                                      | 计算成本高，速度慢，效果依赖嵌入模型和参数调优，可能产生大小不均的块                       | 对语义连贯性要求极高；主题切换明显但无结构分隔符的文档；计算资源充足 |

**选择策略的迭代性:** 实践中，并不存在一个适用于所有情况的“最佳”文档分割策略。选择过程往往是一个基于具体应用场景和数据特性进行实验和优化的迭代过程。通常建议从 `RecursiveCharacterTextSplitter` 开始，因为它在通用性和保持文本结构之间取得了较好的平衡。然后，根据初步的分割效果和下游任务（如 RAG）的评估结果，判断是否存在问题（例如，关键信息被切分、检索到的块上下文不足、块大小不合适等）。如果文档具有明确的结构（Markdown, HTML, 代码），可以尝试结合相应的结构化分割器。如果对语义连贯性有更高要求，并且计算资源允许，可以尝试 `SemanticChunker`，并仔细选择嵌入模型和调整阈值参数。最终的选择应基于实证效果。

### 模块三：嵌入与存储个人数据 (Vectorstores and Embeddings) 


#### 嵌入 (Embeddings)
##### 概念与重要性
嵌入是将文本、图像、音频或其他类型的数据转换为机器学习模型和算法可以轻松使用的数值表示形式的过程 。这些表示形式通常是高维向量（浮点数数组），它们捕捉了原始数据的语义信息 。   

将文本转换为向量至关重要，因为计算机本身不理解自然语言的细微差别。传统的文本处理方法，如词袋模型 (Bag of Words) 或 TF-IDF，虽然可以提取特征，但往往难以捕捉词语和句子之间的深层语义关系 。例如，仅仅计算词语出现的频率无法区分具有相似含义但使用不同词语的句子 。   

嵌入通过将具有相似语义含义的文本映射到向量空间中彼此靠近的点来解决这个问题 。这意味着向量之间的距离（通常用相似性度量来衡量）与原始文本之间的语义相似性相关 。例如，“我喜欢狗”和“我喜欢猫”的嵌入向量在空间中的距离会比“我喜欢狗”和“巴黎是法国的首都”的嵌入向量更近 。这种能力使得机器能够像人类一样理解文本的上下文、相似性和类比 。   

这种语义捕捉能力对于 RAG 应用至关重要。通过将文档块和用户查询都转换为嵌入向量，系统可以有效地找到与查询语义最相关的文档块，即使它们不包含完全相同的关键字 。这大大提高了检索结果的准确性和相关性，从而使 LLM 能够生成更准确、更基于事实的响应 。   

##### 语义含义与向量空间
嵌入模型的核心目标是创建一个“语义空间”，其中**向量的位置反映其对应文本的含义** 。在这个高维空间中，每个嵌入向量可以被视为一组坐标 。模型通过学习大量文本数据来调整这些向量，使得在语义上相似的词语或句子（如同义词、相关概念）在空间中彼此靠近 。   

例如，模型可能会学习到“焦虑”和“紧张”这两个词在很多上下文中可以互换使用，因此它们的嵌入向量会非常接近 。同样，“小狗”和“幼犬”的嵌入向量也会很接近，并且系统能够理解查询“给我看一只小狗”与存储的包含“幼犬”图像的向量是相关的 。   

这种空间映射使得我们可以使用简单的数学运算来量化文本之间的相似性，而无需考虑它们的原始长度或结构 。   

##### 余弦相似度 (Cosine Similarity)
在比较嵌入向量时，余弦相似度是一种常用且重要的度量标准 。它通过计算两个向量之间夹角的余弦值来衡量它们的相似性 。   

余弦相似度的值介于 -1 和 1 之间 。   

- 值接近 1 表示两个向量指向非常相似的方向，表明它们代表的文本在语义上高度相关 。   
- 值接近 0 表示两个向量方向大致垂直，表明它们之间几乎没有语义关联 。   
- 值接近 -1 表示两个向量指向相反的方向，表明它们的含义相反（尽管在实践中，嵌入向量之间的余弦相似度很少接近 -1）。   
余弦相似度在文本分析和向量搜索中特别有用，原因如下：

- **关注方向而非幅度**： 它衡量的是向量的方向（语义内容）而非其大小（例如，文档长度或词频）。这使得它对于比较不同长度的文档非常有效 。即使一个文档比另一个文档长得多，如果它们讨论的是相同的主题，它们的嵌入向量之间的角度也会很小，余弦相似度会很高 。   
- **高维空间中的有效性**： 文本嵌入通常是高维的，余弦相似度在高维空间中仍然是一个可靠的度量 。   
- **计算效率**： 对于归一化（长度为 1）的向量，余弦相似度的计算简化为向量的点积，计算上相对高效 。许多嵌入模型（如 Hugging Face 的一些模型）默认输出归一化向量 。   
OpenAI 等提供商明确建议对其嵌入使用余弦相似度 。在 LangChain 中，许多向量存储实现允许选择或默认使用余弦相似度作为其主要的相似性度量 。   

以下是使用 numpy 计算余弦相似度的 Python 示例 ：

In [23]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """计算两个向量之间的余弦相似度"""
    # 确保输入是 numpy 数组
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # 计算点积
    dot_product = np.dot(vec1, vec2)

    # 计算向量的范数（长度）
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # 避免除以零
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0

    # 计算并返回余弦相似度
    return dot_product / (norm_vec1 * norm_vec2)

query_result = [0.1, 0.5, 0.8, 0.2]
document_result = [0.2, 0.6, 0.7, 0.3]
# 示例用法（假设 query_result 和 document_result 是嵌入向量）
similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.9793792286287206


##### LangChain 嵌入模型接口与集成
LangChain 提供了一个统一的 Embeddings 接口，用于与各种嵌入模型提供商进行交互，简化了在不同模型之间切换的过程 。

###### 核心接口方法

该接口定义了两个核心方法 ：   

- `embed_documents(self, texts: List[str]) -> List[List[float]]`: 此方法接收一个文本（文档）列表作为输入，并返回一个嵌入向量列表，其中每个内部列表对应输入列表中的一个文本。它用于批量嵌入文档，通常是为了将它们**存储**在向量数据库中以供后续检索 。   
- `embed_query(self, text: str) -> List[float]`: 此方法接收单个文本（查询）作为输入，并返回其对应的嵌入向量（一个浮点数列表）。它用于嵌入用户**查询**，以便在向量存储中进行相似性搜索 。

将文档嵌入和查询嵌入区分为两个独立的方法是有意为之的，因为一些嵌入提供商可能会针对这两种不同的用例采用不同的嵌入策略或模型 。例如，查询嵌入可能更侧重于捕捉问题的核心意图，而文档嵌入可能更侧重于表示内容的全面信息。   

###### 主要集成提供商

LangChain 支持与众多嵌入模型提供商的集成 。一些关键的提供商包括：   

- OpenAI: 通过 langchain-openai 包和 OpenAIEmbeddings 类提供集成。支持多种模型，如 text-embedding-3-small 和 text-embedding-3-large 。需要设置 OpenAI API 密钥 。   
- Hugging Face: 提供多种集成方式。
    - HuggingFaceEmbeddings: 直接从 Hugging Face Hub 加载模型，例如 sentence-transformers/all-mpnet-base-v2 。   
    - HuggingFaceInstructEmbeddings: 使用 sentence-transformers 库中的指令嵌入模型，如 hkunlp/instructor-large 。   
    - HuggingFaceInferenceAPIEmbeddings: 通过 Hugging Face Inference API 使用模型 。   
本地模型：支持加载本地运行的 Hugging Face 模型 。   
- Cohere: 通过 langchain-cohere 包和 CohereEmbeddings 类提供集成 。需要 Cohere API 密钥 。   
- Google (Vertex AI & Gemini): 通过 langchain-google-vertexai 和 langchain-google-genai 包提供集成 。   
- Azure OpenAI: 通过 langchain-openai 包和 AzureOpenAIEmbeddings 类提供集成 。   
- Ollama: 支持通过 langchain-ollama 包与本地运行的 Ollama 模型（包括一些 Hugging Face 模型）进行交互 。   
其他提供商: LangChain 还集成了许多其他提供商，如 AI21, Aleph Alpha, Bedrock, Baidu Qianfan, MistralAI, Nomic, VoyageAI, IBM watsonx.ai, NVIDIA NIMs 等 。   

###### DeepSeek 嵌入模型状态

目前，LangChain 的核心库或官方 langchain-deepseek 包**并未直接提供 DeepSeek 的嵌入模型集成** 。langchain-deepseek 包主要关注其聊天模型（如 deepseek-chat 和 deepseek-reasoner）。   

然而，有以下几种方式可以使用 DeepSeek 或与其相关的嵌入：

- **通过第三方平台**:
    - BytePlus ModelArk: 提供了 DeepSeekEmbedding 接口，声称可以免费使用 DeepSeek 嵌入，并与 LangChain 集成 。用户需要注册 BytePlus API 密钥 。   
    - Fireworks AI: 支持 DeepSeek 模型，包括一些可能用于嵌入的变体（如 deepseek-r1-distill-qwen-7b，尽管主要用于聊天）。可以通过 langchain-fireworks 包访问 。   
    - Together AI: 同样支持 DeepSeek 模型，可以通过 langchain-together 包访问 。   
- **通过本地模型运行器 (Ollama)**: DeepSeek 的某些模型（如 deepseek-r1 系列）可以在本地通过 Ollama 运行 。然后可以使用 langchain-ollama 包中的 OllamaEmbeddings 来生成嵌入 。   
- **使用其他兼容模型**: DeepSeek R1 模型本身似乎更侧重于推理和聊天，而不是专门的嵌入生成 。GitHub issue 中提到 deepseek-r1:1.5b 不支持工具调用，这通常与聊天/代理功能相关，而非嵌入 。在 RAG 场景中，通常会选择专门的嵌入模型（如 OpenAI, Hugging Face BGE/Instructor, Nomic 等）与 DeepSeek 聊天/推理模型（如 deepseek-chat 或 deepseek-reasoner）结合使用 。    

###### 选择嵌入模型

选择哪个嵌入模型取决于具体需求，包括性能、成本、数据隐私和易用性。MTEB (Massive Text Embedding Benchmark) 排行榜是比较不同嵌入模型性能的常用资源 。对于需要处理个人数据的场景，使用本地模型（如通过 Hugging Face 或 Ollama）可以提供更高的数据隐私保障 。   

**示例：Ollama**

In [25]:
import os
# 不再需要 getpass，因为本地 Ollama 不需要 API Key
# import getpass
# from langchain_openai import OpenAIEmbeddings # 替换为 OllamaEmbeddings
from langchain_community.embeddings import OllamaEmbeddings # 导入 OllamaEmbeddings

# --- 不再需要 OpenAI API Key 部分 ---
# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("输入 OpenAI API Key:")

# --- 初始化 Ollama 嵌入模型 ---
# 确保 Ollama 服务正在本地运行 (通常在 http://localhost:11434)
# 将 "nomic-embed-text" 替换为你通过 ollama pull 下载并想使用的模型名称
embeddings_model = OllamaEmbeddings(
    model="nomic-embed-text" # 或者 "mxbai-embed-large", "all-minilm", 等
    # OllamaEmbeddings 通常不需要 'dimensions' 参数
)
print(f"使用 Ollama 模型: {embeddings_model.model}")

# --- 后续代码与之前类似 ---

# 嵌入文档列表
documents = ["这是第一个文档。", "这是第二个文档，内容不同。"] # 修改了第二个文档以更好地区分
doc_embeddings = embeddings_model.embed_documents(documents)
print(f"生成的文档嵌入数量: {len(doc_embeddings)}")
if doc_embeddings:
    # 注意：这里应该是获取第一个嵌入向量的维度
    print(f"第一个文档嵌入的维度: {len(doc_embeddings[0])}")
    # print(f"第一个文档嵌入的前5个维度: {doc_embeddings[0][:5]}") # 可以取消注释查看

# 嵌入单个查询
query = "这些文档的主要主题是什么？" # 稍微修改了查询
query_embedding = embeddings_model.embed_query(query)
print(f"查询嵌入的维度: {len(query_embedding)}")
# print(f"查询嵌入的前5个维度: {query_embedding[:5]}") # 可以取消注释查看

# --- 可以添加余弦相似度计算来比较查询和文档 ---
import numpy as np

def cosine_similarity(vec1, vec2):
    """计算两个向量之间的余弦相似度"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)

if doc_embeddings:
    print("\n查询与各文档的相似度:")
    for i, doc_emb in enumerate(doc_embeddings):
        similarity = cosine_similarity(query_embedding, doc_emb)
        print(f"  与文档 {i+1} ('{documents[i]}') 的相似度: {similarity:.4f}")

C:\Users\14798\AppData\Local\Temp\ipykernel_34360\1876706944.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings_model = OllamaEmbeddings(


使用 Ollama 模型: nomic-embed-text
生成的文档嵌入数量: 2
第一个文档嵌入的维度: 768
查询嵌入的维度: 768

查询与各文档的相似度:
  与文档 1 ('这是第一个文档。') 的相似度: 0.6668
  与文档 2 ('这是第二个文档，内容不同。') 的相似度: 0.6187


**示例：Hugging Face**

In [1]:
!pip install sentence-transformers langchain-huggingface langchain-community
!pip install torch
!pip install transformers[torch]
!pip install -U transformers datasets evaluate accelerate timm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
     ---------------------------------------- 2.4/2.4 MB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
     --- ------------------------------------ 2.4/25.7 MB 12.2 MB/s eta 0:00:02
     ------- -------------------------------- 5.0/25.7 MB 12.1 MB/s eta 0:00:02
     ----------- ---------------------------- 7.3/25.7 MB 11.9 MB/s eta 0:00:02
     --------------- ----------------------- 10.0/25.7 MB 11.9 MB/s eta 0:00:02
     ------------------- ------------------- 12.6/25.7 MB 11.8 MB/s eta 0:00:02
     ---------------------- ---------------- 14.9/25.7 MB 11.9 MB/s eta 0:00:01
     -------------------------- ------------ 17.6/25.7

- [miniLM](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
- 下载地址:https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/

In [2]:
# 确保已安装必要的库:
# pip install sentence-transformers langchain-huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# --- 1. 配置嵌入模型 ---

# !! 重要 !!
# 将下面的 'path/to/your/local/all-MiniLM-L6-v2' 替换为你实际存放模型文件的文件夹路径。
# 例如：'C:/Users/YourUser/Downloads/all-MiniLM-L6-v2' 或 '/home/user/models/all-MiniLM-L6-v2'
local_model_directory = "./all-MiniLM-L12-v2" # <--- 修改这里！

# model_name 参数现在直接使用本地文件夹路径
model_name = local_model_directory

# 其他参数保持不变
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

# --- 2. 初始化嵌入模型 (从本地路径加载) ---
try:
    # LangChain/SentenceTransformers 会识别 model_name 是一个本地路径
    # 并尝试从该路径加载模型，而不是从网上下载
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # 更新打印信息，表明是从本地加载
    print(f"成功从本地路径加载 HuggingFace embedding 模型: {model_name}")
except Exception as e:
    # 更新错误信息
    print(f"从本地路径 '{model_name}' 加载模型时出错: {e}")
    print("请确认路径正确，文件夹内包含完整的模型文件 (如 config.json, pytorch_model.bin 等)。")
    exit() # 如果模型加载失败则退出

# --- 3. 准备示例文本 ---
documents = [
    "这是一篇关于人工智能和机器学习的文章。",
    "今天天气真好，适合出去散步。",
    "机器学习是人工智能的一个分支。",
    "如何烹饪美味的意大利面？"
]
query = "告诉我有关机器学习的信息"
print(f"\n用于演示的查询文本: '{query}'")

# --- 4. 生成嵌入 (使用本地加载的模型) ---
# 文档嵌入
try:
    doc_embeddings = hf_embeddings.embed_documents(documents)
    print(f"   (已使用本地模型生成 {len(doc_embeddings)} 个文档嵌入)")
except Exception as e:
    print(f"   生成文档嵌入时出错: {e}")
    doc_embeddings = []

# 查询嵌入
try:
    query_embedding = hf_embeddings.embed_query(query)
    print(f"   (已使用本地模型生成查询嵌入)")
except Exception as e:
    print(f"   生成查询嵌入时出错: {e}")
    query_embedding = None

# --- 5. 显示查询嵌入信息 (与之前相同) ---
if query_embedding:
    query_dim = len(query_embedding)
    print(f"\n查询嵌入维度: {query_dim}")
    num_values_to_show = 5
    embedding_preview = [f"{val:.4f}" for val in query_embedding[:num_values_to_show]]
    print(f"查询嵌入向量 (前 {num_values_to_show} 个值示例): {embedding_preview}")
    print("\n说明: 这个向量由本地加载的模型生成，代表查询的语义。")
else:
    print("\n未能生成查询嵌入。")

# (可选：显示文档嵌入形状)
if doc_embeddings:
    num_docs = len(doc_embeddings)
    embedding_dim_docs = len(doc_embeddings[0])
    print(f"\n(补充信息) 文档嵌入形状: ({num_docs}, {embedding_dim_docs})")

成功从本地路径加载 HuggingFace embedding 模型: ./all-MiniLM-L12-v2

用于演示的查询文本: '告诉我有关机器学习的信息'
   (已使用本地模型生成 4 个文档嵌入)
   (已使用本地模型生成查询嵌入)

查询嵌入维度: 384
查询嵌入向量 (前 5 个值示例): ['0.0254', '0.0829', '-0.0240', '-0.0715', '-0.0129']

说明: 这个向量由本地加载的模型生成，代表查询的语义。

(补充信息) 文档嵌入形状: (4, 384)


**示例：HuggingFaceInstructEmbeddings**


In [5]:
!set GIT_LFS_SKIP_SMUDGE=1 
!git lfs install
!git clone git@hf.co:hkunlp/instructor-large

!pip install langchain-community InstructorEmbedding transformers sentence-transformers torch --upgrade --index-url https://pypi.org/simple

# 示例失败跳过

Git LFS initialized.


fatal: destination path 'instructor-large' already exists and is not an empty directory.


In [2]:
# 确保已安装必要的库:
# pip install InstructorEmbedding sentence_transformers langchain-community torch
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
import os # Optional, mainly for path joining if needed

# --- 1. 配置嵌入模型 (从本地加载) ---

# !! 重要 !!
# 将下面的 'path/to/your/local/instructor-large' 替换为你实际存放模型文件的文件夹路径！
# 例如：'C:/Users/YourUser/Downloads/instructor-large' 或 '/home/user/models/instructor-large'
local_model_directory = "./instructor-large" # <--- 修改这里！

# model_name 参数现在直接使用本地文件夹路径
model_name = local_model_directory

# 其他 Instructor 相关参数保持不变
model_kwargs = {'device': 'cpu'} # 指定设备
# 指令对于 Instructor 模型很重要
embed_instruction = "为检索任务表示以下文档: " # 文档指令 (保持与原意一致)
query_instruction = "为检索相关支持文档表示以下问题: " # 查询指令 (保持与原意一致)

# --- 2. 初始化嵌入模型 (从本地路径加载) ---
try:
    # HuggingFaceInstructEmbeddings 会尝试从提供的本地路径加载模型
    # 它依赖 InstructorEmbedding 库来处理加载
    instructor_embeddings = HuggingFaceInstructEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        embed_instruction=embed_instruction,
        query_instruction=query_instruction
    )
    # 更新打印信息
    print(f"成功从本地路径加载 Instructor embedding 模型: {model_name}")
except Exception as e:
    # 更新错误信息
    print(f"从本地路径 '{model_name}' 加载模型时出错: {e}")
    print("请确认路径正确，文件夹内包含完整的 Instructor 模型文件，并且已安装 'InstructorEmbedding' 库。")
    exit() # 如果模型加载失败则退出

# --- 3. 准备示例文本 (使用更具体的例子) ---
documents = [
    "机器学习是人工智能的一个重要分支，它使计算机能够从数据中学习。",
    "自然语言处理关注计算机与人类语言之间的交互。"
    ]
query = "机器学习和自然语言处理之间有什么联系？"

# --- 4. 生成嵌入 (使用本地加载的模型) ---
# 文档嵌入
try:
    doc_embeddings = instructor_embeddings.embed_documents(documents)
    print(f"\n已使用本地 Instructor 模型生成 {len(doc_embeddings)} 个文档嵌入。")
except Exception as e:
    print(f"   生成文档嵌入时出错: {e}")
    doc_embeddings = []

# 查询嵌入
try:
    query_embedding = instructor_embeddings.embed_query(query)
    print(f"已使用本地 Instructor 模型生成查询嵌入。")
except Exception as e:
    print(f"   生成查询嵌入时出错: {e}")
    query_embedding = None

# --- 5. 打印嵌入的形状/维度 (修正文档形状打印) ---
if doc_embeddings:
    # 修正: 形状是 (文档数量, 嵌入维度)
    num_docs = len(doc_embeddings)
    # 假设所有嵌入维度相同，取第一个的维度来确定嵌入维度
    if num_docs > 0 and isinstance(doc_embeddings[0], list):
         embedding_dim = len(doc_embeddings[0])
         print(f"\nInstructor 文档嵌入形状: ({num_docs}, {embedding_dim})")
    else:
         print("\n无法确定 Instructor 文档嵌入的维度。") # 处理空列表或非列表元素的情况
else:
    print("\n未能生成 Instructor 文档嵌入。")

if query_embedding:
    # 查询嵌入维度
    query_dim = len(query_embedding)
    print(f"Instructor 查询嵌入维度: {query_dim}")
else:
    print("未能生成 Instructor 查询嵌入。")

尝试直接使用 InstructorEmbedding 加载...
错误：直接加载也失败，并提示 'token' 参数错误！这很不寻常，可能需要检查 InstructorEmbedding 库本身或其依赖。


#### 向量存储 (Vector Stores)
向量存储负责持久化嵌入向量，并提供高效的检索机制，是 RAG 系统中连接嵌入和生成步骤的关键桥梁 。   


##### 核心 VectorStore 接口
LangChain 通过 langchain_core.vectorstores.base.VectorStore 类定义了与向量存储交互的标准接口 。这使得开发者可以在不同的向量数据库实现（如 FAISS, Chroma, Pinecone 等）之间切换，而只需对代码进行最小程度的修改（主要在初始化和特定功能调用上）。这种抽象简化了核心 RAG 逻辑的开发，但开发者仍需了解特定提供商的设置和高级功能。   

VectorStore 接口的关键方法包括 ：   

- `add_documents(documents: List[Document], ids: Optional[List[str]] = None, **kwargs)`: 接收 LangChain Document 对象列表，计算嵌入并将其添加到向量存储中。Document 对象包含 page_content 和 metadata 。强烈建议提供唯一的 ids 列表，以便后续更新或删除文档，避免重复添加 。   
- `add_texts(texts: Iterable[str], metadatas: Optional[List[dict]] = None, ids: Optional[List[str]] = None, **kwargs)`: 类似于 add_documents，但直接接收原始文本字符串列表，并允许关联可选的元数据和 ID 。   
- `delete(ids: Optional[List[str]] = None, **kwargs)`: 根据提供的 ID 列表删除向量存储中的向量 。   
- `similarity_search(query: str, k: int = 4, filter: Optional[dict] = None, **kwargs)`: 最常用的搜索方法。它接收一个文本查询，将其嵌入，然后在向量存储中查找最相似的 k 个文档。可以通过 filter 参数基于元数据进行过滤 。k 控制返回结果数量，filter 用于精确匹配元数据字段。   
- `similarity_search_with_score(...)`: 与 similarity_search 类似，但额外返回每个文档与查询的相似度分数。分数的解释取决于向量存储使用的距离度量（例如，FAISS 默认使用 L2 距离，分数越低越好；Pinecone/OpenAI 常用的余弦相似度，分数越高越好）。   
- `similarity_search_by_vector(...)`: 接收一个预先计算好的查询嵌入向量，而不是文本查询，然后执行相似性搜索 。   
- `max_marginal_relevance_search(...) (MMR)`: 一种更高级的搜索策略，旨在优化检索结果的相关性（与查询的相似度）和多样性（避免返回内容过于相似的文档块）。   
- `as_retriever(**kwargs)`: 将向量存储实例转换为 VectorStoreRetriever 对象。这是将向量存储集成到 LangChain 表达式语言 (LCEL) 链中的标准方式 。可以通过 kwargs 指定搜索类型（search_type）和搜索参数（search_kwargs）。

该接口同样支持异步操作，对应的方法名前缀为 a（例如 aadd_documents, asimilarity_search）。   

##### 关键向量存储实现
LangChain 支持多种向量存储实现 。以下是三个常用且在文档中频繁讨论的实现：FAISS、Chroma 和 Pinecone。   

###### 1. FAISS (langchain_community.vectorstores.FAISS)
FAISS (Facebook AI Similarity Search) 是一个用于高效相似性搜索和密集向量聚类的库，尤其擅长处理大规模数据集 。它主要在内存中运行，但 LangChain 提供了将其索引和文档存储保存到本地磁盘的功能。   

**安装**：需要安装 langchain-community 和 faiss-cpu (或 faiss-gpu 用于 GPU 加速) 。 

**初始化**：
- 从文档创建 (推荐)：FAISS.from_documents(docs, embeddings) 是最简单的方式，它会自动创建内存中的 FAISS 索引 (faiss.IndexFlatL2 默认使用 L2 距离) 和文档存储 (InMemoryDocstore) 。   
- 手动初始化：需要提供 embedding_function (嵌入函数)、index (一个 faiss.Index 对象)、docstore (一个 Docstore 对象，如 InMemoryDocstore) 和 index_to_docstore_id (一个整数索引到文档 ID 的映射字典) 。这提供了更多对索引类型和存储的控制。   

**持久化**：
- `save_local(folder_path)`：将 FAISS 索引 (index.faiss) 和文档存储/ID 映射 (index.pkl) 保存到指定文件夹 。   
- `load_local(folder_path, embeddings, allow_dangerous_deserialization=True)`：从本地文件夹加载 FAISS 实例。需要提供相同的嵌入函数。allow_dangerous_deserialization=True 参数在某些环境中可能是必需的，因为它涉及 pickle 反序列化 。

代码示例 (初始化、添加、搜索、保存/加载)：

In [3]:
# 安装: pip install langchain-community faiss-cpu ollama # 或 faiss-gpu
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
# --- 修改开始: 导入 OllamaEmbeddings ---
from langchain_community.embeddings import OllamaEmbeddings
# --- 修改结束 ---
from langchain_core.documents import Document
import os
import shutil # 用于清理保存目录

# --- 修改开始: 配置 Ollama ---
# 确保 Ollama 服务正在运行!
# 指定您在 Ollama 中使用的嵌入模型名称
# 例如: "nomic-embed-text", "mxbai-embed-large", "all-minilm" 等
ollama_model_name = "nomic-embed-text" # <--- 在这里修改为您使用的模型
embeddings = OllamaEmbeddings(model=ollama_model_name)
print(f"使用 Ollama 嵌入模型: {ollama_model_name}")
# --- 修改结束 ---

# --- 示例 Document 列表 (您需要替换为您自己的文档) ---
docs_to_add = [
    Document(page_content="FAISS (Facebook AI Similarity Search) 是一个用于高效相似性搜索和稠密向量聚类的库。"),
    Document(page_content="Ollama 允许在本地运行大型语言模型，包括嵌入模型。"),
    Document(page_content="LangChain 提供与各种嵌入模型和向量存储集成的工具。")
]
# --- 示例结束 ---

if not docs_to_add:
    raise ValueError("请先定义 'docs_to_add' 列表!")

# 选项 1: 从文档列表创建 (最简单)
vector_store = FAISS.from_documents(docs_to_add, embeddings)
# 检查索引中的向量数量 (需要 FAISS 索引对象)
# 注意：FAISS 的 ntotal 属性在通过 from_documents 创建时可能不直接反映文档数，
# 但代表了索引中的向量总数。
print(f"FAISS 索引已创建。索引中的向量数量: {vector_store.index.ntotal}")

# 相似性搜索
query = "FAISS 是什么？"
results = vector_store.similarity_search(query, k=1)
print(f"\n对于查询 '{query}' 的相似性搜索结果: {results}")

# 带分数的相似性搜索 (L2 距离，分数越低越相似)
results_with_scores = vector_store.similarity_search_with_score(query, k=1)
print(f"带分数的相似性搜索结果: {results_with_scores}")

# 本地保存
save_path = "my_faiss_index_ollama_cn"
# 清理旧目录（如果存在）
if os.path.exists(save_path):
    shutil.rmtree(save_path)
vector_store.save_local(save_path)
print(f"\nFAISS 索引已保存至 {save_path}")

# 本地加载
# 注意: allow_dangerous_deserialization=True 可能需要，取决于环境/版本
loaded_vector_store = FAISS.load_local(save_path, embeddings, allow_dangerous_deserialization=True)
print(f"FAISS 索引已从 {save_path} 加载")
loaded_results = loaded_vector_store.similarity_search(query, k=1)
print(f"从加载的索引中搜索结果: {loaded_results}")

# 检查加载后的存储中的文档数量
print(f"加载后的存储向量计数: {loaded_vector_store.index.ntotal}")

# 清理保存的目录 (根据需要取消注释)
# print(f"\n正在清理目录 {save_path}...")
# shutil.rmtree(save_path)
# print("清理完成。")

C:\Users\14798\AppData\Local\Temp\ipykernel_32924\591854810.py:17: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=ollama_model_name)


使用 Ollama 嵌入模型: nomic-embed-text
FAISS 索引已创建。索引中的向量数量: 3

对于查询 'FAISS 是什么？' 的相似性搜索结果: [Document(id='790f6dae-bb25-4b46-801d-0eecb17a6505', metadata={}, page_content='FAISS (Facebook AI Similarity Search) 是一个用于高效相似性搜索和稠密向量聚类的库。')]
带分数的相似性搜索结果: [(Document(id='790f6dae-bb25-4b46-801d-0eecb17a6505', metadata={}, page_content='FAISS (Facebook AI Similarity Search) 是一个用于高效相似性搜索和稠密向量聚类的库。'), np.float32(385.7163))]

FAISS 索引已保存至 my_faiss_index_ollama_cn
FAISS 索引已从 my_faiss_index_ollama_cn 加载
从加载的索引中搜索结果: [Document(id='790f6dae-bb25-4b46-801d-0eecb17a6505', metadata={}, page_content='FAISS (Facebook AI Similarity Search) 是一个用于高效相似性搜索和稠密向量聚类的库。')]
加载后的存储向量计数: 3


###### 2. Chroma (langchain_chroma.Chroma)

Chroma 是一个以开发者体验为中心的 AI 原生开源向量数据库 。它支持多种运行模式：纯内存、内存加持久化、客户端/服务器模式 。   

安装：需要安装 langchain-chroma 和 chromadb 。   

初始化：
- 内存模式：Chroma(collection_name="...", embedding_function=...) 。   
- 持久化模式：通过 persist_directory 参数指定本地保存路径。Chroma 会尽力自动保存，但最佳实践是确保每个路径只有一个客户端实例在运行 。例如：Chroma(collection_name="...", embedding_function=..., persist_directory="./chroma_db")。   
- 从文档创建 (并持久化)：Chroma.from_documents(docs, embeddings, persist_directory="...") 。   
- 加载持久化数据：再次使用 persist_directory 初始化 Chroma 实例即可加载 。例如：Chroma(persist_directory="./chroma_db", embedding_function=...)。   
- 从客户端连接：可以先创建 chromadb.HttpClient (连接远程服务器) 或 chromadb.PersistentClient (本地持久化)，然后将 client 和 collection_name 传递给 Chroma 初始化函数 。   

代码示例 (持久化初始化、添加、搜索、加载)：

In [5]:
# 安装: pip install langchain-chroma chromadb langchain-community ollama
from langchain_chroma import Chroma
# --- 修改开始: 导入 OllamaEmbeddings ---
from langchain_community.embeddings import OllamaEmbeddings
# --- 修改结束 ---
from langchain_core.documents import Document
import os
import shutil # 用于清理持久化目录

# --- 修改开始: 配置 Ollama ---
# 确保 Ollama 服务正在运行!
# 指定您在 Ollama 中使用的嵌入模型名称
# 例如: "nomic-embed-text", "mxbai-embed-large", "all-minilm" 等
ollama_model_name = "nomic-embed-text" # <--- 在这里修改为您使用的模型
embeddings = OllamaEmbeddings(model=ollama_model_name)
print(f"使用 Ollama 嵌入模型: {ollama_model_name}")
# --- 修改结束 ---

# --- 修改: 可以更改持久化目录名称以区分 ---
persist_directory = "./my_chroma_db_ollama_cn"
collection_name = "my_documents_ollama_cn" # 也可以更改集合名称

# 清理之前的运行（如果需要）
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"已清理旧的持久化目录: {persist_directory}")

# 选项 1: 初始化并指定持久化目录
print(f"\n初始化 Chroma 并持久化到: {persist_directory}")
vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings, # 使用 Ollama embeddings
    persist_directory=persist_directory
)

# 添加文档 (提供 ID 是好习惯)
docs_to_add = [
    Document(page_content="Chroma 是一个开源的嵌入数据库。", metadata={"type": "db", "source": "web"}),
    Document(page_content="Ollama 使得在本地运行模型变得容易。", metadata={"type": "tool", "source": "local"}),
    Document(page_content="LangChain 利用 Chroma 进行 RAG。", metadata={"type": "integration", "source": "framework"}),
]
ids = [f"doc_ollama_cn_{i}" for i in range(len(docs_to_add))]
vector_store.add_documents(docs_to_add, ids=ids)
# Chroma 在使用 persist_directory 初始化时通常会自动持久化更改
print(f"已向 Chroma 添加 {vector_store._collection.count()} 个文档。") # 获取文档计数

# # 选项 2: 从文档创建并持久化 (如果您倾向于这种方式)
# print(f"\n从文档创建 Chroma 并持久化到: {persist_directory}")
# vector_store = Chroma.from_documents(
#     docs_to_add,
#     embeddings, # 使用 Ollama embeddings
#     ids=ids,
#     persist_directory=persist_directory,
#     collection_name=collection_name
# )
# print(f"已从文档创建 Chroma，包含 {vector_store._collection.count()} 个文档。")

# 相似性搜索 (无过滤器)
query = "告诉我关于本地模型的信息"
results = vector_store.similarity_search(query, k=2) # 查找最相似的 2 个文档
print(f"\n对于查询 '{query}' 的相似性搜索结果 (无过滤器): {results}")

# 带元数据过滤器的相似性搜索
query_filtered = "关于数据库的信息"
results_filtered = vector_store.similarity_search(
    query_filtered,
    k=1,
    filter={"type": "db"} # 仅查找元数据中 type 为 'db' 的文档
)
print(f"\n对于查询 '{query_filtered}' 带过滤器 ('type': 'db') 的相似性搜索结果: {results_filtered}")


# --- 加载持久化数据 ---
print(f"\n从以下位置加载 Chroma: {persist_directory}")
# 确保嵌入函数一致
loaded_vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings, # 必须使用相同的嵌入函数
    collection_name=collection_name # 加载时最好也指定集合名称
)
print(f"已加载 {loaded_vector_store._collection.count()} 个文档。")

# 在加载后的存储上执行搜索
loaded_results = loaded_vector_store.similarity_search(query, k=1)
print(f"从加载的 Chroma 存储中搜索结果 ('{query}'): {loaded_results}")

# 使用 get() 检查文档是否存在
existing_doc = loaded_vector_store.get(ids=["doc_ollama_cn_1"], include=["metadatas", "documents"])
print(f"\n获取文档 'doc_ollama_cn_1': {existing_doc}")
non_existing_doc = loaded_vector_store.get(ids=["doc_ollama_cn_99"])
print(f"获取文档 'doc_ollama_cn_99': {non_existing_doc}") # 会返回空列表

# 清理持久化目录 (根据需要取消注释)
# print(f"\n正在清理目录 {persist_directory}...")
# shutil.rmtree(persist_directory)
# print("清理完成。")

使用 Ollama 嵌入模型: nomic-embed-text

初始化 Chroma 并持久化到: ./my_chroma_db_ollama_cn
已向 Chroma 添加 3 个文档。

对于查询 '告诉我关于本地模型的信息' 的相似性搜索结果 (无过滤器): [Document(id='doc_ollama_cn_1', metadata={'source': 'local', 'type': 'tool'}, page_content='Ollama 使得在本地运行模型变得容易。'), Document(id='doc_ollama_cn_2', metadata={'source': 'framework', 'type': 'integration'}, page_content='LangChain 利用 Chroma 进行 RAG。')]

对于查询 '关于数据库的信息' 带过滤器 ('type': 'db') 的相似性搜索结果: [Document(id='doc_ollama_cn_0', metadata={'source': 'web', 'type': 'db'}, page_content='Chroma 是一个开源的嵌入数据库。')]

从以下位置加载 Chroma: ./my_chroma_db_ollama_cn
已加载 3 个文档。
从加载的 Chroma 存储中搜索结果 ('告诉我关于本地模型的信息'): [Document(id='doc_ollama_cn_1', metadata={'source': 'local', 'type': 'tool'}, page_content='Ollama 使得在本地运行模型变得容易。')]

获取文档 'doc_ollama_cn_1': {'ids': ['doc_ollama_cn_1'], 'embeddings': None, 'documents': ['Ollama 使得在本地运行模型变得容易。'], 'uris': None, 'data': None, 'metadatas': [{'source': 'local', 'type': 'tool'}], 'included': [<IncludeEnum.documents: 'documents'>, <Inc

###### 3. Milvus
Milvus 像一个需要连接远程服务器（或者本地运行的服务器）的大型“连锁仓库”，适合更大规模或需要多人协作的场景。

In [ ]:
!pip install langchain-community langchain-core faiss-cpu chromadb pymilvus ollama

In [8]:
# --- Milvus 示例 ---
print("\n--- 开始 Milvus 示例 ---")
from langchain_community.vectorstores import Milvus
from pymilvus import utility, connections

# 1. 定义 Milvus 连接参数和集合名
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"
milvus_collection = "simple_milvus_docs_cn"
connection_args = { "host": MILVUS_HOST, "port": MILVUS_PORT }

# 2. (尝试性清理) 连接并尝试删除旧集合 (如果存在)
print(f"尝试连接 Milvus ({MILVUS_HOST}:{MILVUS_PORT}) 并清理旧集合 '{milvus_collection}'...")
try:
    connections.connect("default", **connection_args)
    if utility.has_collection(milvus_collection):
        utility.drop_collection(milvus_collection)
        print(f"旧集合 '{milvus_collection}' 已删除。")
        time.sleep(1) # 等待删除生效
    connections.disconnect("default")
except Exception as e:
    print(f"连接或清理时出错 (可能是 Milvus 未运行?): {e}")
    # 如果无法连接，后续步骤会失败，这里可以选择退出或让它继续尝试
    # exit()

# 3. 从文档创建 Milvus 集合 (如果 Milvus 运行正常)
print(f"正在创建 Milvus 集合 '{milvus_collection}' 并添加文档...")
milvus_store = None # 先设为 None
try:
    milvus_store = Milvus.from_documents(
        docs,
        ollama_embed,
        collection_name=milvus_collection,
        connection_args=connection_args,
    )
    print("Milvus 集合创建并添加文档完成。")
    # Milvus 需要时间处理数据
    print("等待 Milvus 处理...")
    milvus_store.col.flush() # 确保数据写入
    time.sleep(3) # 短暂等待
    print(f"集合中的实体数: {milvus_store.col.num_entities}")
except Exception as e:
    print(f"创建 Milvus 集合失败: {e}")
    print("请确保 Milvus 服务器正在运行并且网络可达！")

# 4. 进行相似性搜索 (仅当集合创建成功时)
if milvus_store:
    print(f"在 Milvus 中搜索 '{query}'...")
    results_milvus = milvus_store.similarity_search(query, k=1)
    print(f"Milvus 搜索结果: {results_milvus}")

    # 5. (演示重新连接) 模拟重新加载/连接
    print("尝试重新连接到 Milvus 集合...")
    try:
        reconnected_milvus = Milvus(
            embedding_function=ollama_embed,
            collection_name=milvus_collection,
            connection_args=connection_args,
        )
        reconnected_milvus.col.load() # 加载集合到内存以供搜索
        print(f"已重新连接，集合实体数: {reconnected_milvus.col.num_entities}")
        reconnected_results = reconnected_milvus.similarity_search(query, k=1)
        print(f"从重新连接的 Milvus 存储中搜索结果: {reconnected_results}")
    except Exception as e:
        print(f"重新连接 Milvus 失败: {e}")
else:
    print("由于 Milvus 集合未能成功创建，跳过搜索和重连步骤。")


# (可选清理 - Milvus) 取消注释以在脚本结束时删除集合
# print(f"\n尝试删除 Milvus 集合 '{milvus_collection}'...")
# try:
#     connections.connect("cleanup", **connection_args)
#     if utility.has_collection(milvus_collection):
#         utility.drop_collection(milvus_collection)
#         print("Milvus 集合已删除。")
#     connections.disconnect("cleanup")
# except Exception as e:
#     print(f"删除 Milvus 集合时出错: {e}")

print("--- Milvus 示例结束 ---")
# --- Milvus 示例结束 ---

2025-05-05 16:49:14,155 [ERROR][handler]: RPC error: [__internal_register], <MilvusException: (code=1, message=Incorrect port or sdk is incompatible with server, please check your port or downgrade your sdk or upgrade your server)>, <Time:{'RPC start': '2025-05-05 16:49:14.105060', 'RPC error': '2025-05-05 16:49:14.155068'}> (decorators.py:140)



--- 开始 Milvus 示例 ---
尝试连接 Milvus (localhost:19530) 并清理旧集合 'simple_milvus_docs_cn'...
连接或清理时出错 (可能是 Milvus 未运行?): <MilvusException: (code=1, message=Incorrect port or sdk is incompatible with server, please check your port or downgrade your sdk or upgrade your server)>
正在创建 Milvus 集合 'simple_milvus_docs_cn' 并添加文档...
创建 Milvus 集合失败: name 'docs' is not defined
请确保 Milvus 服务器正在运行并且网络可达！
由于 Milvus 集合未能成功创建，跳过搜索和重连步骤。
--- Milvus 示例结束 ---


### 模块四：查找正确的信息 (Retrieval)

#### 语义相似性搜索

语义相似性搜索是向量数据库的核心能力，也是 RAG 系统检索的基础。其工作原理如下：

##### 嵌入与语义含义
嵌入模型的核心作用是将具有相似语义含义的文本块映射到向量空间中彼此靠近的位置 。这意味着，即使两个文本块使用的具体词语不同，但如果它们表达的意思相近（例如，“如何修理漏水的水管”和“管道维修”或“漏水解决方案”），它们的嵌入向量在空间中的距离也会很近。这使得检索能够超越传统的关键词匹配，实现基于“意义”的查找 。当用户提出查询时，该查询也会被同一个嵌入模型转换成一个查询向量 。   

##### 量化相似性：距离度量
为了在向量空间中客观地衡量两个嵌入向量（例如，查询向量和文档块向量）的相似程度，需要使用距离度量（Distance Metrics） 。常用的距离度量包括：   
- 余弦相似度（Cosine Similarity）：测量两个向量之间的夹角的余弦值。它关注向量的方向而非大小（长度），因此对于长度不一但语义相似的文本（例如，一个短句和一个长段落）特别有效 。其值域通常在 `[-1, 1]` 或 之间，值越接近 1 表示相似度越高。   
- 欧氏距离（Euclidean Distance, L2 Distance）：测量向量空间中两个向量端点之间的直线距离 。它同时考虑了向量的大小和方向。其值域为 [0, ∞)，值越小表示相似度越高。   
- 点积（Dot Product）：计算两个向量的乘积之和。对于归一化（长度为 1）的向量，点积等价于余弦相似度 。其值域为 [-∞, ∞)，正值表示方向大致相同，负值表示方向大致相反。

##### 高效搜索的索引技术

将查询向量与数据库中存储的每一个文档块向量进行比较（即暴力 K 最近邻搜索，Brute-force kNN）在数据量巨大时（例如数十亿向量）会变得极其缓慢且计算成本高昂 。为了实现大规模、低延迟的相似性搜索，向量数据库采用了**近似最近邻（Approximate Nearest Neighbor, ANN）** 算法和相应的索引结构 。   

ANN 算法的核心思想是通过牺牲一定的精度（即不保证找到绝对最近的邻居，但能以高概率找到非常接近的邻居）来换取搜索速度的大幅提升 。常见的 ANN 索引技术包括：   

- **基于图的方法（Graph-based）**：例如 HNSW (Hierarchical Navigable Small World)。它构建一个多层图结构，在高层进行粗粒度导航，在低层进行精细搜索，以平衡速度和精度 。   
- **基于哈希的方法（Hashing-based）**：例如 LSH (Locality-Sensitive Hashing)。它使用特殊的哈希函数将相似的向量映射到同一个“桶”中，查询时只需在查询向量所在的桶内进行比较，大大减少了搜索空间 。   
- **基于量化的方法（Quantization-based）**：例如 IVF (Inverted File Index) 和 PQ (Product Quantization)。IVF 将向量空间划分为多个区域（簇），查询时只搜索查询向量所属区域及其邻近区域的向量 。PQ 则通过将向量分割成子向量并对子向量进行量化（编码）来压缩向量，从而减少存储和计算开销 。   
- **基于树的方法（Tree-based）**：例如 k-d 树、球树（Ball Trees）等，它们通过递归地划分向量空间来构建索引 。
 
向量数据库通常会实现这些 ANN 算法中的一种或多种，并允许用户根据应用需求在速度、精度和内存使用之间进行权衡 。

##### 基础语义搜索的局限性

尽管语义相似性搜索是 RAG 的强大基础，但它本身也存在一些固有的局限性：

- 依赖嵌入质量：搜索效果高度依赖于嵌入模型的质量。如果模型未能准确捕捉数据的语义细微差别，或者在特定领域表现不佳，搜索结果的相关性就会下降 。   
- **语义相似不等于上下文相关**：有时，与查询向量在语义上最接近的文档块，在具体的上下文或用户意图下可能并非最相关或最有用的信息 。   
- **结果冗余**：如前所述，最相似的几个文档块可能彼此之间也非常相似，导致信息冗余 。   
- **难以处理复杂或专业查询**：对于包含高度专业术语、歧义或需要结合多个信息点的复杂查询，基础语义搜索可能难以准确把握用户意图 。   
- **ANN 的近似性**：ANN 算法为了速度牺牲了绝对精度，可能导致在某些情况下错过最佳匹配结果 。   
- **高维空间的挑战**：“维度灾难”可能影响某些距离度量在高维空间中的有效性 。   

这些局限性凸显了优化 RAG 检索管道的必要性。选择合适的分块策略、嵌入模型、距离度量和索引算法是相互关联的，它们共同决定了检索的性能（延迟、准确性、成本）。优化 RAG 检索需要对整个流程进行整体考虑，而非孤立地调整某个组件。即使向量数据库抽象了许多底层复杂性，但理解嵌入、距离度量和 ANN 算法的基本原理对于有效调整参数（如索引类型或搜索参数 ）和诊断问题（如相关性差或延迟高）仍然至关重要，不能将其视为完全的“黑盒”。向量数据库技术的快速发展  正是 RAG 等基于嵌入的 AI 应用兴起的直接结果，这表明 LLM/嵌入技术的进步与向量存储/检索技术之间存在着相互促进、共同演化的关系。

##### 相似性搜索 (similarity_search)
这是最基础也是最核心的检索操作，目的是根据查询字符串找到语义上最相似的文档块 。   

- k 参数：控制返回的最相似结果的数量，默认值通常为 4 。   
- filter 参数：允许根据文档元数据进行过滤，以缩小搜索范围或精确查找。过滤器通常是一个字典，指定要匹配的元数据键值对。不同向量存储可能支持不同的过滤语法（例如，Chroma 支持 `$eq`, `$ne` 等操作符）。
- 
代码示例 (使用 Chroma)：

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma          # 向量数据库库
from langchain.docstore.document import Document # LangChain 文档格式

# --- 1. 准备样本数据和向量存储 ---

# 定义一些简单的文本数据 (和之前一样)
sample_texts = [
    """第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，
交互式的图表。
Matplotlib 可⽤于 Python 脚本， Python 和 IPython Shell 、 Jupyter notebook ， Web 应⽤程序服务器和各种图形⽤户界⾯⼯具包等。
Matplotlib 是 Python 数据可视化库中的泰⽃... (内容继续)""",
    "这是另一个文档块，可能关于 Python 基础。",
    "这是第三个文档块，也许是关于 Numpy 的介绍。"
]

# 将普通的文本字符串转换成 LangChain 能理解的 Document 对象
documents = [Document(page_content=text) for text in sample_texts]
print(f"准备了 {len(documents)} 个文档对象。")

# --- 选择并初始化 Ollama 嵌入模型 ---
# *** 这是关键的改动 ***
try:
    # 指定你要使用的、已经通过 Ollama 拉取到本地的嵌入模型名称
    # 常见的嵌入模型有 'nomic-embed-text', 'mxbai-embed-large' 等
    ollama_model_name = "nomic-embed-text" # <--- 把这里改成你实际使用的模型名!
    print(f"正在初始化 Ollama 嵌入模型: {ollama_model_name}...")
    # 默认情况下，它会连接到 http://localhost:11434
    embeddings = OllamaEmbeddings(model=ollama_model_name)
    print("Ollama 嵌入模型已初始化。")
    # (可选) 可以尝试生成一个测试嵌入，确保 Ollama 连接正常
    # embeddings.embed_query("测试连接")
    # print("Ollama 连接测试成功。")
except Exception as e:
    print(f"初始化 Ollama 嵌入模型时出错: {e}")
    print("请确保:")
    print("  1. Ollama 服务正在本地运行。")
    print(f"  2. 你已经通过 'ollama pull {ollama_model_name}' 拉取了模型。")
    print("  3. 网络连接正常 (如果需要的话)。")
    exit()

# --- 使用 Chroma 创建内存向量存储 ---
# 这一步和之前类似，只是现在用的是 OllamaEmbeddings
try:
    print("正在创建内存向量存储 (使用 Ollama 嵌入)...")
    vectordb_chinese = Chroma.from_documents(
        documents=documents,  # 要存储的文档
        embedding=embeddings  # 使用配置好的 Ollama 嵌入模型
    )
    print("内存向量存储创建成功。")
except Exception as e:
    print(f"创建向量存储时出错: {e}")
    print("请检查 Ollama 服务是否运行正常，以及模型是否兼容。")
    exit()

# --- 2. 执行相似性搜索 ---

# 定义你要问的问题
question_chinese = "Matplotlib是什么？"
print(f"\n要搜索的问题是: '{question_chinese}'")

# 使用刚才创建的向量存储来执行搜索
try:
    # k=3 表示返回最相似的 3 个结果
    print("正在执行相似性搜索...")
    docs_chinese = vectordb_chinese.similarity_search(question_chinese, k=3)
    print("搜索完成。")

    # --- 3. 显示搜索结果 ---

    print(f"\n为问题 '{question_chinese}' 找到了 {len(docs_chinese)} 个相关文档:")

    # 打印出最相关的那个文档的内容
    if docs_chinese:
        print("\n最相关文档的内容:")
        print(docs_chinese[0].page_content)
    else:
        print("没有找到相关的文档。")

except Exception as e:
    print(f"\n在相似性搜索过程中发生错误: {e}")
    print("请检查向量数据库和 Ollama 服务是否都正常工作。")

准备了 3 个文档对象。
正在初始化 Ollama 嵌入模型: nomic-embed-text...
Ollama 嵌入模型已初始化。
正在创建内存向量存储 (使用 Ollama 嵌入)...


C:\Users\14798\AppData\Local\Temp\ipykernel_18760\2329499258.py:31: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=ollama_model_name)


内存向量存储创建成功。

要搜索的问题是: 'Matplotlib是什么？'
正在执行相似性搜索...
搜索完成。

为问题 'Matplotlib是什么？' 找到了 3 个相关文档:

最相关文档的内容:
第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，
交互式的图表。
Matplotlib 可⽤于 Python 脚本， Python 和 IPython Shell 、 Jupyter notebook ， Web 应⽤程序服务器和各种图形⽤户界⾯⼯具包等。
Matplotlib 是 Python 数据可视化库中的泰⽃... (内容继续)


#### 高级检索策略 1：使用最大边际相关性（MMR）增强多样性
##### 问题：相似性搜索中的冗余
基础的向量相似性搜索旨在找到与用户查询在语义上最接近的文档块。然而，这种方法常常导致一个问题：检索出的排名靠前的文档块不仅与查询相似，彼此之间也非常相似 。想象一下，如果知识库中有多篇文档都从略微不同的角度描述了同一个核心概念，那么针对该概念的查询可能会返回这些高度重叠的文档块。   

这种冗余是有害的，原因如下：

- **浪费上下文窗口**：LLM 的上下文窗口是有限资源。用重复或高度相似的信息填充这个窗口，会挤占本可以容纳其他不同但同样相关信息的空间 。  
- **信息覆盖不足**：过度关注与查询最相似的几个（可能相似的）方面，可能会忽略掉那些虽然与查询的语义距离稍远，但包含独特、重要信息或不同视角的文档块 。   
- **缺乏多样性**：用户（或 LLM）可能无法获得对一个主题的全面、多角度的理解，因为检索结果只反映了信息空间中的一小片区域 。   

##### MMR 解析：平衡相关性与多样性
最大边际相关性（Maximum Marginal Relevance, MMR） 是一种专门设计用来解决上述冗余问题的后处理（或重排序）技术 。它的目标是在选择最终要呈现给用户（或 LLM）的文档集时，同时优化两个标准：   

- 相关性（Relevance）：文档与原始用户查询的相似程度。
- 多样性（Diversity）：文档与已经选择的文档集合之间的不相似程度 。   
MMR 通过一个迭代的过程来实现这种平衡。它首先可能会基于纯粹的相似性找到一个初始的最佳匹配文档。然后，在选择下一个文档时，它会考虑所有剩余的候选文档。对于每个候选文档，MMR 会计算一个分数，该分数是其与查询相关性的“奖励”和其与已选文档相似性的“惩罚”的加权组合。MMR 会选择那个在这个组合分数上得分最高的文档加入结果集。这个过程重复进行，直到选出所需数量（k）的文档 。   

##### MMR 算法与 Lambda 参数
MMR 的核心思想可以用一个公式（或其概念等价形式）来表示 ：   

$MMR(D_i) = \underset{D_i \in R \setminus S}{\operatorname{argmax}} \left$

这里：

- $D_i$ 是候选文档。
- $R$ 是所有候选文档的集合（通常是初始相似性搜索返回的 top-N' 结果）。
- $S$ 是已经选入最终结果集的文档集合。
- $Q$ 是用户查询。
- $Sim(D_i,Q)$ 是文档 $D_i$ 与查询 $Q$ 之间的相似度（相关性）。
- $max_{D_j∈S}Sim(D_i,D_j)$ 是文档 $D_i$ 与已选文档集合 $S$ 中最相似的那个文档的相似度（衡量冗余度）。
- $λ (lambda)$ 是一个介于 0 和 1 之间的参数，用于控制相关性和多样性之间的权衡 。
 
Lambda 参数 (λ) 的作用至关重要：

- 当 λ=1 时，公式只考虑 $Sim(D_i,Q)$，MMR 退化为标准的相似性排序，只关注相关性 。   
- 当 λ=0 时，公式只考虑最小化与已选文档的最大相似度，即最大化多样性，可能牺牲相关性 。
- 当 λ 取 0 到 1 之间的值时（例如 0.5 或 0.7），MMR 会在相关性和多样性之间进行平衡 。

在实际应用中（例如 LangChain 等框架 ），MMR 通常作用于一个初始的候选文档集上。这个初始集是通过常规相似性搜索获取的，其大小由 fetch_k 参数控制，且 fetch_k 通常大于最终需要返回的文档数量 k。MMR 随后从这 fetch_k 个候选中选出最优的 k 个文档。

##### MMR 在 RAG 中的优势与应用场景

在 RAG 系统中使用 MMR 可以带来显著的好处：

- **减少冗余信息**：确保传递给 LLM 的上下文包含更广泛、更少重复的信息 。   
- **提高上下文质量**：通过包含多样化的视角和信息片段，为 LLM 提供更丰富的背景知识，有助于生成更全面、更细致的答案 。   
- **改善复杂查询的处理**：对于需要综合多个方面信息的复杂问题，MMR 检索到的多样化文档集可能比仅包含最相似文档的集合更有用 。

##### 代码示例

关键参数 (通常通过 as_retriever 的 search_kwargs 传递)：
- k: 最终返回的文档数量 。   
- fetch_k: 初始检索的文档数量，MMR 算法将从这些文档中进行选择（通常大于 k，默认值可能是 20）。增加 fetch_k 可以提供更多候选文档，可能提高最终结果的多样性，但也会增加计算量。   
- lambda_mult: 控制相关性与多样性之间的权衡因子，取值范围 。0 表示最大化多样性，1 表示最小化多样性（等同于标准相似性搜索），默认值通常为 0.5 。   

In [2]:
import os
# 导入 LangChain 相关的库
from langchain_community.embeddings import OllamaEmbeddings # Ollama 嵌入模型
from langchain_chroma import Chroma                     # Chroma 向量数据库
from langchain.docstore.document import Document        # LangChain 文档格式

# --- 1. 准备样本数据 (包含重复内容以便观察 MMR 效果) ---

# 定义一些简单的文本数据，故意加入重复内容
doc1_content = """第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形... (内容省略)"""
doc2_content = "这是另一个文档块，可能关于 Python 基础。"
doc3_content = "第三个文档块，也许是关于 Numpy 的介绍。"
doc4_content = "补充内容：Matplotlib也常用于绘制科学图表。" # 另一个与Matplotlib相关的

documents = [
    Document(page_content=doc1_content, metadata={'source': '第一回', 'page': 0}),
    Document(page_content=doc1_content, metadata={'source': '第一回-重复', 'page': 0}), # <-- 重复的内容
    Document(page_content=doc2_content, metadata={'source': 'Python基础', 'page': 1}),
    Document(page_content=doc3_content, metadata={'source': 'Numpy介绍', 'page': 1}),
    Document(page_content=doc4_content, metadata={'source': '补充', 'page': 1}),
]
print(f"准备了 {len(documents)} 个文档对象 (包含重复内容)。")

# --- 2. 初始化 Ollama 嵌入模型和向量存储 ---
try:
    ollama_model_name = "nomic-embed-text" # <--- 确认这是你本地有的 Ollama 嵌入模型
    print(f"正在初始化 Ollama 嵌入模型: {ollama_model_name}...")
    embeddings = OllamaEmbeddings(model=ollama_model_name)
    print("Ollama 嵌入模型已初始化。")

    print("正在创建内存向量存储 (使用 Ollama 嵌入)...")
    # 创建向量存储，我们继续叫它 vectordb_chinese
    vectordb_chinese = Chroma.from_documents(
        documents=documents,
        embedding=embeddings
    )
    print("内存向量存储创建成功。")

except Exception as e:
    print(f"初始化 Ollama 或创建向量存储时出错: {e}")
    print("请检查 Ollama 服务是否运行，模型名称是否正确。")
    exit()

# --- 3. 执行 MMR 搜索 ---

# 定义问题 (我们用 Matplotlib 相关的问题，因为数据是关于这个的)
query = "告诉我关于 Matplotlib 的信息"
print(f"\n要搜索的问题是: '{query}'")

# --- 方法一: 使用 as_retriever() 并设置 search_type='mmr' ---
print("\n--- MMR 搜索方法一: 使用 as_retriever ---")
try:
    # 将向量存储包装成一个 MMR Retriever
    # search_kwargs 用于传递 MMR 的特定参数:
    # k: 最终返回多少个结果
    # fetch_k: 初始获取多少个候选结果 (MMR从中挑选)
    # lambda_mult: 控制相关性(1)与多样性(0)的平衡因子
    mmr_retriever = vectordb_chinese.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 3, 'fetch_k': 5, 'lambda_mult': 0.1} # 返回3个,从5个候选里选, 偏重多样性
    )

    # 执行搜索
    print(f"正在执行 MMR 搜索 (k=3, fetch_k=5, lambda=0.1)...")
    mmr_results = mmr_retriever.invoke(query)
    print("MMR Retriever 搜索完成。")

    # 显示结果
    print(f"\n对于查询 '{query}' 的 MMR (Retriever) 搜索结果:")
    if mmr_results:
        for i, doc in enumerate(mmr_results):
            # 打印元数据，观察是否比普通搜索更多样 (例如，不会连续出现 '第一回' 和 '第一回-重复')
            print(f"  结果 {i}: Metadata: {doc.metadata}, Content: '{doc.page_content[:50]}...'")
    else:
        print("  没有找到结果。")

except Exception as e:
    print(f"\n通过 as_retriever 执行 MMR 搜索时出错: {e}")

# --- 方法二: 直接调用 max_marginal_relevance_search 方法 ---
# (注意: Chroma 向量存储直接支持此方法)
print("\n--- MMR 搜索方法二: 直接调用 max_marginal_relevance_search ---")
try:
    print(f"正在执行直接 MMR 搜索 (k=3, fetch_k=5, lambda=0.1)...")
    mmr_direct_results = vectordb_chinese.max_marginal_relevance_search(
        query,
        k=3,          # 返回 3 个结果
        fetch_k=5,    # 从 5 个候选结果中挑选
        lambda_mult=0.1 # 偏重多样性 (0.0 表示纯多样性, 1.0 表示纯相关性)
    )
    print("直接 MMR 搜索完成。")

    # 显示结果
    print(f"\n对于查询 '{query}' 的直接 MMR 搜索结果:")
    if mmr_direct_results:
        for i, doc in enumerate(mmr_direct_results):
            print(f"  结果 {i}: Metadata: {doc.metadata}, Content: '{doc.page_content[:50]}...'")
            # 同样观察结果的多样性
    else:
        print("  没有找到结果。")

except NotImplementedError:
    # 这个异常理论上对于 Chroma 不会触发，但为了完整性加上
    print("\n错误: 此向量存储实例不支持直接调用 max_marginal_relevance_search。")
except Exception as e:
     print(f"\n调用直接 MMR 搜索时出错: {e}")

准备了 5 个文档对象 (包含重复内容)。
正在初始化 Ollama 嵌入模型: nomic-embed-text...
Ollama 嵌入模型已初始化。
正在创建内存向量存储 (使用 Ollama 嵌入)...
内存向量存储创建成功。

要搜索的问题是: '告诉我关于 Matplotlib 的信息'

--- MMR 搜索方法一: 使用 as_retriever ---
正在执行 MMR 搜索 (k=3, fetch_k=5, lambda=0.1)...
MMR Retriever 搜索完成。

对于查询 '告诉我关于 Matplotlib 的信息' 的 MMR (Retriever) 搜索结果:
  结果 0: Metadata: {'page': 1, 'source': '补充'}, Content: '补充内容：Matplotlib也常用于绘制科学图表。...'
  结果 1: Metadata: {}, Content: '第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 P...'
  结果 2: Metadata: {'page': 1, 'source': 'Numpy介绍'}, Content: '第三个文档块，也许是关于 Numpy 的介绍。...'

--- MMR 搜索方法二: 直接调用 max_marginal_relevance_search ---
正在执行直接 MMR 搜索 (k=3, fetch_k=5, lambda=0.1)...
直接 MMR 搜索完成。

对于查询 '告诉我关于 Matplotlib 的信息' 的直接 MMR 搜索结果:
  结果 0: Metadata: {'page': 1, 'source': '补充'}, Content: '补充内容：Matplotlib也常用于绘制科学图表。...'
  结果 1: Metadata: {}, Content: '第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 P...'
  结果 2: Metadata: {'page': 1, 'source': 'Numpy介绍'}, Content: '第三个文档块，也许是关于 Numpy 的介绍。...'


#### 高级检索策略 2：利用元数据过滤提升精度
##### 利用结构化元数据
向量数据库中存储的文档块通常不仅仅包含文本内容及其嵌入向量，还会关联一些结构化的**元数据（Metadata）** 。这些元数据可以是任何描述文档块属性的信息，例如：   

- 时间戳（创建日期、修改日期）   
- 来源信息（文件名、URL、数据库表名）   
- 类别或标签（主题、部门、文档类型）   
- 作者或用户 ID
- 年份    
- 章节或页码    
- 语言
  
**元数据过滤（Metadata Filtering）** 指的是利用这些结构化属性来缩小搜索范围或精确化检索结果的过程 。它作为向量相似性搜索的补充，允许 RAG 系统施加额外的约束条件。例如，用户可能想查找关于“机器学习”的文档，但只对“过去一年内”发布的、“特定作者撰写”的或“来自某个特定项目”的文档感兴趣。   

元数据过滤的主要优势在于：

- **提高精度**：通过排除不符合特定结构化标准的文档块，可以显著提高检索结果的准确性 。   
- **目标性更强**：能够根据用户的具体需求（如时间范围、来源限制）检索更具针对性的信息 。   
- **实现复杂查询**：支持结合语义内容和结构化属性的混合查询 。   
- **减少噪声**：过滤掉不相关的文档块，为后续的 LLM 生成提供更干净、更集中的上下文 。
   
##### 前置过滤（Pre-filtering） vs 后置过滤（Post-filtering）
在 RAG 管道中结合向量搜索和元数据过滤主要有两种策略：后置过滤和前置过滤。

| 方法     | 机制                                                       | 优点                                                                        | 缺点                                                                                                                               | 最佳适用场景                                                                      | 实现说明                                                                    |
| :------- | :--------------------------------------------------------- | :-------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| 后置过滤 | 1. 向量搜索 Top-K <br> 2. 对 K 个结果应用元数据过滤          | 实现简单，向量相似性为主时高效                                              | 可能丢失相关结果（如果它们不在初始 Top-K 中），可能需要增大 K 值                                                                   | 向量相似性是主要搜索条件，元数据是次要约束；数据库对前置过滤支持不佳              | 广泛支持；注意 K 值的选择                                                   |
| 前置过滤 | 1. 应用元数据过滤 <br> 2. 在过滤后的子集上进行向量搜索       | 结果更准确（尤其当过滤器选择性强时），减少向量搜索噪声                        | 实现可能更复杂，依赖数据库的元数据索引能力 ，过滤器选择性不强时可能性能不佳                                                      | 元数据是主要约束条件；需要确保检索范围被严格限定；数据库支持高效元数据索引      | 依赖数据库实现；可能需要特定的索引策略或数据建模                              |

###### 后置过滤（Post-filtering）

- 机制：首先执行标准的向量相似性搜索，获取排名最高的 K 个候选文档块。然后，对这 K 个候选文档块应用元数据过滤器，剔除不满足条件的文档块，保留最终结果 。   
- 优点：实现相对简单，特别是当向量相似性是主要的搜索驱动因素，而元数据只是次要约束时，这种方法通常效率较高 。许多向量数据库都支持这种方式。   
- 缺点：可能导致结果不完整。如果初始的 Top-K 向量搜索结果中，恰好满足元数据条件的文档块数量很少，或者根本没有，那么即使数据库中存在满足条件的文档块（但它们在初始相似性排名中较低），后置过滤也无法找到它们 。为了缓解这个问题，可能需要增大初始检索的 K 值（即获取更多的候选文档），但这会增加向量搜索的开销 。   

###### 前置过滤（Pre-filtering）

- 机制：在执行向量相似性搜索之前，先应用元数据过滤器。这意味着向量搜索只在满足元数据条件的文档块子集上进行 。   
- 优点：理论上可以提供更准确的结果，特别是当元数据过滤条件非常严格（能显著缩小搜索空间）或者是查询的主要限制因素时 。它能有效减少向量搜索需要处理的数据量，从而降低噪声 。   
- 缺点：在某些向量数据库中，高效地实现前置过滤可能更复杂 。它可能需要数据库支持对元数据进行高效索引，或者需要特定的数据建模策略（如为不同的元数据组合创建单独的索引或表）。如果元数据过滤器选择性不强（即过滤后剩余的数据量仍然很大），前置过滤的性能优势可能不明显，甚至可能比后置过滤慢（例如，在 ChromaDB 中因缺乏元数据索引而导致的性能问题 ）。   

##### 代码示例


In [45]:
import os
# 导入 LangChain 相关的库
from langchain_community.embeddings import OllamaEmbeddings # Ollama 嵌入模型
from langchain_chroma import Chroma                         # Chroma 向量数据库
from langchain.docstore.document import Document          # LangChain 文档格式
# 需要安装 chromadb 库: pip install chromadb
# 需要安装 langchain-community 库: pip install langchain-community

# --- 1. 准备样本数据 (包含语义相关但元数据不同的文档) ---

# 定义一些文本数据，模拟不同内容和元数据
documents = [
    # Semantically similar to query ("Python libraries for plotting")
    Document(page_content="Matplotlib 是 Python 中最流行的绘图库之一。", metadata={'source': 'DocA', 'category': 'Visualization', 'year': 2023}), # Match query, Filter No
    Document(page_content="使用 Seaborn 让数据可视化更美观，它基于 Matplotlib。", metadata={'source': 'DocB', 'category': 'Visualization', 'year': 2024}), # Match query, Filter YES
    Document(page_content="Plotly 提供了交互式图表功能，支持 Python。", metadata={'source': 'DocC', 'category': 'Visualization', 'year': 2024}), # Match query, Filter YES (Potentially lower semantic similarity than DocB for "plotting library")
    Document(page_content="Pandas 用于数据处理，经常与 Matplotlib 结合使用。", metadata={'source': 'DocD', 'category': 'Data Analysis', 'year': 2023}), # Partially related to query, Filter No

    # Not semantically similar to query
    Document(page_content="学习 Python 基础语法。", metadata={'source': 'DocE', 'category': 'Programming', 'year': 2024}), # Not query related, Filter No
    Document(page_content="介绍 Numpy 的核心概念。", metadata={'source': 'DocF', 'category': 'Scientific Computing', 'year': 2023}), # Not query related, Filter No
    Document(page_content="最新的 AI 技术进展。", metadata={'source': 'DocG', 'category': 'Technology', 'year': 2024}), # Not query related, Filter No, Year YES
]
print(f"准备了 {len(documents)} 个文档对象。")

# --- 2. 初始化 Ollama 嵌入模型和向量存储 ---
# 确保 Ollama 服务正在运行，并且 nomic-embed-text 模型已下载
try:
    ollama_model_name = "nomic-embed-text" # <--- 确认这是你本地有的 Ollama 嵌入模型
    print(f"正在初始化 Ollama 嵌入模型: {ollama_model_name}...")
    embeddings = OllamaEmbeddings(model=ollama_model_name)
    print("Ollama 嵌入模型已初始化。")

    print("正在创建内存向量存储 (使用 Ollama 嵌入)...")
    # vectordb_metadata = Chroma.from_documents(documents=documents, embedding=embeddings)
    # For better filtering demonstration, ensure metadata is indexed if your Chroma version supports it implicitly.
    # With recent chromadb, filter parameter works directly.
    vectordb_metadata = Chroma.from_documents(
        documents=documents,
        embedding=embeddings
    )
    print("内存向量存储创建成功。")

except Exception as e:
    print(f"初始化 Ollama 或创建向量存储时出错: {e}")
    print("请检查 Ollama 服务是否运行，模型名称是否正确。")
    exit()

# --- 定义查询和元数据过滤器 ---
query = "告诉我一些关于 Python 库用于绘图的信息" # 查询与 Visualization 相关的文档语义相似
# 我们想要查找：类别是 'Visualization' 并且年份是 2024 的文档
# *** 修正：使用 Chroma/LangChain 识别的过滤语法，用 $and 组合条件 ***
metadata_filter = {
    "$and": [
        {"category": {"$eq": "Visualization"}}, # category 必须等于 'Visualization'
        {"year": {"$eq": 2024}}             # year 必须等于 2024
        # 也可以写成 {"category": "Visualization", "year": 2024}，但带 $eq 更明确
    ]
}
print(f"\n要搜索的问题是: '{query}'")
print(f"要应用的元数据过滤器 (Chroma格式): {metadata_filter}")

# --- 3. 后置过滤 (Post-filtering) ---
print("\n--- 后置过滤 (Post-filtering) ---")
# 机制: 先向量搜索 Top-K，再对结果进行元数据过滤
initial_k_post = 2 # 初始获取 Top-2 向量最相似的结果 (故意设小一些)

# *** 修正：在 try 块前初始化 final_results_post ***
final_results_post = []

try:
    print(f"步骤 1: 执行初始向量搜索 Top-{initial_k_post}...")
    # 使用 similarity_search 获取初始的 Top-K 结果，不带过滤器
    initial_results_post = vectordb_metadata.similarity_search(query, k=initial_k_post)
    print(f"初始搜索结果 ({len(initial_results_post)} 条):")
    # 为了方便对比，尝试获取相似度（similarity_search 默认不返回得分，需要 similarity_search_with_score）
    initial_results_with_score = vectordb_metadata.similarity_search_with_score(query, k=initial_k_post)
    for i, (doc, score) in enumerate(initial_results_with_score):
         print(f"  结果 {i}: Score: {score:.4f}, Metadata: {doc.metadata}, Content: '{doc.page_content[:30]}...'")


    print(f"\n步骤 2: 对初始结果应用元数据过滤: {metadata_filter}")
    # final_results_post = [] # <-- 原始代码在这里初始化，如果步骤1失败，会跳过
    # Note: Manual filtering needs to correctly interpret the complex filter structure
    # For simplicity in manual filtering, let's assume the filter is a simple key-value check for this example.
    # A robust manual filter would need to parse the $and/$or structure.
    # Let's filter based on the original simple dict structure for the manual step, assuming it corresponds.
    simple_filter_check = {"category": "Visualization", "year": 2024} # Corresponding simple check

    for (doc, score) in initial_results_with_score: # Iterate through results with score
        # 手动检查文档的元数据是否符合过滤条件 (使用简单检查逻辑)
        match = True
        for key, value in simple_filter_check.items():
            # 使用 .get() 方法安全访问元数据键
            if doc.metadata.get(key) != value:
                match = False
                break
        if match:
            final_results_post.append((doc, score)) # Store doc and its original score


    # 显示最终结果
    print(f"\n最终后置过滤结果 ({len(final_results_post)} 条):")
    if final_results_post:
        # 再次按相似度排序，虽然对 Top-K_initial 过滤后可能不是严格 Top-N
        final_results_post.sort(key=lambda x: x[1], reverse=True)
        for i, (doc, score) in enumerate(final_results_post):
            print(f"  结果 {i}: Score: {score:.4f}, Metadata: {doc.metadata}, Content: '{doc.page_content[:30]}...'")
    else:
        print("  没有找到符合过滤条件的文档。")

except Exception as e:
    print(f"执行后置过滤时出错: {e}")
    # 即使出错，final_results_post 也是 [], 不会引发 NameError

# --- 4. 前置过滤 (Pre-filtering) ---
print("\n--- 前置过滤 (Pre-filtering) ---")
# 机制: 先应用元数据过滤，再在过滤后的子集上进行向量搜索 Top-K
final_k_pre = 3 # 最终需要的 Top-K 结果数量 (在过滤后的子集里)

# *** 修正：在 try 块前初始化 final_results_pre ***
final_results_pre = []

try:
    print(f"步骤 1 和 2: 应用元数据过滤并在过滤后的子集上进行向量搜索 Top-{final_k_pre}...")
    # 直接使用向量存储的 similarity_search_with_score 并传入 filter 参数
    # Chroma 会在内部处理过滤，通常在向量搜索之前或过程中利用索引
    final_results_pre_with_score = vectordb_metadata.similarity_search_with_score(
        query,
        k=final_k_pre,
        filter=metadata_filter # <-- 在这里应用修正后的过滤器语法
    )
    final_results_pre = [(doc, score) for doc, score in final_results_pre_with_score] # 将结果转为列表，方便后续使用len()等

    print("前置过滤搜索完成。")

    # 显示最终结果
    print(f"\n最终前置过滤结果 ({len(final_results_pre)} 条):")
    if final_results_pre:
        # 结果已经是按相似度排序的
        for i, (doc, score) in enumerate(final_results_pre):
             print(f"  结果 {i}: Score: {score:.4f}, Metadata: {doc.metadata}, Content: '{doc.page_content[:30]}...'")
    else:
         print("  没有找到符合过滤条件的文档。")

except Exception as e:
    print(f"执行前置过滤时出错: {e}")
    print(f"错误类型: {type(e).__name__}")
    print("请检查过滤器语法是否正确，以及向量数据库是否支持该过滤操作。")
    # 即使出错，final_results_pre 也是 [], 不会引发 NameError


# --- 结果比较和解释 ---
# 这里的代码现在可以安全地访问 final_results_post 和 final_results_pre
print("\n--- 结果比较 ---")
print(f"元数据过滤器: {metadata_filter}")
print(f"后置过滤 (初始Top-{initial_k_post}) 找到 {len(final_results_post)} 条符合过滤条件的最终结果。")
print(f"前置过滤 (最终Top-{final_k_pre} 在过滤后子集上) 找到 {len(final_results_pre)} 条最终结果。")

print("\n解释:")
print(f"- 后置过滤先找到向量最相似的 Top-{initial_k_post} 个文档，然后检查这 {initial_k_post} 个文档的元数据是否符合条件。")
print("- 前置过滤直接告诉向量数据库：只在那些满足过滤器条件的文档中，找出向量最相似的 Top-K 个。")
print("- **关键区别:** 后置过滤可能因为满足过滤条件的文档不在初始 Top-K 内而遗漏它；前置过滤则确保了所有满足条件的文档都有机会参与向量排序。")
print("- 在这个示例中，我们期望看到后置过滤可能因为 `initial_k_post` 设为 2 而遗漏满足条件 ('Visualization', 2024) 的文档 (DocB 或 DocC)，因为向量相似度更高的 DocA 可能占据了 Top-2 的位置。")
print("- 前置过滤则会在 DocB 和 DocC (以及其他满足条件的文档，如果有的话) 中寻找向量最相似的 Top-3。")
print("- 修正了 Chroma 的过滤语法，使用 `$and` 操作符来组合多个过滤条件。")

准备了 7 个文档对象。
正在初始化 Ollama 嵌入模型: nomic-embed-text...
Ollama 嵌入模型已初始化。
正在创建内存向量存储 (使用 Ollama 嵌入)...
内存向量存储创建成功。

要搜索的问题是: '告诉我一些关于 Python 库用于绘图的信息'
要应用的元数据过滤器 (Chroma格式): {'$and': [{'category': {'$eq': 'Visualization'}}, {'year': {'$eq': 2024}}]}

--- 后置过滤 (Post-filtering) ---
步骤 1: 执行初始向量搜索 Top-2...
初始搜索结果 (2 条):
  结果 0: Score: 240.2000, Metadata: {}, Content: '这是另一个文档块，可能关于 Python 基础。...'
  结果 1: Score: 240.2000, Metadata: {'page': 1, 'source': 'Python基础'}, Content: '这是另一个文档块，可能关于 Python 基础。...'

步骤 2: 对初始结果应用元数据过滤: {'$and': [{'category': {'$eq': 'Visualization'}}, {'year': {'$eq': 2024}}]}

最终后置过滤结果 (0 条):
  没有找到符合过滤条件的文档。

--- 前置过滤 (Pre-filtering) ---
步骤 1 和 2: 应用元数据过滤并在过滤后的子集上进行向量搜索 Top-3...
前置过滤搜索完成。

最终前置过滤结果 (3 条):
  结果 0: Score: 325.6039, Metadata: {'category': 'Visualization', 'source': 'DocC', 'year': 2024}, Content: 'Plotly 提供了交互式图表功能，支持 Python。...'
  结果 1: Score: 325.6039, Metadata: {'category': 'Visualization', 'source': 'DocC', 'year': 2024}, Content: 'Plotly 提供了交互式图表功能，支持

#### 高级检索策略 3：利用 LLM 赋能智能检索（LLM 辅助检索）

##### 引言：LLM 作为检索增强器

传统的 RAG 流程中，LLM 主要扮演着“生成器”的角色，在检索完成后基于提供的上下文生成最终答案。然而，随着 LLM 能力的增强，一种新的范式正在兴起：利用 LLM 自身的智能来改进和优化 RAG 流程中检索这一环节。

LLM 强大的自然语言理解、生成和推理能力，可以被用来克服传统检索方法（无论是基于关键词还是基于向量）在处理用户查询、评估文档相关性以及优化结果排序等方面的不足。这种利用 LLM 来增强检索过程的方法，可以统称为 LLM 辅助检索（LLM-Assisted Retrieval）。

**LLM 辅助检索技术概览**

| 技术                             | 描述                                       | 机制                                                                                                 | 目标/优势                                                                                                  | 关键参考      |
| :------------------------------- | :----------------------------------------- | :--------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :------------ |
| 查询重写/转换 (Query Rewriting)  | 使用 LLM 修改或生成更适合检索系统的查询    | LLM 对原始查询进行释义、扩展、分解、或提取元数据                                                       | 克服查询模糊性/复杂性，提高检索召回率和精度                                                                |               |
| 假设性文档嵌入 (HyDE)            | 使用 LLM 生成的假设性文档的嵌入进行检索    | 1. LLM 根据查询生成假设性答案/文档 <br> 2. 对假设性文档进行嵌入 <br> 3. 使用该嵌入向量进行相似性搜索      | 解决查询与答案文档在嵌入空间中的不匹配问题，提高零样本检索性能                                             |               |
| 基于 LLM/交叉编码器的重排序 (Re-ranking) | 使用更强大的模型（LLM 或交叉编码器）对初始检索结果进行重新排序 | 对初始候选文档集，使用 LLM 或交叉编码器计算更精确的相关性分数，然后按新分数排序                      | 提高检索精度，将最相关文档排在前面，优化 LLM 输入上下文，减少噪声，缓解“中间丢失”问题                      |               |

LLM 辅助检索技术代表了一种趋势，即让检索过程本身变得更加“智能”和自适应，利用 LLM 的推理能力来克服纯粹基于算法或向量的方法的局限性。像查询重写、HyDE 和 LLM 重排序这样的技术，都明确地利用 LLM 来解释查询、生成上下文或判断相关性——这些任务需要的理解深度超过了简单的向量运算。这标志着使用 AI 来改进 AI 输入的实践。



In [8]:
# 导入所需模块
import getpass
import os
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# 导入 DeepSeek (Assuming it's available in langchain_community, adjust if needed)
from langchain import hub
from langchain_deepseek import ChatDeepSeek


os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("输入您的 DeepSeek API 密钥：")

# --- Configuration ---
# Ensure your DeepSeek API key is set as an environment variable
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
if not DEEPSEEK_API_KEY:
    raise ValueError("DEEPSEEK_API_KEY environment variable not set. Please set it.")

# Specify the DeepSeek model you want to use (e.g., "deepseek-chat", "deepseek-coder")
DEEPSEEK_MODEL_NAME = "deepseek-chat" # <-- CHANGE IF NEEDED

# --- Initialize Components ---

# 初始化 DeepSeek 模型
try:
    # Use ChatDeepseek for chat model interface
    llm = ChatDeepSeek(
        model=DEEPSEEK_MODEL_NAME,
        temperature=0
        # You might add other parameters like 'max_tokens' if needed
    )
    # Test connection (optional, but good practice)
    llm.invoke("Hello!")
    print(f"Successfully connected to DeepSeek model: {DEEPSEEK_MODEL_NAME}")
except Exception as e:
    print(f"Error connecting to DeepSeek model '{DEEPSEEK_MODEL_NAME}': {e}")
    print("Please ensure your API key is correct and the model name is valid.")
    exit() # Exit if connection fails

输入您的 DeepSeek API 密钥： ········


sk-2a425adfd5684c288081ea5874495744
Successfully connected to DeepSeek model: deepseek-chat


##### 查询重写/转换 (Query Rewriting/Transformation)

**问题：** 用户输入的自然语言查询可能存在多种问题，使其不适合直接用于检索系统（无论是基于关键词还是语义向量）：

* 模糊性或歧义性：查询的意图不明确。
* 措辞不佳：使用的词语可能与知识库中的相关文档不匹配。
* 复杂性：查询可能包含多个子问题或约束条件。
* 过于简洁：查询可能缺乏足够的上下文信息。

**机制：** 查询重写利用 LLM 在发送查询到检索系统之前，对原始查询进行修改或转换，生成一个或多个更优化、更可能检索到相关信息的查询。

**具体方法示例：**

* **Rewrite-Retrieve-Read (LangChain)：** 明确提出原始查询并非最优，主张先用 LLM 重写查询，再进行检索和阅读。其实现展示了如何处理包含无关信息的分心查询，通过 LLM 生成一个更聚焦于核心问题的搜索查询。
* **RePhraseQuery (LangChain)：** 作为一种简单的检索器包装器，它在用户输入和底层检索器之间插入一个 LLM，用于预处理查询，例如去除不相关信息（如人名、地名）以生成更简洁的向量库查询。其目标是简化和清洗查询。
* **MultiQueryRetriever (LangChain)：** 利用 LLM 从用户输入生成多个不同角度的查询，对每个查询进行检索，然后合并所有结果并去重，以获取更丰富、更全面的相关文档集。这种方法旨在通过扩大搜索范围来克服单一视角查询的局限性，提高召回率。
* **Step-back Prompting (LangChain/Conceptual)：** 针对复杂问题，先生成一个更宽泛、更一般性的“后退一步”查询，用于检索背景知识或上下文信息，然后再用原始查询结合这些背景信息进行精确回答。这是一种时间或逻辑上的分解，适用于需要前置知识的问题。
* **查询分解（Query Decomposition, LangChain/LlamaIndex）：** 将一个复杂的用户查询分解为多个更简单的子问题，每个子问题可以独立回答或路由到不同的知识源/工具进行处理。这对于处理多方面查询或在组合式 RAG 系统中进行路由至关重要。LlamaIndex 进一步区分了单步分解（生成一个更简单的子查询）和多步分解（生成一系列需要依次执行的子查询）。
* **SQL 查询生成 (LangChain)：** 这是针对结构化数据（如 SQL 数据库）的特定查询转换技术。它将用户的自然语言问题转换为可在数据库上执行的 SQL 查询语句。
* **路由（Routing, LlamaIndex）：** 虽然不直接重写查询文本，但它通过分析查询来确定应该将查询发送到哪个（或哪些）工具或索引进行处理。这是一种元级别的查询处理，关注查询的“去向”而非“措辞”。
* **查询松弛（Query Relaxation, Haystack/Conceptual）：** 主要用于处理多词查询返回零结果的情况，通过移除查询中的一个或多个词项来放宽约束，以获取部分匹配的结果。这种技术可能改变原始查询意图，被视为介于搜索和推荐之间的技术。


总体来看，各种查询重写技术在“通过修改查询来改善检索效果”这一核心目标上具有高度一致性。它们都认识到原始用户查询的不足，并尝试利用 LLM 进行优化。不同之处主要体现在：

* **针对的查询缺陷类型不同：** 有的针对歧义性，有的针对复杂性，有的针对无关信息，有的针对零结果问题。
* **采用的转换策略不同：** 单次重写、生成多个查询、分解为子问题、简化/清洗、生成结构化查询（如 SQL）、移除词项等。
* **框架实现方式不同：** LangChain 提供了多种链（Chain）和检索器（Retriever）实现（如 LLMChain, RePhraseQueryRetriever, MultiQueryRetriever），并利用 LangChain Expression Language (LCEL) 进行组合。LlamaIndex 则将许多这类转换实现为 QueryTransform 模块，可插入到 TransformQueryEngine 或 MultiStepQueryEngine 中。Haystack 则通过其 SearchQuerySet API 和可定制的表单/视图层提供查询修改的入口。

这种多样性本身就说明了用户查询失败模式的多样性，没有一种单一的重写策略能适用于所有情况。开发者需要根据具体的应用场景和查询特点来选择最合适的重写技术。

###### Rewrite-Retrieve-Read

此示例展示了 LangChain Expression Language (LCEL) 的强大之处。它首先定义了一个基础 RAG 链，并演示了其在处理包含无关信息（"分心"）的查询时的失败。然后，它构建了一个 rewriter 链，该链使用从 LangChain Hub 加载的提示模板 (langchain-ai/rewrite) 来指示 LLM 生成一个更适合搜索引擎的查询。最后，rewrite_retrieve_read_chain 将重写器、检索器和基础 RAG 逻辑组合在一起：原始查询首先被重写，然后使用重写后的查询进行检索，最后将检索到的上下文与原始查询一起用于生成答案。

In [9]:
# 初始化搜索工具
search = DuckDuckGoSearchAPIWrapper()
def retriever(query):
    return search.run(query)

# 定义基础 RAG 提示模板
template = """基于以下上下文回答用户问题：
<context>
{context}
</context>
问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# --- Rewrite-Retrieve-Read Implementation ---

# 定义重写查询的提示模板 (从 LangChain Hub 加载)
rewrite_prompt = hub.pull("langchain-ai/rewrite")

# 定义解析器以移除末尾的 '**'
def _parse(text):
     return text.strip().strip('"').strip('**').strip()

# 构建重写器链 (rewrite_prompt | DeepSeek LLM | 输出解析器 | 自定义解析函数)
# Using the initialized DeepSeek 'llm' instance here
rewriter = rewrite_prompt | llm | StrOutputParser() | _parse

# 构建 Rewrite-Retrieve-Read 链
# Using the initialized DeepSeek 'llm' instance for the final answer generation
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm # Use the DeepSeek LLM here
    | StrOutputParser()
)

# --- Execution ---

# 分心查询示例
distracted_query = "那个 Sam Bankman-Fried 的审判太疯狂了！langchain 是什么？"

# 使用 Rewrite-Retrieve-Read 链处理分心查询
print("\n--- Running Rewrite-Retrieve-Read with DeepSeek ---")
try:
    result = rewrite_retrieve_read_chain.invoke(distracted_query)
    print("Rewrite-Retrieve-Read output:\n", result)
except Exception as e:
    print(f"An error occurred during chain execution: {e}")

# (Optional) 测试重写器单独工作
# print("\n--- Testing Rewriter Separately ---")
# try:
#     rewritten = rewriter.invoke({"x": distracted_query})
#     print("Rewritten query:", rewritten)
# except Exception as e:
#     print(f"An error occurred during rewriting: {e}")

D:\Users\14798\anaconda3\envs\mcp\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



--- Running Rewrite-Retrieve-Read with DeepSeek ---
Rewrite-Retrieve-Read output:
 LangChain 是一个开源框架，旨在简化基于大型语言模型（LLM）的应用程序开发。它提供模块化工具和标准化接口，帮助开发者快速构建、连接和部署与LLM相关的功能，例如问答系统、文本生成或数据增强应用。

**关键特点：**  
1. **模块化设计**：提供预构建组件（如记忆管理、API集成工具链），可灵活组合。  
2. **多模型支持**：兼容OpenAI、Anthropic等主流LLM，避免供应商锁定。  
3. **链式编排**：通过“链”（Chains）将多个步骤（如数据预处理→LLM调用→结果解析）自动化。  
4. **代理系统**：允许LLM根据目标动态调用工具（如搜索、计算），类似AutoGPT的早期实现。  

**典型用例**：  
- 构建定制化客服聊天机器人  
- 开发基于文档的智能检索工具  
- 快速实验不同LLM在特定任务上的表现  

与Sam Bankman-Fried的FTX事件无直接关联，但若需分析庭审文本或构建金融合规工具，LangChain可加速此类NLP应用的开发。


###### RePhraseQuery

RePhraseQueryRetriever 包装了一个基础检索器（如向量存储的 as_retriever()）。当调用 invoke 时，它首先将用户查询传递给一个 LLM（或 LLMChain）进行重写（默认提示旨在去除无关信息并提取核心查询），然后使用重写后的查询执行实际的检索。示例还展示了如何通过提供自定义的 LLMChain 来完全控制重写逻辑，甚至可以改变查询的风格（如海盗语）。

In [15]:
import logging
from langchain.retrievers import RePhraseQueryRetriever
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# --- 设置 ---
logging.basicConfig()
logging.getLogger("langchain.retrievers.re_phraser").setLevel(logging.INFO)

# 加载和分割文档
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# 初始化潜入模型
ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text") # Embedding model


# 创建向量存储 (示例，实际使用需替换)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=ollama_embeddings)
# 假设 vectorstore 已创建并包含数据

# --- 使用默认提示的 RePhraseQueryRetriever ---
# 假设 vectorstore 已经存在
retriever_from_llm = RePhraseQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

query1 = "嗨，我是 Lance。任务分解的方法有哪些？"
docs1 = retriever_from_llm.invoke(query1)
print(f"Query 1: {query1}\nRetrieved docs count: {len(docs1)}\n")

query2 = "我住在旧金山。内存的类型有哪些？"
docs2 = retriever_from_llm.invoke(query2)
print(f"Query 2: {query2}\nRetrieved docs count: {len(docs2)}\n")

# --- 使用自定义提示的 RePhraseQueryRetriever ---
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""你是一个助手，负责接收用户的自然语言查询并将其转换为向量库查询。
在此过程中，请删除所有与检索任务无关的信息，并返回一个新的、简化的、
用于向量库检索的问题。新的用户查询应该用海盗语说出来。
这是用户的查询：{question}""",
)
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT)

# 假设 vectorstore 已经存在
retriever_from_llm_chain = RePhraseQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain
)

query3 = "嗨，我是 Lance。什么是最大内积搜索？"
docs3 = retriever_from_llm_chain.invoke(query3)
print(f"Query 3: {query3}\nRetrieved docs count: {len(docs3)}\n")

INFO:langchain.retrievers.re_phraser:Re-phrased question: 以下是转换后的检索查询：  

**"任务分解的方法有哪些？"**  

说明：  
1. 移除了问候语和个人信息（“嗨，我是 Lance”），因其与检索任务无关。  
2. 保留核心问题“任务分解的方法”，确保查询聚焦于目标信息。  

此优化后的查询更适合向量数据库检索，能有效匹配相关文档或知识片段。


Query 1: 嗨，我是 Lance。任务分解的方法有哪些？
Retrieved docs count: 4



INFO:langchain.retrievers.re_phraser:Re-phrased question: 内存的类型有哪些？


Query 2: 我住在旧金山。内存的类型有哪些？
Retrieved docs count: 4



INFO:langchain.retrievers.re_phraser:Re-phrased question: {'question': '嗨，我是 Lance。什么是最大内积搜索？', 'text': 'Arrr, what be maximum inner product search, matey?'}


Query 3: 嗨，我是 Lance。什么是最大内积搜索？
Retrieved docs count: 4



###### MultiQueryRetriever  

MultiQueryRetriever 利用 LLM 根据原始查询生成多个变体查询。from_llm 类方法提供了一种便捷的初始化方式。当调用 invoke 时，它会记录生成的多个查询，对每个查询执行检索，最后返回所有检索结果的并集（去重）。示例还展示了如何通过提供自定义的 llm_chain（包含提示和输出解析器）来定制查询生成过程。

In [20]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import logging
from langchain_chroma import Chroma # 假设已创建 vectordb
from langchain_openai import OpenAIEmbeddings # 假设已创建 vectordb
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 设置 (假设已设置 OpenAI API Key 和 vectordb) ---
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# 假设 vectordb 是一个已加载数据的 Chroma 实例
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

# --- MultiQueryRetriever 简单用法 ---
question = "任务分解的方法有哪些？"

# # 假设 vectordb 已经存在
# retriever_from_llm = MultiQueryRetriever.from_llm(
#     retriever=vectordb.as_retriever(), llm=llm
# )

# unique_docs = retriever_from_llm.invoke(question)
# #INFO:langchain.retrievers.multi_query:Generated queries: ['1. 常见的任务分解技术或策略有哪些？  ', '2. 如何将一个复杂任务拆解成多个子任务？  ', '3. 在项目管理或执行中，有哪些实用的任务拆分方法？']
# print(f"Original question: {question}")
# print(f"Retrieved unique docs count: {len(unique_docs)}")

# --- MultiQueryRetriever 自定义提示 (需要 LineListOutputParser) ---
from langchain.prompts import PromptTemplate
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.output_parsers import BaseOutputParser
import re
from typing import List

class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines)) # Remove empty lines

output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""你是一位 AI 语言模型助手。你的任务是根据给定的单个用户查询生成 3 个
不同版本的查询，以便从向量数据库中检索相关文档。通过对用户问题应用语义分解
或从不同角度改写问题，你可以生成多个视角的查询，帮助用户克服
基于距离的相似性搜索的一些局限性。请提供这些用换行符分隔的替代问题。
原始问题：{question}""",
)

llm_chain = QUERY_PROMPT | llm | output_parser

# 假设 vectordb 已经存在
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain # 注意这里 parser_key 不需要了，因为链直接输出列表
)

unique_docs_custom = retriever.invoke(question)
print(f"Retrieved unique docs count (custom prompt): {len(unique_docs_custom)}")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 常见的任务分解技术或策略有哪些？  ', '2. 如何将一个复杂任务拆解为多个子任务？  ', '3. 在项目管理或AI规划中，有哪些任务分割的实践方法？']


Retrieved unique docs count (custom prompt): 7


###### 分解/Step-Back (概念性)

此示例展示了如何使用 LangChain 的 PromptTemplate 和 LLM（通过 | 操作符链接，即 LCEL）来实现三种不同的查询转换：

- 重写：使查询更具体。
- Step-back：生成更通用的背景查询。
- 分解：将复杂查询拆分为多个子查询。 核心在于设计合适的提示模板来引导 LLM 完成所需的转换任务。实际应用中，这些转换后的查询将用于后续的检索步骤。

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

# --- 设置 ---
re_write_llm = llm
step_back_llm = llm
sub_query_llm = llm

# --- 查询重写 (更具体) ---
query_rewrite_template = """将以下查询改写得更具体、更详细，以便更好地从知识库中检索信息。
原始查询：{original_query}
重写后的查询："""
query_rewrite_prompt = PromptTemplate(input_variables=["original_query"], template=query_rewrite_template)
query_rewriter = query_rewrite_prompt | re_write_llm

def rewrite_query(original_query):
    response = query_rewriter.invoke({"original_query": original_query})
    return response.content.strip()

# --- Step-back Prompting (更通用) ---
step_back_template = """对于以下问题，我们应该先解决哪个更普遍、更宽泛的问题，以帮助理解其背景？
原始查询：{original_query}
Step-back 查询："""
step_back_prompt = PromptTemplate(input_variables=["original_query"], template=step_back_template)
step_back_chain = step_back_prompt | step_back_llm

def generate_step_back_query(original_query):
    response = step_back_chain.invoke({"original_query": original_query})
    return response.content.strip()

# --- 查询分解 (子查询) ---
subquery_decomposition_template = """将以下复杂查询分解为一系列更简单的子查询，每个子查询都侧重于原始查询的一个特定方面。
原始查询：{original_query}
子查询（每行一个）："""
subquery_decomposition_prompt = PromptTemplate(input_variables=["original_query"], template=subquery_decomposition_template)
subquery_decomposer_chain = subquery_decomposition_prompt | sub_query_llm

def decompose_query(original_query: str):
    response = subquery_decomposer_chain.invoke({"original_query": original_query})
    sub_queries = [q.strip() for q in response.content.strip().split('\n') if q.strip()]
    # 移除可能的编号，例如 "1. "
    sub_queries = [q.split('. ', 1)[-1] if '. ' in q else q for q in sub_queries]
    return sub_queries

# --- 示例 ---
original_query = "气候变化对环境有哪些影响？"

rewritten = rewrite_query(original_query)
print(f"Original: {original_query}\nRewritten (Specific): {rewritten}\n")
# 输出示例: 气候变化对各种生态系统的具体影响有哪些，包括温度、降水模式、海平面和生物多样性的变化？

step_back = generate_step_back_query(original_query)
print(f"Original: {original_query}\nStep-back (General): {step_back}\n")
# 输出示例: 气候变化的一般影响是什么？

sub_queries = decompose_query(original_query)
print(f"Original: {original_query}\nSub-queries:")
for i, sq in enumerate(sub_queries):
    print(f"{i+1}. {sq}")
# 输出示例:
# 1. 气候变化如何影响生物多样性和生态系统？
# 2. 气候变化对海洋状况和海洋生物有何影响？
# 3. 气候变化如何影响天气模式和极端天气事件？
# 4. 气候变化对陆地环境（如森林和沙漠）有何影响？

Original: 气候变化对环境有哪些影响？
Rewritten (Specific): 以下是重写后的更具体、更详细的查询版本，涵盖气候变化的多个关键影响维度，便于从知识库中精准检索信息：

---
**重写后的查询：**  
1. **生态系统层面**  
   - 气候变化如何改变陆地生态系统（如森林、草原）的生物多样性和物种分布？  
   - 海洋酸化和水温上升对珊瑚礁及海洋生物链的具体影响是什么？  
   - 极地冰川融化对北极熊、企鹅等极地物种的生存威胁有哪些实证数据？  

2. **自然灾害与极端天气**  
   - 全球变暖与飓风、干旱、野火等极端天气事件的频率和强度增加有何关联？  
   - 近20年因气候变化引发的洪涝灾害在亚洲和非洲的典型案例及经济损失数据？  

3. **人类社会经济影响**  
   - 农业：主要粮食作物（如小麦、水稻）的产量因温度上升和降水模式改变受到哪些影响？  
   - 公共卫生：疟疾、登革热等热带疾病向温带地区扩散与气候变化的关联性如何？  
   - 基础设施：海平面上升对沿海城市（如迈阿密、孟买）的长期威胁及当前应对措施？  

4. **长期不可逆效应**  
   - 若全球升温超过2°C，可能触发哪些临界点（如亚马逊雨林退化、永冻土融化）？  
   - 当前冰川退缩速率对2080年全球淡水供应的预测影响？  

5. **区域差异**  
   - 撒哈拉以南非洲与北欧国家在气候变化脆弱性上的关键差异是什么？  
   - 小岛屿发展中国家（如马尔代夫）面临的海平面上升风险是否被现有模型低估？  

--- 

**改写说明：**  
1. **结构化拆分**：将宽泛问题分解为5个明确维度，避免模糊检索。  
2. **具体指标**：要求实证数据、案例、时间范围（如"近20年"）和地理标签。  
3. **科学术语**：使用"海洋酸化""临界点"等专业词汇提高匹配精度。  
4. **对比视角**：包含区域差异、长期/短期影响等对比维度。  

此版本更适合检索学术文献、政策报告或数据库中的结构化数据。

Original: 气候变化对环境有哪些影响？
Step-back (General): 为了更全面地理解“气候变化对环境有哪些影响”这一具体问题，建议先探讨以下更普遍、更宽泛的背景问

##### LlamaIndex - 查询转换 (HyDE 和多步分解) 

**问题：** 在向量空间中，用户的简短查询与其对应的、信息丰富的答案文档块之间可能存在语义鸿沟（semantic gap）。也就是说，查询的嵌入向量与最相关答案文档的嵌入向量可能距离较远，导致标准相似性搜索效果不佳。

**机制：** HyDE (Hypothetical Document Embeddings) 提出了一种创新的解决方案：

1.  **生成假设性文档：** 给定用户查询，首先使用一个指令遵循（instruction-following）的 LLM（如 GPT-3.5）来生成一个假设性的、理想化的答案文档。这个文档可能包含事实错误，但其目的是捕捉查询的核心意图和预期答案的结构与内容。例如，对于查询“什么是 HyDE？”，LLM 可能生成一段解释 HyDE 原理的文字。
2.  **嵌入假设性文档：** 使用与索引真实文档相同的嵌入模型，将这个生成的假设性文档转换成一个嵌入向量。
3.  **使用假设性嵌入进行搜索：** 将这个假设性文档的嵌入向量（而不是原始查询的嵌入向量）作为查询向量，在向量数据库中执行相似性搜索，以找到与之最相似的真实文档块。

**原理：** HyDE 的核心假设是，一个好的答案文档在语义上更接近另一个（即使是假设的）答案文档，而不是提出问题的查询本身。通过生成一个与查询意图高度相关的假设性答案，HyDE 创造了一个更有效的“锚点”来在向量空间中定位相关的真实文档。

**优势：**

* **提高检索准确性：** 通过更好地匹配查询意图，尤其是在查询和文档表达方式差异较大时，可以提升检索效果。
* **零样本能力：** HyDE 不需要针对特定任务进行相关性标注数据的训练或微调，使其能够很好地泛化到新领域。
* **减少幻觉：** 通过检索与假设性（理想化）答案更相关的文档，可能有助于减少最终生成阶段的幻觉。

**局限性：**

* **依赖 LLM 生成质量：** HyDE 的效果取决于 LLM 生成假设性文档的相关性和质量。如果 LLM 对于某个主题完全不了解或产生严重错误的信息，生成的假设性文档可能误导检索过程。
* **潜在的错误引入：** 假设性文档中的事实错误虽然不直接进入最终答案，但可能影响检索到的文档集。
* **多语言挑战：** 在多语言场景下，LLM 在非英语资源上的生成能力以及对比编码器（用于嵌入）在多语言上的饱和问题可能带来挑战。

尽管 HyDE 这样的技术颇具创新性，但也引入了新的潜在故障点：LLM 生成的假设性内容的质量。这凸显了一种权衡，即提高检索相关性可能需要承担由辅助 LLM 本身引入潜在不准确性或偏见的风险。

> 示例失败

In [99]:
%pip install llama-index-embeddings-ollama
!pip install llama-index

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/78/6c/13a556d33b9cf1b00bad3daa6c4a13172f5f79f166476a49a6a69a3a5818/llama_index_embeddings_ollama-0.6.0-py3-none-any.whl (3.4 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/33/3f/164de150e983b3a16e8bf3d4355625e51a357e7b3b1deebe9cc1f7cb9af8/ollama-0.4.8-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [104]:
!pip install llama-index-llms-langchain

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/97/4d/deab35c0a36746f8aec280ca66cee1346db7fe7c4c0d8a8987f59a964cf3/llama_index_llms_langchain-0.6.1-py3-none-any.whl (6.1 kB)


In [108]:
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.core.indices.query.query_transform import HyDEQueryTransform, StepDecomposeQueryTransform
from llama_index.core.query_engine import TransformQueryEngine, MultiStepQueryEngine
from llama_index.embeddings.ollama import OllamaEmbedding

# --- 1. 配置全局嵌入模型 ---
print("配置 Ollama 嵌入模型...")
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text",  # 确保已通过 `ollama pull nomic-embed-text` 下载此模型
    base_url="http://localhost:11434", # Ollama API 服务地址
    ollama_additional_kwargs={"mirostat": 0} # 可选的 Ollama 特定参数
)
Settings.embed_model = ollama_embedding
Settings.llm = llm
print(f"全局嵌入模型已设置为: {Settings.embed_model.model_name}")

# --- 2. 加载文档 ---
# 创建一个示例数据目录和文件（如果不存在）
if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("data/example.txt"):
    with open("data/example.txt", "w", encoding="utf-8") as f:
        f.write("这是第一个示例文档，关于 LlamaIndex 和 Ollama。\n")
        f.write("LlamaIndex 是一个用于构建 LLM 应用的数据框架。\n")
if not os.path.exists("data/another_example.txt"):
    with open("data/another_example.txt", "w", encoding="utf-8") as f:
        f.write("这是第二个示例文档。\n")
        f.write("Ollama 允许在本地运行大型语言模型和嵌入模型。\n")

print("加载文档...")
# 从./data 目录加载文档
documents = SimpleDirectoryReader("./data").load_data()
print(f"成功加载 {len(documents)} 个文档。")

# --- 3. 创建 VectorStoreIndex ---
# `from_documents` 将自动使用 Settings.embed_model
print("开始创建 VectorStoreIndex...")
index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True # 显示索引构建进度条
)
print("VectorStoreIndex 创建完成！")
print(f"索引 ID: {index.index_id}")

# # --- 4. （可选）验证索引 ---
# print("尝试进行基本查询...")
# query_engine = index.as_query_engine()
# response = query_engine.query("什么是 Ollama？")
# print("\n查询: 什么是 Ollama？")
# print("响应:")
# print(response)

# response = query_engine.query("LlamaIndex 的用途是什么？")
# print("\n查询: LlamaIndex 的用途是什么？")
# print("响应:")
# print(response)

配置 Ollama 嵌入模型...
全局嵌入模型已设置为: nomic-embed-text
加载文档...
成功加载 3 个文档。
开始创建 VectorStoreIndex...


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/37 [00:00<?, ?it/s]

VectorStoreIndex 创建完成！
索引 ID: c324b69b-a288-4b22-ba13-0b94e5ed5675


- HyDE 查询转换 

In [114]:
# --- HyDE 查询转换 ---
query_str_hyde = "paul graham 去 RISD 后做了什么？"
hyde = HyDEQueryTransform(include_original=True) # include_original=True 表示同时使用原始查询和假设文档的嵌入
base_query_engine = index.as_query_engine()
hyde_query_engine = TransformQueryEngine(base_query_engine, query_transform=hyde)

response_hyde = hyde_query_engine.query(query_str_hyde)
print("HyDE Query Response:\n", response_hyde)

- 多步查询分解

In [125]:
!pip install llama_index

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [135]:
from llama_index.llms.langchain import LangChainLLM

wrapped_llm = LangChainLLM(llm=llm)
print(f"LLM Wrapper Type: {type(wrapped_llm)}")
Settings.llm = wrapped_llm

# --- 多步查询分解 ---
query_str_multi_step = "作者创办的加速器项目的第一批成员有谁？"
# 假设 llm 实例已创建
step_decompose_transform = StepDecomposeQueryTransform(llm=wrapped_llm, verbose=True)
multi_step_query_engine = MultiStepQueryEngine(
    query_engine=base_query_engine,
    query_transform=step_decompose_transform,
)

response_multi_step = multi_step_query_engine.query(query_str_multi_step)
print("\nMulti-Step Query Response:\n", response_multi_step)

LLM Wrapper Type: <class 'llama_index.llms.langchain.base.LangChainLLM'>
[llm/start] [llm:ChatDeepSeek] Entering LLM run with input:
{
  "prompts": [
    "Human: The original question is as follows: 作者创办的加速器项目的第一批成员有谁？\nWe have an opportunity to answer some, or all of the question from a knowledge source. Context information for the knowledge source is provided below, as well as previous reasoning steps.\nGiven the context and previous reasoning, return a question that can be answered from the context. This question can be the same as the original question, or this question can represent a subcomponent of the overall question.It should not be irrelevant to the original question.\nIf we cannot extract more information from the context, provide 'None' as the answer. Some examples are given below: \n\nQuestion: How many Grand Slam titles does the winner of the 2020 Australian Open have?\nKnowledge source context: Provides names of the winners of the 2020 Australian Open\nPrevious reasonin

##### 基于 LLM / 交叉编码器的重排序 (Re-ranking)

**问题：** 初始的检索阶段（例如，基于向量相似性的搜索）通常是为了速度和召回率而设计的，可能会返回大量候选文档，但其中一些文档的相关性可能不高，或者排名不理想。此外，研究表明 LLM 在处理放置于上下文窗口中间的信息时能力会下降（“中间丢失”问题，lost in the middle），因此将最相关的文档放在上下文的开头或结尾至关重要。

**机制：** 重排序是在初始检索之后增加的一个阶段，使用一个更强大、更精确但通常也更慢的模型来重新评估初始候选文档集的相关性，并根据新的相关性分数对它们进行重新排序。目标是确保最终传递给生成 LLM 的文档是 K 个最相关的文档。

**主要方法：**

* **交叉编码器（Cross-Encoders）：** 这是一种特殊类型的 Transformer 模型（通常基于 BERT），它将查询和单个候选文档同时作为输入，并输出一个表示它们之间相关性的分数。与用于初始检索的双编码器（Bi-encoders，分别编码查询和文档，然后计算相似度）相比，交叉编码器能够更深入地交互和理解查询与文档之间的细微关系，因此通常能提供更准确的相关性判断。但缺点是计算成本较高，因为它需要对每个查询-文档对都进行一次完整的模型推理。
* **基于 LLM 的重排序：** 直接利用通用 LLM 的能力来评估相关性。这可以通过多种方式实现：
    * **评分：** 设计一个提示，要求 LLM 为每个候选文档相对于查询的相关性打分（例如，1 到 5 分）。
    * **成对比较：** 向 LLM 提供查询和一对候选文档，并询问哪个文档与查询更相关。通过多次成对比较，可以推断出整体的排名。

**优势：**

* **提高精度：** 将真正最相关的文档排到最前面，过滤掉噪声和不太相关的结果。
* **优化 LLM 输入：** 为生成 LLM 提供更高质量、更集中的上下文，有助于生成更准确、更相关的答案。
* **缓解“中间丢失”问题：** 通过将最关键的信息放在上下文窗口的有利位置（通常是开头），提高 LLM 利用这些信息的可能性。

**考虑因素：**

* **计算成本与延迟：** 重排序，尤其是使用大型 LLM 或对大量候选文档使用交叉编码器时，会增加额外的计算开销和处理时间。需要在精度提升和性能影响之间进行权衡。

LLM 辅助检索的兴起催生了一个反馈循环：更好的检索带来更好的 RAG 输出，而 LLM 辅助检索的能力又推动了对更复杂 RAG 架构（如模块化 RAG）的进一步研究。这表明检索和生成组件之间的协同作用正变得日益紧密和复杂。

**专用模型/服务：**

- RankLLM (LangChain)：一个灵活的重排序框架，支持多种模型（如 RankZephyr, RankVicuna, MonoT5, DuoT5, GPT 等）和不同的排序范式（列表式、成对式、点对式）。需要注意的是，某些模型（如 RankZephyr）对硬件资源（特别是 VRAM）有较高要求 。   
- NVIDIA NeMo Reranker (LangChain)：一个经过 GPU 加速优化的模型，专门用于计算给定段落包含问题答案的概率分数 。通过 - - LangChain 的 NVIDIARerank 类和 ContextualCompressionRetriever 集成。   
- FlashRank (LangChain)：一个轻量级的 Python 库，基于优化的交叉编码器实现快速重排序 。   
- Mixedbread AI Reranker (Haystack)：一个商业化的重排序服务/模型，通过 Haystack 的特定组件 (MixedbreadAIReranker) 集成 。   
- 其他 LangChain 集成：LangChain 还集成了 Cohere, VoyageAI, Jina 等多种第三方重排序服务 。   
- Haystack Rankers API：Haystack 框架提供了一系列内置的 Ranker 组件，它们基于不同的逻辑进行排序，例如 LostInTheMiddleRanker（基于“中间丢失”现象优化排序）、MetaFieldRanker（基于文档元数据字段排序）、TransformersSimilarityRanker（基于交叉编码器语义相似度）、SentenceTransformersDiversityRanker（考虑相关性的同时最大化结果多样性）。   


**代码示例**

In [3]:
# 导入所需模块
import getpass
import os
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# 导入 DeepSeek (Assuming it's available in langchain_community, adjust if needed)
from langchain import hub
from langchain_deepseek import ChatDeepSeek


os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("输入您的 DeepSeek API 密钥：")

# --- Configuration ---
# Ensure your DeepSeek API key is set as an environment variable
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
if not DEEPSEEK_API_KEY:
    raise ValueError("DEEPSEEK_API_KEY environment variable not set. Please set it.")

# Specify the DeepSeek model you want to use (e.g., "deepseek-chat", "deepseek-coder")
DEEPSEEK_MODEL_NAME = "deepseek-chat" # <-- CHANGE IF NEEDED

# --- Initialize Components ---

# 初始化 DeepSeek 模型
try:
    # Use ChatDeepseek for chat model interface
    llm = ChatDeepSeek(
        model=DEEPSEEK_MODEL_NAME,
        temperature=0
        # You might add other parameters like 'max_tokens' if needed
    )
    # Test connection (optional, but good practice)
    llm.invoke("Hello!")
    print(f"Successfully connected to DeepSeek model: {DEEPSEEK_MODEL_NAME}")
except Exception as e:
    print(f"Error connecting to DeepSeek model '{DEEPSEEK_MODEL_NAME}': {e}")
    print("Please ensure your API key is correct and the model name is valid.")
    exit() # Exit if connection fails

输入您的 DeepSeek API 密钥： ········


Successfully connected to DeepSeek model: deepseek-chat


In [6]:
# -*- coding: utf-8 -*-

# --- 核心 LangChain 导入 ---
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS # 使用 FAISS 作为示例
from langchain_community.embeddings import OllamaEmbeddings

# --- 1. 准备组件 ---
texts = [
    Document(page_content="人工智能（AI）是计算机科学的一个分支，致力于创造能够执行通常需要人类智能的任务的机器。"),
    Document(page_content="机器学习（ML）是人工智能的一个子集，专注于开发能够从数据中学习并做出决策或预测的系统。"),
    Document(page_content="深度学习（DL）是机器学习的一个子领域，它使用受人脑结构启发的神经网络（特别是深度神经网络）来学习复杂模式。"),
    Document(page_content="自然语言处理（NLP）是人工智能领域的一部分，涉及计算机理解、解释和生成人类语言的能力。"),
    Document(page_content="FAISS 是一个由 Facebook AI 开发的库，用于高效地进行相似性搜索和密集向量聚类。它特别适合处理大规模数据集。"),
    Document(page_content="Ollama 是一个工具，可以让你在本地轻松运行大型语言模型（LLM），例如 Llama 2、Mistral 等。"),
    Document(page_content="LangChain 是一个框架，旨在简化基于大型语言模型的应用程序的开发，例如聊天机器人、问答系统和数据分析工具。")
]
ollama_model_name = "nomic-embed-text"
embeddings = OllamaEmbeddings(model=ollama_model_name)
print(f"使用 Ollama 嵌入模型: {ollama_model_name}")

# --- 2. 初始化基础检索器 ---
#    使用你的文档和嵌入模型创建一个向量存储，并从中获取检索器
#    确保安装了 'faiss-cpu' 或 'faiss-gpu' (`pip install faiss-cpu`)
try:
    vectorstore = FAISS.from_documents(texts, embeddings)
    base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # 初始检索 5 个文档
except Exception as e:
    print(f"创建基础检索器时出错。请确保 'texts' 和 'embedding_function' 有效，并且安装了 FAISS: {e}")
    exit()

# --- 3. 初始化压缩器 ---
#    LLMChainExtractor 使用 LLM 从检索到的文档中提取相关部分
compressor = LLMChainExtractor.from_llm(llm=llm)

# --- 4. 创建 ContextualCompressionRetriever ---
#    它组合了基础检索器和压缩器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# --- 5. 使用检索器进行查询 ---
query = "什么是机器学习？" # <-- 输入你的查询
print(f"\n正在使用查询: '{query}'")

try:
    # 调用检索器：获取初始文档，然后使用 LLM 进行压缩/提取
    compressed_docs = compression_retriever.invoke(query)

    # --- 6. 打印结果 ---
    print("\n--- 压缩后的结果 ---")
    if compressed_docs:
        for i, doc in enumerate(compressed_docs):
            # compressed_docs 包含的是 LLM 提取出的相关片段
            print(f"结果 {i+1}: {doc.page_content}")
    else:
        print("压缩器未能找到相关的片段。")

except Exception as e:
    print(f"检索过程中出错。请确保所有组件（LLM、嵌入、FAISS）都已正确配置并可用: {e}")

使用 Ollama 嵌入模型: nomic-embed-text

正在使用查询: '什么是机器学习？'

--- 压缩后的结果 ---
结果 1: 机器学习（ML）是人工智能的一个子集，专注于开发能够从数据中学习并做出决策或预测的系统。


### 模块五：构建基于个人数据的应用

#### LangChain 表达式语言 (LCEL)：声明式标准

##### 概念
LangChain 表达式语言（LCEL）是 LangChain 中用于组合现有 **Runnable（可运行组件）** 以构建新 Runnable 的一种声明式方法 。开发者描述期望达成的 目标，而非具体执行的 步骤，这使得 LangChain 能够优化底层的执行流程 。使用 LCEL 构建的任何链本身也是一个 Runnable，遵循统一的接口规范 。   

##### 优势

使用 LCEL 构建链具有多项优势，LangChain 会对其执行进行优化：

- **优化执行**: LCEL 支持通过 RunnableParallel 实现组件的并行运行，并通过 Runnable Batch API 并行处理多个输入，显著降低延迟 。同时，任何 LCEL 链都天然支持通过 Runnable Async API 进行异步调用（如 .ainvoke(), .astream()），这对于需要处理大量并发请求的服务器环境至关重要 。   
- **简化流式处理**: LCEL 使得链的输出流式化变得简单，允许在链执行过程中逐步返回结果，优化语言模型的首个令牌响应时间（time-to-first-token）。   
- **可观测性**: 对于复杂的链，理解每一步的执行流程至关重要。LCEL 会自动将所有执行步骤记录到 LangSmith，提供全面的可观测性和调试能力 。   
- **可组合性与标准 API**: 所有 LCEL 链都遵循标准的 Runnable 接口，确保了与其他 LangChain 组件的无缝集成和嵌套组合 。   
- **易于部署**: 使用 LCEL 构建的链可以方便地通过 LangServe 进行生产部署 。

##### 基本组合方式

LCEL 提供了两种主要的组合原语：RunnableSequence（通常通过管道符 | 实现）和 RunnableParallel。

###### 顺序组合 (| 或 .pipe())
管道操作符 (|) 在 LCEL 中被重载，用于创建 RunnableSequence。它允许将多个 Runnable 对象按顺序链接起来，前一个 Runnable 的输出自动成为下一个 Runnable 的输入 。   

In [8]:
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("讲一个关于{topic}的笑话")
parser = StrOutputParser()

# 使用管道符 | 组合链
chain = prompt | llm | parser

在这个例子中，prompt、llm 和 parser 这三个 Runnable 对象被依次连接。当调用 `chain.invoke({"topic": "狗"})` 时，输入字典首先传递给 prompt 进行格式化，生成的提示字符串再传递给 llm，llm 的输出（一个聊天消息对象）最后传递给 parser 提取文本内容 。`.pipe()` 方法提供了与 `|` 等效的功能 。   

###### 并行组合 (RunnableParallel)

RunnableParallel 用于同时执行多个 Runnable 对象，并将相同的输入传递给每一个对象。这在需要对同一输入执行多个独立操作并合并结果时非常有用 。   

In [9]:
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI # 示例 LLM
from langchain_core.output_parsers import StrOutputParser

joke_prompt = PromptTemplate.from_template("讲一个关于{topic}的笑话")
fact_prompt = PromptTemplate.from_template("告诉我一个关于{topic}的简短事实")
parser = StrOutputParser()

# 使用 RunnableParallel 组合
parallel_chain = RunnableParallel({
    "joke": joke_prompt | llm | parser,
    "fact": fact_prompt | llm | parser
})

这里，RunnableParallel 接收一个字典，字典的键（"joke", "fact"）定义了并行分支的名称，值是对应的 Runnable 链。当调用 `parallel_chain.invoke({"topic": "猫"})` 时，输入会同时传递给 "joke" 分支和 "fact" 分支，它们并行执行。最终输出是一个包含 "joke" 和 "fact" 键及其对应执行结果的字典 。LCEL 支持将字典自动转换为 RunnableParallel，将函数自动转换为 RunnableLambda，以简化语法 。   


##### LCEL 适用性考量

虽然 LCEL 是 LangChain 推荐的构建链的标准方式，但并非所有场景下的最优选择。对于仅涉及单次 LLM 调用的简单任务，直接调用模型可能更简洁 。而对于包含复杂逻辑、条件分支、循环或多智能体交互的状态化工作流，LangGraph 提供了更强大的控制能力和状态管理机制，是更合适的选择 。LCEL 的价值在于其能够优化执行（异步、流式、并行）和提供可观测性，因此在中等复杂度的链（如典型的 RAG 流程）中，当需要这些特性时，LCEL 是理想的工具。开发者应根据具体任务的复杂度和对 LCEL 特定优势的需求来决定是否采用 LCEL，避免在过于简单的任务上增加不必要的抽象，或在过于复杂的任务上受其限制。   


#### DeepSeek 集成 (langchain-deepseek)

##### 关键初始化参数

实例化 ChatDeepSeek 时，一些关键参数包括 ：   

- model (str): 指定要使用的 DeepSeek 模型名称，例如 "deepseek-chat" 或 "deepseek-reasoner"。
- temperature (float): 控制采样温度，影响生成文本的随机性。较低的值使输出更确定，较高的值增加多样性。
- max_tokens (Optional[int]): 限制模型单次生成响应的最大 token 数量。
- api_key (Optional[str]): DeepSeek API 密钥。
- timeout (Optional[float]): API 请求的超时时间（秒）。
- max_retries (int): 请求失败时的最大重试次数。

##### 设置

In [10]:
!pip install -U langchain-deepseek

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


##### 基本调用
初始化 ChatDeepSeek 实例后，可以使用 .invoke() 方法进行调用，通常传入一个包含系统消息（system message）和用户消息（human message）的列表 。   

In [11]:
from langchain_deepseek import ChatDeepSeek
import os

# 确保 DEEPSEEK_API_KEY 环境变量已设置
os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("输入您的 DeepSeek API 密钥：")

# 或者 llm = ChatDeepSeek(api_key="...", model=...)
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)

messages = [
    ("system", "你是一个乐于助人的助手。"),
    ("human", "你好！"),
]
response = llm.invoke(messages)
print(response.content)

输入您的 DeepSeek API 密钥： ········


你好！😊 很高兴见到你！有什么我可以帮你的吗？


##### Runnable 接口

重要的是，ChatDeepSeek 类实现了 LangChain 标准的 Runnable 接口 。这意味着它可以无缝地集成到使用 LCEL 构建的链中，作为链的一个环节参与执行。   

#### DeepSeek 模型选择：deepseek-chat vs. deepseek-reasoner
DeepSeek API 提供了两款主要的聊天模型：deepseek-chat 和 deepseek-reasoner，它们各自具有不同的优势和适用场景 。   

- deepseek-chat (V3)
这款模型通常被称为 DeepSeek V3，是一个通用的、速度较快的模型 。它基于庞大的数据集（超过 15 万亿 token）进行训练，擅长处理对话、写作、总结和基本的推理任务 。其采用的混合专家（Mixture-of-Experts, MoE）架构有助于提高响应速度和效率 。如果应用场景是构建聊天机器人、写作助手或需要自然流畅交互的任务，deepseek-chat 通常是首选 。   

- deepseek-reasoner (R1)
这款模型通常被称为 DeepSeek R1，是为复杂推理、逻辑、数学和多步骤问题解决而设计的专用模型 。它在 V3 的基础上，通过强化学习和思维链（Chain-of-Thought, CoT）机制进行了优化，使其在需要深度分析和结构化解决方案的任务上表现出色 。然而，这种深度的思考过程也意味着 deepseek-reasoner 的响应速度通常比 deepseek-chat 慢，因为它在生成最终答案之前会进行内部的推理步骤 。


#### 构建高级问答 (QA) 系统

##### 目标
目标是构建一个能够根据检索到的个人文档片段，精确回答用户问题的系统，并且该系统应仅依赖于提供的上下文信息，避免编造答案。

##### 基于 LCEL 的核心 RAG QA
###### 流程
典型的检索增强生成（RAG）问答流程可以通过 LCEL 清晰地表达出来：用户的 question 输入，首先传递给 retriever 获取相关的 documents，然后将 question 和格式化后的 documents（作为 context）填入 prompt 模板，接着将 prompt 发送给 LLM 生成回答，最后使用 OutputParser 解析 LLM 的输出 [User Query]。

###### LCEL 优势

在此场景下使用 LCEL 的好处在于其声明式的可组合性、代码的清晰度，以及对异步和流式处理的潜在支持，这对于构建响应迅速的 QA 系统可能很有价值 。相比旧的 RetrievalQA 链，LCEL 提供了更好的灵活性和透明度 `[User Query]`。   

###### 处理“答案未找到”的情况

为了防止 LLM 在无法从提供的上下文中找到答案时“自由发挥”或产生幻觉，必须在提示（prompt）中明确指示。可以在 RAG 提示模板中加入类似“请仅根据提供的上下文回答问题。如果上下文中没有足够信息来回答问题，请明确说明你不知道答案，不要尝试猜测。”的指令 `[User Query]`。

##### 优化代码实现 (LCEL)
以下是使用 LCEL 构建个人数据 QA 系统的优化 Python 代码示例：

example.txt文本内容：
```txt
Project Phoenix 预算会议纪要

上次关于 Project Phoenix 预算的会议于 2024 年 5 月 15 日召开。与会人员包括 Alice、Bob 和 Charlie。

会议的主要议题是确定下一季度的预算分配。经过讨论，与会人员一致同意将 Project Phoenix 的预算增加 15%，以加速关键功能的开发。

具体来说，新增预算将用于以下方面：

*   增加研发团队的资源投入。
*   购买更先进的测试设备。
*   进行更广泛的用户调研。

Alice 负责跟进预算增加的审批流程。Bob 将负责制定详细的预算分配计划。Charlie 将负责与相关团队沟通预算调整情况。

会议结束时，大家对 Project Phoenix 的未来充满信心，并相信增加的预算将有助于项目的成功。
```

In [20]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings # 或其他 embedding 模型
from langchain_chroma import Chroma # 或其他 vector store
from langchain_deepseek import ChatDeepSeek
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader # 示例加载器
from langchain_core.prompts import ChatPromptTemplate
import os

# --- 假设之前的步骤已完成 ---
# 1. 加载和切分文档 (示例)
loader = TextLoader("data/example.txt", encoding='utf-8')
# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_split_docs = text_splitter.split_documents(documents)
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_split_docs = text_splitter.split_documents(data)

# 2. 创建 Embeddings 和 VectorStore (示例)
ollama_model_name = "nomic-embed-text"
embeddings = OllamaEmbeddings(model=ollama_model_name)
vectorstore = Chroma.from_documents(documents=all_split_docs, embedding=embeddings)

# 3. 创建 Retriever
# 假设 vectorstore 已经创建并填充了数据
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 获取最相关的 3 个文档块

# 4. 初始化 DeepSeek LLM
# 确保 DEEPSEEK_API_KEY 环境变量已设置
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)
print("ChatDeepSeek 模型已初始化。")

# --- 假设以上变量 retriever 和 llm 已定义 ---

# 5. 定义包含明确指令的 RAG 提示模板
rag_prompt_template = """请仅根据下面提供的上下文来回答问题。

上下文:
{context}

问题: {question}

如果根据上下文无法回答问题，请直接回答“根据我所掌握的信息，无法回答该问题。”，不要编造答案。"""
prompt = ChatPromptTemplate.from_template(rag_prompt_template)

# 6. 定义文档格式化函数
def format_docs(docs):
    """将 Document 对象列表格式化为单一字符串"""
    return "\n\n".join(doc.page_content for doc in docs)

# 7. 构建 RAG 链 (使用 LCEL)
# RunnableParallel 用于将 retriever 的输出和原始问题并行传递
# retriever | format_docs: 将检索到的文档列表格式化为字符串
# RunnablePassthrough(): 将原始问题直接传递下去
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

# 8. 测试 QA
question_personal = "关于 Project Phoenix 的预算，上次会议达成了什么结论？" # 假设此信息在个人文档中
print(f"用户问题: {question_personal}")
# 假设 retriever 和 llm 已正确配置并可用
answer_personal = rag_chain.invoke(question_personal)
print(f"AI 回答: {answer_personal}") # 预期从文档中得到答案

question_general = "法国的首都是哪里？" # 假设此信息不在个人文档中
print(f"\n用户问题: {question_general}")
answer_general = rag_chain.invoke(question_general)
print(f"AI 回答: {answer_general}") # 预期回答类似“根据我所掌握的信息，无法回答该问题。”

# --- 清理 (如果需要) ---
# vectorstore.delete_collection()

ChatDeepSeek 模型已初始化。
用户问题: 关于 Project Phoenix 的预算，上次会议达成了什么结论？
AI 回答: 关于 Project Phoenix 的预算，上次会议达成的一致结论是：将预算增加 15%，以加速关键功能的开发。新增预算将用于增加研发团队的资源投入、购买更先进的测试设备以及进行更广泛的用户调研。Alice、Bob 和 Charlie 分别负责跟进审批流程、制定详细分配计划和沟通调整情况。

用户问题: 法国的首都是哪里？
AI 回答: 根据我所掌握的信息，无法回答该问题。


- Retriever 设置: 代码中明确展示了如何从 vectorstore 创建 retriever，并设置了 `search_kwargs={"k": 3}` 来获取最相关的 3 个文档块。
- LCEL 结构: 链的构建清晰地展示了 LCEL 的并行 `({...})` 和顺序 (|) 组合。`RunnableParallel (通过字典语法 {...})` 确保了上下文 (context) 和原始问题 (question) 都被传递到 prompt。RunnablePassthrough() 是一个特殊的 Runnable，它将输入原封不动地传递下去，这里用于将用户的原始问题传递给 prompt。
- 明确指令: 自定义的 rag_prompt_template 包含了处理答案不在数据中情况的关键指令。
- 测试用例: 包含了两个测试问题，一个旨在测试系统能否从（假设存在的）个人数据中找到答案，另一个测试系统在信息缺失时是否能遵循指令回答“不知道”。

#### 开发对话式 RAG (聊天机器人)

##### 目标

目标是创建一个能够进行多轮对话的聊天机器人。这个机器人不仅能回答基于个人数据的问题，还能记住之前的对话内容，并利用这些历史信息来理解上下文相关的后续问题。

##### 记忆与状态管理的重要性

标准的 LLM 调用是无状态的，每次交互都是独立的`[User Query]`。为了实现连贯、自然的对话，聊天机器人必须具备“记忆”能力，即存储和访问之前的交流内容`[User Query]`。这种记忆使得机器人能够理解指代（例如，用户问“它怎么样了？”时，机器人知道“它”指的是上一轮提到的“Project Phoenix”）、处理依赖上下文的后续问题（例如，“那它的成本呢？”），并提供更加个性化和流畅的交互体验。

##### 现代记忆管理：LangGraph 持久化

###### 演进路径

LangChain 中处理聊天历史的方式经历了演变。早期版本依赖于专门的内存类，如 `ConversationBufferMemory`。随后，LCEL 引入了 `RunnableWithMessageHistory``[User Query]`，提供了一种将历史管理包装到 `Runnable` 链中的方式。然而，根据最新的 LangChain 文档和推荐，对于需要记忆功能的新应用，**强烈推荐使用 LangGraph 及其内置的持久化功能**。

###### 为何选择 LangGraph?

LangGraph 提供了一个更通用、更强大的状态管理框架，其优势在于：

* **持久化任意状态**: LangGraph 不仅能持久化聊天消息历史，还能管理应用程序所需的任何其他状态信息（例如，用户偏好、中间计算结果等）。这对于构建复杂的、需要维护多种状态的智能体至关重要。
* **内置持久化层**: 它自带持久化机制，可以轻松地将状态存储在内存中（用于快速原型设计或简单场景），或者无缝对接外部存储后端，如 SQLite、PostgreSQL 或 Redis，以实现更健壮、可扩展的持久化。
* **简化的多轮状态管理**: LangGraph 的架构天然适合处理多轮交互，自动处理状态的加载和保存，简化了开发流程。

###### 与 RunnableWithMessageHistory 的对比

`RunnableWithMessageHistory` 主要专注于管理聊天消息历史，对于需要管理更复杂状态的应用来说，其灵活性有限。虽然 `RunnableWithMessageHistory` 仍然可用，并且可以作为 LangGraph 中的一个节点使用，但 LangChain 的官方指导明确建议新项目优先采用 LangGraph 的原生持久化机制。

这种从独立内存类到 `RunnableWithMessageHistory` 再到 LangGraph 的演进，反映了构建日益复杂的对话式和智能体应用的趋势。简单的消息历史记录往往不足以支撑高级交互，LangGraph 提供了一个更能满足这些需求的、统一且强大的状态管理解决方案，代表了 LangChain 生态系统中处理记忆和状态的当前最佳实践。

##### 历史感知检索 (create_history_aware_retriever)

###### 问题背景

在多轮对话中，用户经常提出依赖先前对话内容的问题，例如在讨论了“Project Phoenix”之后问“那它的成本呢？”。如果直接将这种省略了主语的后续问题发送给基于向量相似度的检索器，通常效果不佳，因为检索器需要一个独立的、信息完整的查询才能有效工作。

###### 解决方案：查询语境化

为了解决这个问题，需要在检索之前增加一个“查询语境化”或“查询重写”的步骤。这一步利用 LLM 和聊天历史记录，将用户当前可能不完整的输入改写成一个可以独立理解的查询。例如，将“那它的成本呢？”结合历史记录改写成“Project Phoenix 的成本是多少？”。

###### create_history_aware_retriever 函数

LangChain 提供了 `create_history_aware_retriever` 函数专门用于实现此功能。

* **目的**: 该函数创建一个链（`Runnable`），它接收当前的聊天历史和用户最新的输入，利用一个 LLM 来生成一个适合检索的、上下文完整的搜索查询，然后将这个重写后的查询传递给底层的基本检索器。
* **参数**:
    * `llm`: 用于重写查询的语言模型实例（`Runnable`）。
    * `retriever`: 底层的基本检索器（例如，从向量存储创建的 `Runnable`），它接收字符串查询并返回文档列表。
    * `prompt`: 一个特殊的提示模板，用于指导 LLM 进行查询重写。此模板必须包含用于插入聊天历史的 `MessagesPlaceholder(variable_name="chat_history")` 和用于插入当前用户输入的占位符（通常是 `{input}`）。
* **返回类型**: 一个 `Runnable` 对象。这个 `Runnable` 接收一个包含 `input` 和 `chat_history` 键的字典作为输入，并输出一个 `Document` 对象列表。
* **示例提示**: 用户查询中提供的 `contextualize_q_system_prompt` 和 `contextualize_q_prompt` 就是一个很好的例子，它指示 LLM 根据历史和最新问题重构问题，如果不需要重构则返回原问题。
* **手动传递历史**: 需要注意，除非将 `create_history_aware_retriever` 创建的链包装在像 LangGraph 这样的状态管理系统中，否则每次调用时都需要手动传入 `chat_history`。

##### 构建对话链 (create_retrieval_chain)

###### 目的

在通过历史感知检索器获取了与重写后查询相关的文档之后，需要将这些文档与原始问题（以及可能的历史记录）结合起来，最终生成回答。`create_retrieval_chain` 函数用于构建这个后续的处理链。

###### create_retrieval_chain 函数

* **作用**: 此函数将一个检索器（通常是前面创建的 `history_aware_retriever`）和一个文档处理链（`combine_docs_chain`）组合起来。
* **参数**:
    * `retriever`: 实现了检索功能的 `Runnable`（例如，`history_aware_retriever` 的输出），它返回文档列表。
    * `combine_docs_chain`: 一个 `Runnable`，负责接收原始输入、检索到的文档（通常在 `context` 键下）以及聊天历史（`chat_history`），并生成最终的字符串回答。这个链通常使用 `create_stuff_documents_chain` 创建。
* **返回类型**: 一个 `Runnable` 对象。其输出通常是一个字典，至少包含 `answer`（最终回答）和 `context`（检索到的文档）等键。

###### create_stuff_documents_chain 函数

这个辅助函数通常用于创建 `create_retrieval_chain` 所需的 `combine_docs_chain`。它的作用是接收一个 LLM 和一个提示模板，并将检索到的文档列表格式化后“塞入”（stuff）到提示模板中指定的 `context` 变量位置，然后将填充好的提示传递给 LLM 生成答案。

##### 优化代码实现 (LangGraph)

以下内容介绍了如何使用 LangGraph 来构建和管理对话式 RAG 应用的状态，这是当前推荐的方法，取代了原用户查询中基于 `RunnableWithMessageHistory` 的示例。 (请注意：原始输入未包含 LangGraph 的具体代码示例)。

In [41]:
%pip install -U --quiet langgraph langchain-openai langchain-community tiktoken

Note: you may need to restart the kernel to use updated packages.


LangGraph 相关导入 

In [44]:
import os
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_deepseek import ChatDeepSeek
from langchain_community.vectorstores import FAISS # Using FAISS for example
from langchain_community.embeddings import FakeEmbeddings # Using FakeEmbeddings for example
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage

# --- LangGraph Related Imports ---
from typing import List, TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

初始化

In [47]:
# 假定 LLM 和检索器已定义 (实际代码中会初始化)
# 替换为您的实际 LLM 和 API 密钥设置
# 重要：设置 DeepSeek API 密钥环境变量或直接传入
# os.environ = "YOUR_API_KEY" # 设置 API 密钥环境变量的示例 (请使用安全的方式)
try:
    # 确保 API 密钥可用，例如通过环境变量 DEEPSEEK_API_KEY
    llm = ChatDeepSeek(model="deepseek-chat", temperature=0)  # 初始化 DeepSeek LLM
except Exception as e:
    print(f"Error initializing LLM. Ensure API key is set. Error: {e}")  # 初始化 LLM 失败时打印错误
    llm = None  # 初始化失败时将 llm 设为 None

# 示例向量存储设置 (请替换为您的实际向量存储)
try:
    vectorstore = FAISS.from_texts(  # 从文本列表创建 FAISS 向量存储
        ["会议决定将 Project Phoenix 的预算增加 10% 用于额外的原型设计。",  # 示例文档1
         "增加预算的决定是因为初步测试结果非常有希望，表明投入更多资源可以显著加快上市时间。"],  # 示例文档2
        embedding=FakeEmbeddings(size=768)  # 使用 FakeEmbeddings 作为嵌入函数（示例）
    )
    base_retriever = vectorstore.as_retriever()  # 将向量存储转换为检索器
except Exception as e:
    print(f"Error initializing Vector Store/Retriever. Error: {e}")  # 初始化向量存储/检索器失败时打印错误
    base_retriever = None  # 初始化失败时将 base_retriever 设为 None

1. 定义 LangGraph 状态

In [48]:
# 1. 定义 LangGraph 状态
class ConversationState(TypedDict):  # 定义一个 TypedDict 作为图的状态结构
    messages: List[BaseMessage]  # 状态中包含一个 'messages' 列表，存储所有消息

2. 定义提示模板

In [51]:
# --- 2. Define Prompt Templates ---
# 2. 定义提示模板
# 2.1 Query Recontextualization Prompt
# 2.1 查询重构提示
# ***** 修改 1: 将占位符变量名更改为 "messages" *****
contextualize_q_system_prompt = """根据聊天记录和用户最新的问题，将用户问题改写成一个独立的、无需参考聊天记录就能理解的问题。请注意不要回答问题，只需在必要时重构问题，否则按原样返回。"""  # 重构问题的系统提示
contextualize_q_prompt = ChatPromptTemplate.from_messages([  # 创建重构问题的聊天提示模板
    ("system", contextualize_q_system_prompt),  # 添加系统消息
    MessagesPlaceholder(variable_name="messages"),  # 添加消息占位符，用于历史消息
    ("human", "{input}"),  # 添加人类消息占位符，用于最新问题
])

# 2.2 Final Question-Answering Prompt
# 2.2 最终问答提示
qa_system_prompt = """你是一个问答助手。请使用以下检索到的上下文来回答问题。如果不知道答案，就说不知道。回答尽量简洁，最多三句话。\n\n上下文:\n{context}"""  # 问答系统的系统提示
qa_prompt = ChatPromptTemplate.from_messages([  # 创建问答聊天提示模板
    ("system", qa_system_prompt),  # 添加系统消息
    ("human", "{input}"),  # 添加人类消息占位符，用于重构后的问题或原始问题
])


3. 创建核心 Runnable 组件

In [52]:
# --- 3. Create Core Runnable Components ---
# 3. 创建核心可运行组件
# 在继续之前确保 llm 和 base_retriever 有效
if llm and base_retriever:  # 检查 LLM 和检索器是否初始化成功
    # 3.1 History-Aware Retriever Runnable
    # 3.1 历史感知检索器可运行对象
    history_aware_retriever_runnable = create_history_aware_retriever(  # 创建历史感知检索器链
        llm, base_retriever, contextualize_q_prompt  # 使用 LLM, 基础检索器和重构提示
    )

    # 3.2 Question-Answering Generation Runnable
    # 3.2 问答生成可运行对象
    Youtube_runnable = create_stuff_documents_chain(llm, qa_prompt)  # 创建填充文档并生成答案的链

    # 3.3 Create the Full RAG Chain
    # 3.3 创建完整的 RAG 链
    # 此链现在需要 "messages" 和 "input" 作为检索步骤的输入
    # 它会将 "input" (可能已重构) 和 "context" 传递给问答步骤
    rag_chain = create_retrieval_chain(history_aware_retriever_runnable, Youtube_runnable)  # 创建完整的检索链 (检索 + 生成)
else:
    print("LLM or Retriever not initialized. Cannot create chains.")  # 如果 LLM 或检索器未初始化，打印错误
    rag_chain = None  # 设置 rag_chain 为 None

4. 定义 LangGraph 节点 

In [53]:
# --- 4. Define LangGraph Node ---
def execute_rag_chain_node(state: ConversationState):
    """Node: Executes the full RAG chain using the conversation history."""
    print("--- Executing RAG Chain Node ---")
    if not rag_chain:
        print("Error: RAG chain not available.")
        # Append an error message or handle appropriately
        error_message = AIMessage(content="Sorry, the RAG chain is not configured correctly.")
        return {"messages": state['messages'] + [error_message]}

    messages = state['messages']
    if not messages:
        print("Error: No messages in state.")
        # Return state to avoid breaking, but log error
        return state

    # The last message is the newest human input
    last_human_message = messages[-1]
    if not isinstance(last_human_message, HumanMessage):
        print(f"Error: Last message is not a HumanMessage: {last_human_message}")
        # Handle error - maybe return state or an error message
        error_message = AIMessage(content="Error processing input.")
        return {"messages": messages + [error_message]}

    input_content = last_human_message.content

    # The history is all messages *before* the last one
    chat_history = messages[:-1]

    # ***** CHANGE 2: Pass history using the key "messages" *****
    print(f"Input to rag_chain: input='{input_content}', history_len={len(chat_history)}")

    # Invoke the RAG chain
    try:
        response = rag_chain.invoke({
            "messages": chat_history, # Pass history under the key "messages"
            "input": input_content
        })
        print(f"RAG Chain Response: {response}")

        # Ensure 'answer' is in the response
        if "answer" in response:
             ai_response_content = response["answer"]
        else:
             print("Error: 'answer' key not found in RAG chain response.")
             ai_response_content = "Sorry, I encountered an issue generating a response."

        ai_message = AIMessage(content=ai_response_content)

        # Return the updated list of messages
        updated_messages = messages + [ai_message]
        return {"messages": updated_messages}

    except Exception as e:
        print(f"Error invoking rag_chain: {e}")
        # Append an error message to the state
        error_message = AIMessage(content=f"An error occurred: {e}")
        return {"messages": messages + [error_message]}

5. 构建并编译 LangGraph

In [54]:
# --- 5. Build and Compile LangGraph ---
workflow = StateGraph(ConversationState)

# Add the node
workflow.add_node("rag_node", execute_rag_chain_node) # Use the corrected node function name

# Define the entry point and edges
workflow.set_entry_point("rag_node")
workflow.add_edge("rag_node", END)

# Set up memory saver
memory = MemorySaver()

# Compile the graph
conversational_rag_app = workflow.compile(checkpointer=memory)

6. 示例对话 

In [55]:
# --- 6. Example Conversation ---
# Ensure the app compiled successfully
if conversational_rag_app:
    session_id = "user_session_lg_fixed_002"
    config = {"configurable": {"thread_id": session_id}}

    # First turn
    print("\n用户: 关于 Project Phoenix 的预算，上次会议达成了什么结论？")
    initial_input = {"messages": [HumanMessage(content="关于 Project Phoenix 的预算，上次会议达成了什么结论？")]}
    try:
        response1_state = conversational_rag_app.invoke(initial_input, config=config)
        if response1_state and 'messages' in response1_state and len(response1_state['messages']) > 1:
            ai_answer1 = response1_state["messages"][-1].content
            print(f"AI: {ai_answer1}")
        else:
            print("AI: 未能获取到回答。")
            print(f"State after first invoke: {response1_state}")
    except Exception as e:
        print(f"Error during first invocation: {e}")


    # Second turn (relies on history)
    print("\n用户: 为什么做出那个决定？")
    # Subsequent calls pass only the new message
    follow_up_input = {"messages": [HumanMessage(content="为什么做出那个决定？")]}
    try:
        response2_state = conversational_rag_app.invoke(follow_up_input, config=config)
        if response2_state and 'messages' in response2_state and len(response2_state['messages']) > 1:
             # Check if the last message is indeed an AIMessage
             if isinstance(response2_state["messages"][-1], AIMessage):
                 ai_answer2 = response2_state["messages"][-1].content
                 print(f"AI: {ai_answer2}")
             else:
                 print("AI: The last message was not from the AI. Check execution flow.")
                 print(f"Last message: {response2_state['messages'][-1]}")
        else:
            print("AI: 未能获取到回答。")
            print(f"State after second invoke: {response2_state}")
    except Exception as e:
        print(f"Error during second invocation: {e}")


    # Verify conversation history
    try:
        current_state = conversational_rag_app.get_state(config)
        print(f"\n会话 '{session_id}' 的当前状态 (消息):")
        if current_state and current_state.values and 'messages' in current_state.values:
            print(current_state.values['messages'])
        else:
            print("无法获取当前状态或消息列表为空。")
            print(f"Raw current_state: {current_state}")
    except Exception as e:
        print(f"获取状态时出错: {e}")

else:
    print("\nApplication could not be compiled due to earlier errors.")



用户: 关于 Project Phoenix 的预算，上次会议达成了什么结论？
--- Executing RAG Chain Node ---
Input to rag_chain: input='关于 Project Phoenix 的预算，上次会议达成了什么结论？', history_len=0
RAG Chain Response: {'messages': [], 'input': '关于 Project Phoenix 的预算，上次会议达成了什么结论？', 'context': [Document(id='6ed34f1b-ea65-4319-887a-e3de587bf035', metadata={}, page_content='增加预算的决定是因为初步测试结果非常有希望，表明投入更多资源可以显著加快上市时间。'), Document(id='89f515a5-92d2-4647-9ab7-a32eb72bee39', metadata={}, page_content='会议决定将 Project Phoenix 的预算增加 10% 用于额外的原型设计。')], 'answer': '会议决定将 Project Phoenix 的预算增加 10%，用于额外的原型设计。'}
AI: 会议决定将 Project Phoenix 的预算增加 10%，用于额外的原型设计。

用户: 为什么做出那个决定？
--- Executing RAG Chain Node ---
Input to rag_chain: input='为什么做出那个决定？', history_len=0
RAG Chain Response: {'messages': [], 'input': '为什么做出那个决定？', 'context': [Document(id='89f515a5-92d2-4647-9ab7-a32eb72bee39', metadata={}, page_content='会议决定将 Project Phoenix 的预算增加 10% 用于额外的原型设计。'), Document(id='6ed34f1b-ea65-4319-887a-e3de587bf035', metadata={}, page_content='增加预算的决定是因为初步测试结果非常

- LangGraph 替换 RunnableWithMessageHistory: 这个示例使用 LangGraph StateGraph 来管理对话状态，特别是消息历史 (ConversationState 使用 MessagesState 或自定义 TypedDict)。这是当前推荐的方法，因为它提供了更灵活和强大的状态管理能力 。   
- 状态定义: ConversationState 定义了图在每个时间点需要跟踪的信息。这里简化为只包含 messages 列表。
- 节点定义: 定义了图中的处理单元（节点）。这里简化地展示了两个节点概念，但在实际应用中，通常会将 `create_history_aware_retriever` 和 `create_retrieval_chain` 的逻辑组合在一个或多个节点内执行。关键在于节点函数接收 state 并返回更新后的 state 部分。
- 图构建与编译: 使用 StateGraph 定义节点和它们之间的转换（边）。compile() 方法将图编译成可运行的应用，并通过 checkpointer 参数指定状态持久化的方式（这里使用 MemorySaver 进行内存存储）。   
- 调用: 调用编译后的 app 时，需要提供输入（通常是包含新用户消息的 state 字典）和 config。config 中的 `configurable={"thread_id": "..."}` 用于区分不同的会话，LangGraph 会根据 thread_id 自动加载和保存对应的状态 。
- 状态访问: 可以使用 `app.get_state(config)` 来获取特定会话的当前状态。

#### 实现高效的文本总结

##### 目标
利用 LangChain 和 DeepSeek LLM 对个人文档（如长篇报告、日记、会议记录）或聊天记录进行有效的总结，快速把握核心内容或回顾要点。

##### 总结策略 (load_summarize_chain)
LangChain 提供了 `load_summarize_chain` 函数，这是一个高级接口，用于方便地创建处理总结任务的链。它封装了不同的总结策略，通过 `chain_type` 参数进行选择。

**主要参数:**

* **llm**: 用于执行总结任务的语言模型实例 (例如 `ChatDeepSeek`)。
* **chain_type (str)**: 指定使用的总结策略，主要有 `"stuff"`, `"map_reduce"`, `"refine"`。
* **map_prompt, combine_prompt** (可选, 用于 `map_reduce`): 自定义 Map 和 Reduce 阶段的提示模板。
* **question_prompt, refine_prompt** (可选, 用于 `refine`): 自定义初始总结和后续优化步骤的提示模板。
* **verbose** (bool, 可选): 是否打印链执行的详细日志。
* **return_intermediate_steps** (bool, 可选, 用于 `map_reduce` 和 `refine`): 是否在结果中包含中间步骤的输出（例如 Map 阶段的各个总结）。

###### 不同 chain_type 详解:

**stuff**:
* **机制**: 最简单直接的方法。将所有输入文档（或文本块）连接起来，一次性放入提示中，让 LLM 进行总结。
* **适用场景**: 文档总长度较短，能够完全容纳在 LLM 的单次上下文窗口内；或者使用了具有超大上下文窗口（如 100k+ tokens）的 LLM。
* **优点**: 速度快（只需一次 LLM 调用），实现简单。
* **缺点**: 严格受限于 LLM 的上下文窗口大小。

**map_reduce**:
* **机制**: 采用分治策略。首先，将文档分成多个块（如果尚未切分）。然后，对每个块独立进行总结（Map 步骤）。最后，将所有块的总结收集起来，再让 LLM 对这些总结进行最终的归纳总结（Reduce 步骤）。
* **适用场景**: 处理非常长或数量非常多的文档，其总长度远超 LLM 单次上下文窗口。
* **优点**: 能够处理任意长度和数量的文档，不受单次 LLM 调用上下文窗口的限制；Map 步骤可以并行处理，提高效率。
* **缺点**: 需要多次 LLM 调用（Map 阶段每个块一次，Reduce 阶段至少一次），导致成本和延迟可能更高；最终总结可能丢失部分细节，因为 Reduce 阶段是基于初步总结进行的。

**refine**:
* **机制**: 采用迭代优化策略。按顺序处理文档块。首先对第一个块生成一个初步总结。然后，对于后续的每个块，将当前块的内容和上一轮的总结一起提供给 LLM，让其在前一轮总结的基础上进行优化和补充，生成新的总结。
* **适用场景**: 需要构建更详细、更连贯的总结，能够整合跨块的信息，或者总结任务本身具有迭代性质。
* **优点**: 可以逐步构建更精细的总结，较好地保留上下文信息。
* **缺点**: 处理过程是串行的，速度较慢；需要多次 LLM 调用（每个块一次）；后面的文档块对最终总结的影响可能更大。

##### 策略对比表 (优化版)
为了更清晰地选择合适的总结策略，下表对比了它们的关键特性：

**LangChain 总结策略对比**

| 特性 (Feature)              | stuff                       | map_reduce                                      | refine                                      |
| :-------------------------- | :-------------------------- | :---------------------------------------------- | :------------------------------------------ |
| **工作机制 (Mechanism)** | 单次传递所有内容            | Map: 分块总结<br>Reduce: 合并总结             | 迭代处理块，逐步优化总结                    |
| **优点 (Pros)** | 简单, 快速 (1 次 LLM 调用)   | 可扩展性好 (处理长/多文档), 可并行 Map         | 可构建详细总结, 逐步整合上下文              |
| **缺点 (Cons)** | 受上下文窗口限制            | 多次 LLM 调用 (成本/延迟高), 可能丢失细节       | 串行处理 (慢), 多次 LLM 调用, 后续文档影响可能更大 |
| **最佳使用场景 (Best Use Case)** | 短文档 / 超大上下文窗口 LLM | 非常长或大量的文档                            | 需要高细节或迭代构建上下文的总结任务            |
| **LLM 调用次数 (LLM Calls)** | 1                           | N (Map) + M (Reduce) <br>*(N=块数, M≥1)* | N (N=块数)                                  |

这个表格明确指出了不同策略在处理方式、优缺点、适用场景以及对 LLM 调用次数（直接影响成本和延迟）上的差异，有助于开发者根据具体需求（如文档长度、对细节的要求、预算和时间限制）做出明智的选择。例如，成本敏感且文档不长的项目应首选 `stuff`，而处理海量文档则必须考虑 `map_reduce` 或 `refine`，并在后两者中根据对细节和速度的需求进行权衡。

##### 优化代码实现
以下是使用 `ChatDeepSeek` 和 `load_summarize_chain` 实现不同总结策略的优化代码示例：

长篇报告.txt

```txt
标题：人工智能在现代教育中的应用与深远影响分析报告

摘要：
本报告旨在全面探讨人工智能（AI）技术在现代教育领域的应用现状、潜在优势、面临的挑战以及其对教与学方式产生的深远影响。随着 AI 技术的快速发展和日益成熟，它正逐步渗透到教育的各个环节，从个性化学习、智能辅导到教学管理和效果评估，展现出巨大的变革潜力。然而，与此同时，数据隐私、算法公平性、技术伦理以及对教师角色的冲击等问题也日益凸显，需要教育界、技术开发者及政策制定者共同面对和解决。

引言：
教育是社会发展的基石，而科技进步是推动教育变革的关键力量。进入 21 世纪，以深度学习、自然语言处理、计算机视觉等为代表的人工智能技术取得了突破性进展，开始在各行各业展现其强大的赋能作用。教育领域，作为一个知识密集、高度依赖个性化交互的场景，自然成为了 AI 技术探索应用的重要阵地。从 K12 基础教育到高等教育，再到职业培训和终身学习，AI 的身影无处不在。本报告将系统梳理 AI 在教育中的具体应用场景，分析其带来的机遇与挑战，并展望未来的发展趋势。

第一部分：AI 在教育中的主要应用场景

1.1 个性化学习路径推荐：
AI 最具潜力的应用之一是实现真正意义上的个性化学习。通过分析学生的学习习惯、知识掌握程度、兴趣偏好以及认知特点，AI 系统能够为每个学生量身定制学习内容、进度和路径。例如，智能学习平台可以根据学生在练习题上的表现，动态调整后续内容的难度和类型，针对薄弱环节进行强化训练，从而避免“一刀切”的教学模式，显著提升学习效率和效果。自适应学习系统（Adaptive Learning Systems）是这一应用的典型代表，它们能够实时追踪学生进度，并提供即时反馈。

1.2 智能辅导与答疑系统：
AI 驱动的聊天机器人和虚拟助教能够 7x24 小时为学生提供答疑服务，解决他们在课后遇到的学习问题。这些系统不仅能回答事实性问题，还能通过自然语言处理技术理解学生的提问意图，进行一定程度的启发式引导。对于重复性高、基础性的问题，AI 助教可以有效分担教师的负担，让教师能更专注于深层次的教学互动和对学生的情感关怀。此外，AI 也可以在写作、编程等技能学习中提供实时反馈和批改建议。

1.3 智能教学管理与评估：
AI 技术能够协助教师和学校管理者进行更高效的教学管理。例如，利用 AI 进行课堂行为分析（需关注隐私伦理），可以帮助教师了解学生的参与度和注意力集中情况。在考试评估方面，AI 可以自动批改客观题，甚至对主观题（如作文、简答题）进行辅助评分，提供初步的评价意见，大大减轻教师的阅卷压力。同时，AI 对学习过程数据的分析也能为教学效果评估提供更全面、客观的依据，帮助优化教学策略。

1.4 教育资源智能推荐与生成：
AI 可以根据教学大纲、学生需求和最新的知识发展，自动筛选、整合、甚至生成相关的教学资源，如课件、习题、阅读材料等。这不仅丰富了教学资源库，也提高了资源的时效性和匹配度。一些先进的 AI 模型已经开始尝试生成具有一定创造性的教学内容，为教学创新提供了新的可能。

第二部分：AI 教育应用的优势与机遇

2.1 提升学习效率与效果：
个性化学习路径和即时反馈机制，使得学生可以按照最适合自己的节奏和方式学习，有效缩短了掌握知识所需的时间，提高了学习的深度和广度。

2.2 促进教育公平：
优质的 AI 教育资源和辅导系统可以突破地域和时间的限制，让偏远地区或资源匮乏的学生也能接触到高质量的教育内容和辅导服务，一定程度上弥合教育鸿沟。

2.3 解放教师生产力：
AI 承担了部分重复性、事务性的工作（如批改作业、基础答疑），使教师能够将更多精力投入到课程设计、课堂互动、学生个性化指导和情感交流等更具创造性和人文关怀的教学环节。

2.4 数据驱动的教学决策：
AI 对学习过程数据的深度分析，为教师和教育管理者提供了精细化的洞察，有助于更科学地调整教学方法、优化课程设置，实现基于证据的教育决策。

第三部分：挑战与伦理考量

3.1 数据隐私与安全：
AI 教育应用需要收集和分析大量的学生数据，如何确保这些敏感数据的安全存储和合规使用，防止泄露和滥用，是亟待解决的关键问题。学生的个人隐私必须得到最高级别的保护。

3.2 算法偏见与公平性：
AI 系统的决策基于其训练数据和算法设计。如果训练数据本身存在偏见（如性别、种族、社会经济背景等），或者算法设计不合理，可能导致 AI 在推荐学习资源、评估学生表现时产生歧视，加剧而非缓解教育不公。

3.3 技术依赖与数字鸿沟：
过度依赖 AI 可能削弱学生自主学习、批判性思维和人际交往能力。同时，不同地区、不同家庭在接触和使用 AI 教育工具方面可能存在差距，形成新的数字鸿沟。

3.4 教师角色的转变与适应：
AI 的广泛应用要求教师从传统的知识传授者转变为学习引导者、课程设计者和学生成长伙伴。这对教师的数字素养、教学理念和专业能力提出了新的要求，需要相应的培训和支持体系。

3.5 技术伦理与人本关怀：
教育的核心是人的发展。在追求技术效率的同时，必须警惕技术对人的异化。如何确保 AI 的应用始终服务于学生的全面发展和福祉，而不是仅仅追求分数和效率，是重要的伦理议题。教育过程中的情感交流、价值塑造和师生关系，是 AI 难以完全替代的。

第四部分：未来展望与建议

展望未来，AI 与教育的融合将更加深入。混合式学习（Blended Learning）将成为常态，AI 作为智能学伴和教学助手，与人类教师协同工作。情感计算、脑机接口等前沿技术可能进一步增强 AI 教育应用的体验和效果。虚拟现实（VR）/增强现实（AR）与 AI 的结合将创造出沉浸式的、高度互动的学习环境。

为应对挑战并抓住机遇，我们提出以下建议：
1. 建立健全法律法规和伦理规范，明确数据使用边界，保障学生隐私和算法公平。
2. 加大教师培训力度，提升教师的 AI 素养和应用能力，促进教师角色成功转型。
3. 鼓励跨学科研究，探索 AI 在教育中更深层次、更负责任的应用模式。
4. 关注并努力消除数字鸿沟，确保所有学生都能公平地受益于 AI 教育技术。
5. 坚持以人为本的原则，强调 AI 的辅助作用，始终将学生的全面发展放在首位，重视批判性思维、创造力及社会情感能力的培养。

结论：
人工智能为现代教育带来了前所未有的机遇，有望推动教育向更个性化、高效化、公平化的方向发展。然而，伴随而来的技术、伦理和社会挑战亦不容忽视。只有在审慎规划、严格监管、持续探索和人文关怀的指引下，AI 才能真正成为赋能教育、促进人类全面发展的有益工具。教育的未来，将是人机协同、智慧共生的新篇章。
```

环境设置

In [84]:
from langchain.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from langchain_deepseek import ChatDeepSeek
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader # 示例加载器
from langchain import hub
import os

# 假设 DEEPSEEK_API_KEY 环境变量已设置
# --- 文档加载与分割 ---
try:
    # 确保文件存在且路径正确
    loader = TextLoader("data/长篇报告.txt", encoding='utf-8')
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=4000, # 根据 LLM 上下文和提示复杂度调整
        chunk_overlap=200
    )
    all_split_docs = text_splitter.split_documents(documents)
    print(f"文档已加载并分割成 {len(all_split_docs)} 个块。")
except Exception as e:
    print(f"加载/分割文档时出错: {e}")
    all_split_docs = None # 阻止后续错误

# --- 初始化 LLM ---
try:
    llm = ChatDeepSeek(model="deepseek-chat", temperature=0)
    print("ChatDeepSeek 模型已初始化。")
except Exception as e:
    print(f"初始化 LLM 时出错: {e}")
    # 优雅地处理 LLM 初始化失败
    exit()

文档已加载并分割成 2 个块。
ChatDeepSeek 模型已初始化。


- Map-Reduce 示例

In [85]:
print(f"\n准备对 {len(all_split_docs)} 个文档块进行 Map-Reduce 摘要...")
# Map Prompt - 注意 input_variables 和模板中的变量名 'text'
map_prompt_template = """以下是一段需要总结的文本：
    "{text}"
    请根据这段文本写一个简洁的摘要。
    简洁摘要："""
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

# Combine Prompt - 注意 input_variables 和模板中的变量名 'docs'
combine_prompt_template = """以下是一些摘要：
    {docs}
    请将这些摘要整合提炼成一个最终的、统一的摘要，涵盖所有主要主题。
    最终摘要："""
combine_prompt = PromptTemplate(template=map_prompt_template, input_variables=["docs"])

# 通过 kwargs 传递 prompts
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True  # 建议开启 verbose 进行调试
)
print("Map-Reduce 链初始化成功。")

# 运行链
# 对较新版本的 LangChain 使用 invoke 方法
summary_map_reduce_output = map_reduce_chain.invoke(
    {"input_documents": all_split_docs},
    return_only_outputs=True
)

print("\n--- Map-Reduce 摘要结果 ---")
# 安全地获取输出文本
print(summary_map_reduce_output.get('output_text', '未能提取摘要文本。'))


准备对 2 个文档块进行 Map-Reduce 摘要...
Map-Reduce 链初始化成功。


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段需要总结的文本：
    "标题：人工智能在现代教育中的应用与深远影响分析报告

摘要：
本报告旨在全面探讨人工智能（AI）技术在现代教育领域的应用现状、潜在优势、面临的挑战以及其对教与学方式产生的深远影响。随着 AI 技术的快速发展和日益成熟，它正逐步渗透到教育的各个环节，从个性化学习、智能辅导到教学管理和效果评估，展现出巨大的变革潜力。然而，与此同时，数据隐私、算法公平性、技术伦理以及对教师角色的冲击等问题也日益凸显，需要教育界、技术开发者及政策制定者共同面对和解决。

引言：
教育是社会发展的基石，而科技进步是推动教育变革的关键力量。进入 21 世纪，以深度学习、自然语言处理、计算机视觉等为代表的人工智能技术取得了突破性进展，开始在各行各业展现其强大的赋能作用。教育领域，作为一个知识密集、高度依赖个性化交互的场景，自然成为了 AI 技术探索应用的重要阵地。从 K12 基础教育到高等教育，再到职业培训和终身学习，AI 的身影无处不在。本报告将系统梳理 AI 在教育中的具体应用场景，分析其带来的机遇与挑战，并展望未来的发展趋势。

第一部分：AI 在教育中的主要应用场景

1.1 个性化学习路径推荐：
AI 最具潜力的应用之一是实现真正意义上的个性化学习。通过分析学生的学习习惯、知识掌握程度、兴趣偏好以及认知特点，AI 系统能够为每个学生量身定制学习内容、进度和路径。例如，智能学习平台可以根据学生在练习题上的表现，动态调整后续内容的难度和类型，针对薄弱环节进行强化训练，从而避免“一刀切”的教学模式，显著提升学习效率和效果。自适应学习系统（Adaptive Learning Systems）是这一应用的典型代表，它们能够实时追踪学生进度，并提供即时反馈。

1.2 智能辅导与答疑系统：
AI 驱动的聊天机器人和虚拟助教能够 7x24 小时为学生提供答疑服务，解决他们在课后遇到的学习问题。这些系统不仅能回答事实性问题，还能通过自然语言处理技术理

D:\Users\iSPACE\anaconda3\envs\rag\Lib\site-packages\langchain_openai\chat_models\base.py:422: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  class _AllReturnType(TypedDict):



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段需要总结的文本：
    "**人工智能在现代教育中的应用与影响**  

人工智能（AI）正深刻变革教育领域，通过个性化学习推荐、智能辅导、教学管理和资源生成等应用提升效率与公平性，同时解放教师生产力。然而，数据隐私、算法偏见、技术依赖和数字鸿沟等挑战需多方协作解决，以确保AI在教育中的伦理与可持续发展。

人工智能（AI）在教育领域的应用既带来机遇也面临挑战。机遇包括个性化学习、混合式教学常态化和沉浸式技术（如VR/AR）的增强；挑战涉及技术依赖削弱学生能力、数字鸿沟加剧、教师角色转型压力及技术伦理问题。建议采取以下措施：完善法规保障隐私与公平、加强教师AI培训、推动跨学科研究、缩小数字鸿沟、坚持人本原则。未来教育需实现人机协同，在审慎监管与人文关怀下，使AI真正服务于学生的全面发展。"
    请根据这段文本写一个简洁的摘要。
    简洁摘要：

> Finished chain.

> Finished chain.

--- Map-Reduce 摘要结果 ---
人工智能（AI）正在变革教育领域，通过个性化学习、智能辅导等技术提升效率与公平性，同时面临数据隐私、算法偏见和数字鸿沟等挑战。未来需完善法规、加强教师培训、缩小技术差距，并坚持人机协同与伦理监管，以实现AI在教育中的可持续发展。


- Refine 示例 

In [86]:
# --- Refine 示例 ---
print(f"\n准备使用 Refine 方式总结 {len(all_split_docs)} 个文档块...")
# 自定义或加载 Refine 提示
question_prompt_template = """请为以下文本写一个简洁的总结：
"{text}"
简洁总结："""
question_prompt = PromptTemplate.from_template(question_prompt_template)

refine_prompt_template = (
    "你的任务是产出最终的总结。\n"
    "我们提供了一个截至目前的现有总结：{existing_answer}\n"
    "我们有机会使用下面的更多上下文来优化现有的总结（仅在需要时）。\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "考虑到新的上下文，优化原始总结。\n"
    "如果上下文没有用，则返回原始总结。"
)
refine_prompt = PromptTemplate.from_template(refine_prompt_template)

refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True, # 可以看到中间步骤
    input_key="input_documents",
    output_key="output_text",
    verbose=True
)

summary_refine = refine_chain.invoke({"input_documents": all_split_docs}, return_only_outputs=True)

print("\n--- Refine 总结结果 ---")
print(summary_refine.get('output_text', '未能提取总结文本。'))
# print("\n--- Refine 中间步骤 ---")
# for step in summary_refine.get('intermediate_steps',):
#     print(step)


准备使用 Refine 方式总结 2 个文档块...


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
请为以下文本写一个简洁的总结：
"标题：人工智能在现代教育中的应用与深远影响分析报告

摘要：
本报告旨在全面探讨人工智能（AI）技术在现代教育领域的应用现状、潜在优势、面临的挑战以及其对教与学方式产生的深远影响。随着 AI 技术的快速发展和日益成熟，它正逐步渗透到教育的各个环节，从个性化学习、智能辅导到教学管理和效果评估，展现出巨大的变革潜力。然而，与此同时，数据隐私、算法公平性、技术伦理以及对教师角色的冲击等问题也日益凸显，需要教育界、技术开发者及政策制定者共同面对和解决。

引言：
教育是社会发展的基石，而科技进步是推动教育变革的关键力量。进入 21 世纪，以深度学习、自然语言处理、计算机视觉等为代表的人工智能技术取得了突破性进展，开始在各行各业展现其强大的赋能作用。教育领域，作为一个知识密集、高度依赖个性化交互的场景，自然成为了 AI 技术探索应用的重要阵地。从 K12 基础教育到高等教育，再到职业培训和终身学习，AI 的身影无处不在。本报告将系统梳理 AI 在教育中的具体应用场景，分析其带来的机遇与挑战，并展望未来的发展趋势。

第一部分：AI 在教育中的主要应用场景

1.1 个性化学习路径推荐：
AI 最具潜力的应用之一是实现真正意义上的个性化学习。通过分析学生的学习习惯、知识掌握程度、兴趣偏好以及认知特点，AI 系统能够为每个学生量身定制学习内容、进度和路径。例如，智能学习平台可以根据学生在练习题上的表现，动态调整后续内容的难度和类型，针对薄弱环节进行强化训练，从而避免“一刀切”的教学模式，显著提升学习效率和效果。自适应学习系统（Adaptive Learning Systems）是这一应用的典型代表，它们能够实时追踪学生进度，并提供即时反馈。

1.2 智能辅导与答疑系统：
AI 驱动的聊天机器人和虚拟助教能够 7x24 小时为学生提供答疑服务，解决他们在课后遇到的学习问题。这些系统不仅能回答事实性问题，还能通过自然语言处理技术理解学生的提问意图，进行一定程度的启发式引导。对于重复性

- Stuff 示例 (适用于短文档)

In [88]:
# --- Stuff 示例 (适用于短文档) ---
print("\n准备使用 Stuff 方式总结...")
# 确保 short_docs 的总长度在模型上下文限制内
stuff_chain = load_summarize_chain(llm, chain_type="stuff", verbose=True)

if hasattr(stuff_chain, 'invoke'):
    summary_stuff = stuff_chain.invoke({"input_documents": all_split_docs}, return_only_outputs=True)
else:
    summary_stuff = stuff_chain.run(short_docs)

print("\n--- Stuff 总结结果 ---")
if isinstance(summary_stuff, dict):
    print(summary_stuff.get('output_text', '未能提取总结文本。'))
else:
    print(summary_stuff)


准备使用 Stuff 方式总结...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"标题：人工智能在现代教育中的应用与深远影响分析报告

摘要：
本报告旨在全面探讨人工智能（AI）技术在现代教育领域的应用现状、潜在优势、面临的挑战以及其对教与学方式产生的深远影响。随着 AI 技术的快速发展和日益成熟，它正逐步渗透到教育的各个环节，从个性化学习、智能辅导到教学管理和效果评估，展现出巨大的变革潜力。然而，与此同时，数据隐私、算法公平性、技术伦理以及对教师角色的冲击等问题也日益凸显，需要教育界、技术开发者及政策制定者共同面对和解决。

引言：
教育是社会发展的基石，而科技进步是推动教育变革的关键力量。进入 21 世纪，以深度学习、自然语言处理、计算机视觉等为代表的人工智能技术取得了突破性进展，开始在各行各业展现其强大的赋能作用。教育领域，作为一个知识密集、高度依赖个性化交互的场景，自然成为了 AI 技术探索应用的重要阵地。从 K12 基础教育到高等教育，再到职业培训和终身学习，AI 的身影无处不在。本报告将系统梳理 AI 在教育中的具体应用场景，分析其带来的机遇与挑战，并展望未来的发展趋势。

第一部分：AI 在教育中的主要应用场景

1.1 个性化学习路径推荐：
AI 最具潜力的应用之一是实现真正意义上的个性化学习。通过分析学生的学习习惯、知识掌握程度、兴趣偏好以及认知特点，AI 系统能够为每个学生量身定制学习内容、进度和路径。例如，智能学习平台可以根据学生在练习题上的表现，动态调整后续内容的难度和类型，针对薄弱环节进行强化训练，从而避免“一刀切”的教学模式，显著提升学习效率和效果。自适应学习系统（Adaptive Learning Systems）是这一应用的典型代表，它们能够实时追踪学生进度，并提供即时反馈。

1.2 智能辅导与答疑系统：
AI 驱动的聊天机器人和虚拟助教能够 7x24 小时为学生提供答疑服务，解决他们在课后遇到的学习问题。这些系统不仅能回答事实性问题，还能通过自然语言处理技术理解学生的提问意图

## 基于 LangGraph 实现具有长期记忆能力
### 引言：高级 AI 代理中记忆的必要性
#### “健忘代理”问题
想象一下，与一个个人助理互动，但每次交流它都会忘记用户的偏好、过去的对话以及之前的指令。这样的助理显然效率不高 。这正是 AI 代理中长期记忆旨在解决的核心挑战。传统的无状态代理（stateless agents）在每次交互后会丢失所有上下文，导致对话重复、缺乏个性化，无法真正理解和适应用户的需求。为了构建更智能、更有用、更能与用户建立持久关系的 AI 代理，赋予它们记忆能力至关重要。   

#### LangGraph 简介
LangGraph 是 LangChain 推出的一个强大的 Python/JavaScript 库，专门用于构建有状态的、多参与者的应用程序，特别是复杂的代理式工作流（agentic workflows）。它通过节点（Nodes）、边（Edges）和共享状态（State）的概念，允许开发者精确地定义和控制信息如何在工作流的不同步骤之间传递和处理。这种架构使其特别适合管理对话状态和集成各种形式的记忆机制，从而构建能够进行连贯、有上下文感知对话的代理。   

#### 长期记忆：实现持久化的关键
在 AI 代理的背景下，长期记忆（Long-Term Memory, LTM）指的是跨越不同对话会话（sessions）或线程（threads）存储和回忆信息的能力 。这使得代理能够记住用户的特定事实、偏好、过去的互动结论等，从而提供个性化的响应并保持交互的连续性。这与短期记忆（Short-Term Memory, STM）形成对比，后者通常只关注当前单个对话会话中的信息 。长期记忆是代理实现学习、适应和与用户建立更深层次理解的基础。   

### 核心概念：LangGraph 基础与代理记忆
#### LangGraph 要素：构建有状态工作流的基石
理解 LangGraph 的核心组件对于构建记忆代理至关重要：

- **状态 (State)**: 这是 LangGraph 工作流的核心。它通常是一个 Python TypedDict，定义了在图的不同节点之间流动的所有信息的结构 。对于对话代理，通常会使用 MessagesState 作为基类，它能自动管理和累积对话消息历史 。状态对象在整个图的执行过程中被传递和更新。   
- **节点 (Nodes)**: 节点是执行具体操作的工作单元。它可以是一个 Python 函数或任何可运行的对象（Runnable）。节点接收当前状态，执行其任务（例如调用 LLM、使用工具、处理数据），并返回对状态的更新 。   
- **边 (Edges)**: 边定义了工作流中节点之间的控制流向 。边可以将一个节点的输出连接到下一个节点。LangGraph 还支持条件边（Conditional Edges），允许根据当前状态的值动态地决定下一个要执行的节点，从而实现复杂的逻辑分支。图中还有特殊的 START 和 END 节点，分别表示工作流的开始和结束 。   
- **图 (Graphs - StateGraph)**: StateGraph 类是用于构建这些有状态工作流的主要接口 。开发者通过向 StateGraph 实例添加节点和边来定义代理的行为逻辑。   
- **检查点 (Checkpointers)**: 检查点是 LangGraph 中用于持久化**单个对话线程状态**的机制，它实现了短期记忆 。通过配置检查点（如 MemorySaver 用于内存存储，SqliteSaver 用于 SQLite 数据库，RedisSaver 用于 Redis ），可以在每个步骤后保存图的状态快照。这使得对话可以在中断后恢复，并保留了特定会话的完整历史记录。需要强调的是，检查点通常是线程范围（thread-scoped）的，即它保存的是特定 thread_id 的状态 。

#### 解密代理记忆：短期 vs. 长期
在构建智能代理时，区分不同类型的记忆至关重要：

- **短期记忆 (Short-Term Memory - 工作/上下文记忆)**: 指在**单个、持续进行的对话线程**中可用的信息 。这主要由检查点机制通过保存 MessagesState（包含对话历史）来实现。短期记忆对于维持当前对话的上下文、理解指代关系和进行连贯的多轮交流至关重要。然而，短期记忆（尤其是基于 LLM 上下文窗口的记忆）通常有长度限制。当对话历史过长时，可能会超出 LLM 的处理能力或导致成本增加。管理这种限制的一种常见技术是进行对话摘要（Summarization），即在对话进行到一定长度后，将早期的消息压缩成摘要形式 。   
- **长期记忆 (Long-Term Memory - 持久/跨线程记忆)**: 指存储在特定线程状态之外的信息，旨在**跨越多个、可能在时间上分离的对话会话**进行访问 。长期记忆使得代理能够“记住”关于用户的事实（如姓名、偏好）、重要的历史信息或先前得出的结论，即使在开启一个全新的对话会话时也能调用这些信息。LangGraph 通过引入“存储”（Stores）和自定义“命名空间”（Namespaces）的概念来支持长期记忆的实现 。命名空间通常包含用户 ID 或其他标识符，用于组织和隔离不同用户或上下文的记忆 。   
- **记忆的相互作用**:
    - 短期记忆（对话历史）为代理提供了决定何时以及何种信息需要存入或从长期记忆中检索的即时上下文。
    - 当长期记忆被检索出来后，它会被加载到短期记忆的上下文中（例如，作为系统提示的一部分，或填充到状态的特定字段中），从而影响代理在当前回合的响应生成 。明确展示了在调用代理节点之前加载记忆的模式，而则显示了检索到的记忆被插入到系统提示中。   
    - 这意味着设计有效的记忆系统需要同时考虑这两种记忆类型以及它们如何协同工作。检查点负责处理“当下”的对话状态，而长期记忆存储则负责处理“过去”的持久信息。
#### 长期记忆的类型（概念性）

- 情景记忆 (Episodic Memory): 存储与特定时间和地点相关的个人经历和事件，例如用户过去的具体交互或偏好表达 。   
- 语义记忆 (Semantic Memory): 存储关于世界的一般事实性知识，例如关于旅行目的地的信息或特定领域的知识 。在代理上下文中，这也可以包括关于用户的稳定事实（如姓名、住址）。提到了将信息存储为文本或结构化的知识三元组（主语、谓语、宾语）。   
- 程序记忆 (Procedural Memory): 存储关于如何执行任务或遵循特定流程的知识，例如代理学习到的特定交互策略或系统提示的优化 。   
后面实现的向量存储机制可以灵活地存储代表语义记忆（用户事实）和情景记忆（用户过去的陈述或偏好）的文本信息。

### 环境设置：准备您的工作空间
在开始构建具有长期记忆的 LangGraph 代理之前，需要确保开发环境已正确配置。

In [230]:
# !pip install langgraph langchain-openai langchain-community pydantic uuid tiktoken

In [90]:
# 导入所需模块
import getpass
from langchain_deepseek import ChatDeepSeek

os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("输入您的 DeepSeek API 密钥：")
DEEPSEEK_MODEL_NAME = "deepseek-chat"

# 初始化 DeepSeek 模型
llm = ChatDeepSeek(
    model=DEEPSEEK_MODEL_NAME,
    temperature=0
)
llm.invoke("Hello!")
print(f"Successfully connected to DeepSeek model: {DEEPSEEK_MODEL_NAME}")

输入您的 DeepSeek API 密钥： ········


Successfully connected to DeepSeek model: deepseek-chat


### 构建长期记忆代理：分步实施
现在，我们将逐步构建一个能够跨会话记住用户信息的 LangGraph 代理。

#### 定义具有记忆感知的代理状态
状态定义了在 LangGraph 工作流中流动的数据结构。为了让代理能够感知和利用长期记忆，我们需要在状态中包含相关字段。

- 我们将基于 MessagesState 来自动处理对话历史 。   
- 我们将添加一个自定义字段 recall_memories，用于存储在当前回合从长期记忆中检索到的相关信息。这个字段是将 LTM 引入 STM 上下文的桥梁 。   
- 使用 typing.TypedDict 和 typing.Annotated 来清晰地定义状态结构及其更新方式 。Annotated 结合 operator.add 可以指定消息列表应如何聚合（追加）。   

**代码片段：定义 State**

In [210]:
import operator
from typing import TypedDict, Annotated, List
from langchain_core.messages import AnyMessage

class AgentState(TypedDict):
    # messages 字段将存储对话历史
    # Annotated[List[AnyMessage], operator.add] 表示新消息会被追加到列表中
    messages: Annotated[List[AnyMessage], operator.add]

    # recall_memories 字段用于存储从 LTM 检索到的、与当前对话相关的记忆
    # 这个字段会在每个回合开始时由 load_memories 节点填充
    recall_memories: List[str]

#### 选择并实现用于记忆持久化的向量存储

为了实现长期记忆的存储和检索，我们需要一个持久化的存储机制。向量存储因其支持语义搜索而成为一个理想的选择，它能根据意义的相似性而非仅仅是关键词来查找相关记忆 。   

**代码片段：设置嵌入和向量存储**

In [211]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_core.vectorstores import VectorStore

# 初始化嵌入模型
# 建议使用较新的模型，如 mxbai-embed-large 或 nomic-embed-text
# 请确保模型名称与您的 OpenAI 账户权限匹配
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

# 初始化内存向量存储
# 注意：这是一个内存实现，关闭程序后数据会丢失
# 对于持久化存储，请替换为其他向量存储实现，如 Redis, FAISS, Chroma 等
# 这里我们不显式配置 index，InMemoryVectorStore 会在添加数据时处理
vector_store: VectorStore = InMemoryVectorStore(embedding=embedding_model)

print("向量存储和嵌入模型已初始化 (内存模式)")

向量存储和嵌入模型已初始化 (内存模式)


#### 精心设计用于记忆存储和检索的工具
我们将采用“工具调用”（Tool Calling）或“函数调用”（Function Calling）的方法来管理长期记忆 。这意味着 LLM 将根据对话上下文自主决定何时需要存储信息或检索过去的记忆，并将这些操作作为工具来调用。这种方法将记忆管理融入了代理的推理过程 。   

- `@tool` 装饰器: 使用 `langchain_core.tools` 中的 @tool 装饰器可以方便地将 Python 函数定义为可供 LLM 调用的工具 。   
- `upsert_memory` 工具:
    - 功能: 接收一段文本 (memory_content)，将其作为记忆存入长期存储。
    - 实现:
        - 接收要存储的内容。
        - 关键: 需要知道这条记忆属于哪个用户。我们将设计工作流，使得调用此工具的节点能够从 config 中获取 `user_id`，并将其作为元数据（metadata）与记忆内容一起存储。
        - 使用向量存储的 `add_texts` 方法，传入记忆内容和包含 `user_id` 的元数据。向量存储会自动处理嵌入和索引。
        - 返回一个确认信息给 LLM/用户，告知记忆已保存 。   
- `search_memory` 工具:
    - 功能: 接收一个查询 (search_query)，在长期记忆中搜索相关信息。
    - 实现:
        - 接收搜索查询。
        - 关键: 同样需要知道当前用户的 `user_id`，以便只在该用户的记忆空间中搜索。
        - 使用向量存储的 `similarity_search` 方法 。在 `InMemoryVectorStore` 或其他支持元数据过滤的存储中，可以在搜索时指定过滤条件，例如 `metadata={'user_id': current_user_id}`。   
        - 格式化检索到的相关记忆（例如，将文档内容连接成一个字符串），并返回给 LLM。
- 处理 `user_id`:
    - 如上所述，工具函数本身通常不直接访问 LangGraph 的 config。
    - 解决方案是在调用这些工具的 LangGraph 节点（例如，稍后定义的 `tool_node` 或执行工具调用的自定义逻辑）中，从传入的 config 对象里提取 `user_id` 。   
    - 然后，这个节点在调用实际的 `upsert_memory` 或 `search_memory` 函数时，将 `user_id` 作为参数传递给它们，或者传递给向量存储的过滤参数。本教程将采用这种节点层面的处理方式。
代码片段：定义记忆工具

In [212]:
import uuid
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# 用于 upsert_memory 工具的输入模型
class UpsertMemoryInput(BaseModel):
    memory_content: str = Field(description="需要存储的记忆内容")

# 用于 search_memory 工具的输入模型
class SearchMemoryInput(BaseModel):
    search_query: str = Field(description="用于在记忆中搜索相关信息的查询语句")

# 全局 user_id 变量，将在节点中设置
# 注意：这只是为了简化示例，实际应用中应通过 config 传递
# current_tool_user_id = None

# 实际的记忆存储函数，接收 user_id
def _upsert_memory(memory_content: str, user_id: str):
    """将记忆内容和 user_id 存储到向量数据库"""
    if not user_id:
        return "错误：无法确定用户ID，无法存储记忆。"
    # 使用 add_texts 存储，并附加 user_id 作为元数据
    # 使用 uuid 生成唯一的文档 ID
    doc_id = str(uuid.uuid4())
    vector_store.add_texts(
        texts=[memory_content],
        metadatas=[{"user_id": user_id, "doc_id": doc_id}],
        ids=[doc_id] # 指定文档 ID
    )
    print(f"为用户 {user_id} 存储记忆: {memory_content}")
    return f"好的，我已经记住了：'{memory_content}'"

# 实际的记忆搜索函数，接收 user_id
def _search_memory(search_query: str, user_id: str, k: int = 3):
    """根据查询和 user_id 在向量数据库中搜索记忆"""
    if not user_id:
        return "错误：无法确定用户ID，无法搜索记忆。"
    # 使用 similarity_search 进行搜索，并通过 filter 参数指定 user_id
    # 注意：InMemoryVectorStore 可能不直接支持 filter 参数，
    # 我们需要手动过滤或使用支持过滤的 VectorStore (如 Chroma, Redis 等)
    # 这里我们先进行无过滤搜索，然后手动筛选
    # 对于生产环境，强烈建议使用原生支持元数据过滤的向量存储
    try:
        # 尝试使用 filter 参数 (如果后端支持)
        # results = vector_store.similarity_search(search_query, k=k*2, filter={"user_id": user_id})

        # 如果 InMemoryVectorStore 不支持 filter，则获取更多结果后手动过滤
        all_results = vector_store.similarity_search_with_score(search_query, k=k*5) # 获取更多结果
        filtered_results = [
            doc for doc, score in all_results if doc.metadata.get("user_id") == user_id
        ]
        results = filtered_results[:k] # 取前 k 个匹配用户的结果

    except Exception as e:
        print(f"搜索时发生错误 (可能 filter 不支持): {e}")
        # 备用方案：获取所有结果后手动过滤
        all_results = vector_store.similarity_search_with_score(search_query, k=k*5)
        filtered_results = [
            doc for doc, score in all_results if doc.metadata.get("user_id") == user_id
        ]
        results = filtered_results[:k]

    if not results:
        return "在您的记忆中没有找到相关信息。"

    # 格式化结果
    formatted_memories = "\n".join([f"- {doc.page_content}" for doc in results])
    print(f"为用户 {user_id} 找到相关记忆:\n{formatted_memories}")
    return f"在您的记忆中找到以下相关信息：\n{formatted_memories}"


# 定义 upsert_memory 工具
@tool("upsert_memory", args_schema=UpsertMemoryInput)
def upsert_memory_tool(memory_content: str) -> str:
    """
    存储一段关于用户的记忆。当用户明确要求记住某事，或者对话中出现重要的用户信息（如姓名、偏好、地址等）时使用。
    """
    # 注意：这个工具函数本身不执行存储操作。
    # 它仅用于被 LLM 调用，实际的存储逻辑（包括 user_id 处理）
    # 将在 execute_tools_node 中完成。
    # 这里返回的内容会被忽略，因为 execute_tools_node 会调用 _upsert_memory。
    # 为了清晰起见，可以返回一个指示性的字符串，但它不会直接显示给用户。
    return f"意图：存储记忆 '{memory_content}'"


# 定义 search_memory 工具
@tool("search_memory", args_schema=SearchMemoryInput)
def search_memory_tool(search_query: str) -> str:
    """
    在用户的长期记忆中搜索与查询相关的信息。当用户询问关于他们自己的信息，或者需要基于过去的偏好/事实来回答问题时使用。
    """
    # 注意：这个工具函数本身不执行搜索操作。
    # 实际的搜索逻辑（包括 user_id 处理）将在 execute_tools_node 中完成。
    # 返回值同样会被忽略。
    return f"意图：搜索记忆，查询 '{search_query}'"

# 将工具放入列表
memory_tools = [upsert_memory_tool, search_memory_tool]

注意: 上述代码中，工具函数本身被修改为不直接执行记忆操作，而是返回一个包含操作意图（upsert 或 search）和相关数据（内容或查询）的字典。实际的 `_upsert_memory` 和 `_search_memory` 函数调用（包括 user_id 的注入）将在稍后定义的自定义工具执行器（ToolExecutor）中完成。这是一种更健壮的处理方式，将工具定义与执行逻辑解耦。

#### 设计代理的工作流：关键节点
现在我们定义构成 LangGraph 工作流的核心节点。

##### `load_memories` 节点 (关键补充)
- 目的: 在主 LLM 代理进行推理之前，从长期记忆中加载与当前对话相关的上下文信息。这确保了代理能够“记起”与当前用户和话题相关的重要过往信息 。 明确建议在调用主代理节点之前添加一个加载记忆的节点。   
- 功能:
  - 接收当前 `AgentState` 和 `config` 对象。
    - 从 `config` 中提取 `user_id` 。   
    - 如果 `state['messages']` 不为空，则使用最新的用户消息内容作为查询，调用 `_search_memory` 函数（传入 `user_id`）在向量存储中进行相似性搜索。
    - 将检索到的记忆（格式化为字符串列表）更新到 `state['recall_memories']` 字段中。
代码片段:

In [213]:
from langchain_core.runnables import RunnableConfig

def load_memories_node(state: AgentState, config: RunnableConfig):
    """
    在代理运行前加载相关记忆。
    从 config 中获取 user_id，根据最新消息搜索该用户的记忆，
    并将结果存入 state['recall_memories']。
    """
    user_id = config["configurable"].get("user_id")
    if not user_id:
        print("警告：load_memories_node 未找到 user_id")
        return {"recall_memories":[]}

    messages = state['messages']
    if not messages:
        # 如果没有消息历史，则不加载记忆
        return {"recall_memories":[]}

    # 使用最新一条消息（通常是用户输入）作为搜索查询
    # 确保消息内容是字符串
    last_message_content = ""
    if isinstance(messages[-1].content, str):
        last_message_content = messages[-1].content
    elif isinstance(messages[-1].content, list): # 处理 content 是列表的情况 (例如包含图片)
         # 查找文本部分
         for item in messages[-1].content:
             if isinstance(item, dict) and item.get("type") == "text":
                 last_message_content = item.get("text", "")
                 break
         if not last_message_content:
             print("警告: 最新消息内容格式复杂，无法提取文本进行记忆搜索。")
             return {"recall_memories":[]}
    else:
         print(f"警告: 未知消息内容类型: {type(messages[-1].content)}")
         return {"recall_memories":[]}


    print(f"为用户 {user_id} 加载记忆，基于查询: '{last_message_content}'")

    # 调用实际的搜索函数
    retrieved_memory_str = _search_memory(last_message_content, user_id)

    # 将检索结果（如果不是错误消息）放入 state
    # 假设 _search_memory 返回一个包含记忆的字符串或一条提示信息
    recall_memories_list = []
    if "错误：" not in retrieved_memory_str and "没有找到" not in retrieved_memory_str:
         # 简单地将整个返回字符串作为一个记忆项
         recall_memories_list = [retrieved_memory_str]
         print(f"加载到状态的记忆: {recall_memories_list}")
    else:
         print("未找到相关记忆或发生错误。")


    return {"recall_memories": recall_memories_list}

##### agent 节点 (核心逻辑)

功能: 与 LLM 交互的核心节点。
- 准备系统提示（System Prompt），包含代理的角色、指令，并明确告知其可以使用的记忆工具 (upsert_memory, search_memory) 。   
- 将从 `state['recall_memories']` 中加载的相关记忆整合到传递给 LLM 的上下文中（例如，放在系统提示之后，用户问题之前）。
- 包含 `state['messages']` 中的当前对话历史。
- 调用 LLM（例如 ChatOpenAI），并将绑定了记忆工具的模型传递给它，使其能够进行工具调用。
- 接收 LLM 的响应，这可能是一个直接的回答，也可能是一个或多个工具调用请求。
- 将 LLM 的响应（AIMessage，可能包含 tool_calls）更新到 `state['messages']` 中。

In [214]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import AIMessage

# 初始化 LLM 模型，并绑定我们定义的记忆工具
# 使用支持函数调用的模型，如 gpt-4o-mini, gpt-4-turbo 等
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)
agent_llm_with_tools = llm.bind_tools(memory_tools)

def agent_node(state: AgentState, config: RunnableConfig):
    """
    代理节点：与 LLM 交互，处理记忆和对话历史。
    """
    user_id = config["configurable"].get("user_id") # 获取 user_id 用于提示
    recalled_memories = state.get('recall_memories',)
    messages = state['messages']

    # 构建系统提示，包含角色、指令、可用工具和检索到的记忆
    system_prompt_content = (
        "你是一个乐于助人的 AI 助手，拥有长期记忆能力。\n"
        f"当前用户ID: {user_id}\n"
        "你可以使用以下工具来管理关于该用户的长期记忆:\n"
        "- 'upsert_memory': 当用户要求你记住某事，或对话中出现关键信息时，用此工具存储记忆。\n"
        "- 'search_memory': 当你需要回忆关于用户的旧信息来回答问题时，用此工具搜索记忆。\n"
        "在回答用户问题前，请先检查下面的'相关记忆'部分，看看是否有用信息。\n"
        "不要在你的回答中提及'记忆'这个词，自然地使用信息即可。\n"
        "----\n"
        "相关记忆:\n"
        f"{'\n'.join(recalled_memories) if recalled_memories else '无'}\n"
        "----"
    )
    # print(f"构建的系统提示:\n{system_prompt_content}") # 调试用

    # 准备传递给 LLM 的消息列表
    system_message = SystemMessage(content=system_prompt_content)
    llm_messages = [system_message] + messages # 将系统消息放在列表开头

    print(f"调用 LLM (用户 {user_id})...")
    # 调用绑定了工具的 LLM
    ai_response: AIMessage = agent_llm_with_tools.invoke(llm_messages)
    print(f"LLM 响应: {ai_response.content}")
    if ai_response.tool_calls:
        print(f"LLM 请求调用工具: {ai_response.tool_calls}")

    # 返回包含新 AI 消息的状态更新
    return {"messages": [ai_response]}

##### execute_tools_node (执行工具)

- 功能: 执行由 agent 节点请求的工具调用。
- 实现: 这是一个普通的函数节点。
    - 从 `state['messages']` 中获取最新的 AIMessage，并检查其 tool_calls 属性。
    - 如果不存在 tool_calls，则直接返回（不应发生，因为条件边会处理）。
    - 从 config 中获取 user_id。
    - 遍历 tool_calls。
    - 对于每个 tool_call，获取工具名称 (tool_name)、参数 (tool_input) 和调用 ID (tool_call_id)。
    - 根据 tool_name，调用相应的私有函数 (_upsert_memory 或 _search_memory)，并将 tool_input 中的参数以及获取到的 user_id 传递给它。
    - 将私有函数返回的结果包装成 ToolMessage 对象，使用对应的 tool_call_id。
    - 收集所有 ToolMessage 对象。
    - 状态更新: 返回一个字典，其中 messages 键对应收集到的 ToolMessage 列表，这些消息将被追加到 `state['messages']` 中。

代码片段 (使用自定义 ToolExecutor):

In [215]:
from langchain_core.messages import ToolMessage

def execute_tools_node(state: AgentState, config: RunnableConfig):
    """
    执行由 LLM 请求的工具调用。
    从 config 获取 user_id 并传递给实际的记忆操作函数。
    """
    messages = state['messages']
    last_message = messages[-1]

    # 检查是否存在工具调用
    if not last_message.tool_calls:
        print("警告：execute_tools_node 被调用，但没有工具调用。")
        return {} # 或者根据需要返回特定状态

    user_id = config["configurable"].get("user_id")
    if not user_id:
        print("错误: execute_tools_node 无法获取 user_id")
        # 返回错误信息或空结果，避免执行工具
        error_tool_messages = [
            ToolMessage(
                content=f"执行工具 '{tc['name']}' 失败：无法获取 user_id。",
                tool_call_id=tc['id']
            )
            for tc in last_message.tool_calls
        ]
        # 返回包含这些错误消息的字典以更新状态
        return {"messages": error_tool_messages}

    tool_messages = []

    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_input = tool_call["args"]
        tool_call_id = tool_call["id"]
        print(f"执行工具 '{tool_name}' (用户 {user_id})，输入: {tool_input}")

        output = ""
        try:
            if tool_name == "upsert_memory":
                # 调用实际的存储函数，传入 user_id
                output = _upsert_memory(tool_input["memory_content"], user_id)
            elif tool_name == "search_memory":
                # 调用实际的搜索函数，传入 user_id
                output = _search_memory(tool_input["search_query"], user_id)
            else:
                output = f"错误：未知的工具 '{tool_name}'"
        except Exception as e:
            output = f"执行工具 '{tool_name}' 时出错: {e}"
            print(output) # 打印错误详情

        print(f"工具 '{tool_name}' 输出: {output}")
        tool_messages.append(ToolMessage(content=str(output), tool_call_id=tool_call_id))

    return {"messages": tool_messages}

创建一个自定义的 MemoryToolExecutor 类来显式处理 user_id 的获取和传递。


#### 编排流程：定义图的边和条件逻辑
现在我们将节点连接起来，定义代理的工作流程。

- StateGraph 初始化: 创建 StateGraph 实例，并指定我们定义的 AgentState 。   

- 添加节点: 使用 add_node 方法将我们定义的 `load_memories_node`、`agent_node` 和 `tool_node` 添加到图中 。   

- 入口点: 定义图的起始节点。我们将从 `load_memories_node` 开始，以便在代理思考之前加载相关记忆 。使用 `builder.add_edge(START, 'load_memories_node')`。   
- 连接加载器到代理: 将 `load_memories_node` 的输出连接到 `agent_node：builder.add_edge('load_memories_node', 'agent_node')`。
- 条件边 (代理到工具或结束): 这是工作流的核心路由逻辑。在 agent_node 执行后，需要根据 LLM 的响应决定下一步：
    - 如果 AIMessage 包含 tool_calls，则路由到 tool_node 以执行工具。
    - 如果 AIMessage 不包含 tool_calls（即 LLM 直接给出了最终答案），则路由到 END，结束当前回合。
    - 使用 `builder.add_conditional_edges` 方法，配合一个路由函数来实现。该函数检查 `state['messages'][-1]` 是否有 tool_calls 属性 。
- 连接工具回代理: 在 tool_node 执行完工具后，需要将结果（ToolMessage）反馈给 agent_node，让 LLM 能够基于工具执行的结果生成最终响应。使用 `builder.add_edge('tool_node', 'agent_node')`。

In [216]:
from langgraph.graph import StateGraph, START, END

# 创建 StateGraph 实例
builder = StateGraph(AgentState)

# 添加节点
builder.add_node("load_memories_node", load_memories_node)
builder.add_node("agent_node", agent_node)
# 使用新的 execute_tools_node 函数作为节点
builder.add_node("tool_node", execute_tools_node) # 节点名称仍为 "tool_node" 以保持一致性

# 定义入口点
builder.set_entry_point("load_memories_node") # 使用 set_entry_point

# 连接加载器到代理
builder.add_edge("load_memories_node", "agent_node")

# 定义条件路由逻辑
def should_continue(state: AgentState) -> str:
    """
    决定在 agent_node 之后是调用工具还是结束。
    """
    last_message = state['messages'][-1]
    # 检查 AIMessage 是否有 tool_calls 属性且不为空
    if getattr(last_message, "tool_calls", None):
        # 如果 LLM 请求调用工具，则路由到 tool_node
        print("路由决策：调用工具 (tool_node)")
        return "tool_node"
    else:
        # 否则，结束当前回合
        print("路由决策：结束 (END)")
        return END

# 添加从 agent_node 出发的条件边
builder.add_conditional_edges(
    "agent_node",
    should_continue,
    {
        "tool_node": "tool_node", # 目标是名为 "tool_node" 的节点 (即 execute_tools_node)
        END: END
    }
)

# 添加从 tool_node (execute_tools_node) 回到 agent_node 的边
builder.add_edge("tool_node", "agent_node")

print("图的节点和边已定义。")

图的节点和边已定义。


#### 编译图并配置状态持久化 (检查点)
最后一步是编译图，并为其配置检查点以实现对话状态（短期记忆）的持久化。

- 检查点选择: 对于本教程，我们将使用 MemorySaver，这是一个简单的内存检查点，用于在单个程序运行期间保持对话线程的状态 。如果需要跨程序运行持久化对话状态，可以使用 SqliteSaver  或 RedisSaver 。   
- 编译: 使用 builder.compile() 方法编译图。将 checkpointer 参数设置为我们选择的 MemorySaver 实例 。   
- 可视化 (可选但推荐): 可以使用 graph.get_graph().draw_mermaid_png() 生成图的可视化表示，有助于理解工作流程 。需要安装 pygraphviz 或其他依赖。   

代码片段：编译图并设置检查点

In [232]:
from langgraph.checkpoint.memory import MemorySaver

# 初始化内存检查点
memory_saver = MemorySaver()

# 编译图，并附加检查点
# 检查点负责保存每个 thread_id 的对话状态 (STM)
graph = builder.compile(checkpointer=memory_saver)

print("图已编译并配置了内存检查点。")

from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image, display # 如果在 Jupyter 中需要显示

print("\n正在尝试使用 Pyppeteer 生成图可视化...")

# 指定使用 Pyppeteer 方法进行绘制
mermaid_syntax = graph.get_graph().draw_mermaid()
print(mermaid_syntax)

图已编译并配置了内存检查点。

正在尝试使用 Pyppeteer 生成图可视化...
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	load_memories_node(load_memories_node)
	agent_node(agent_node)
	tool_node(tool_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> load_memories_node;
	agent_node -.-> tool_node;
	load_memories_node --> agent_node;
	tool_node --> agent_node;
	agent_node -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



1. 复制上面 mermaid 文本。
2. 打开浏览器访问 Mermaid 在线编辑器: https://mermaid.live
3. 将复制的文本粘贴到在线编辑器的 'Code' 区域，即可看到图形。
4. 你也可以将此语法保存到 .md 文件中，在支持 Mermaid 的 Markdown 查看器（如 GitHub、Obsidian）中查看。

### 演示持久化记忆：跨会话运行代理
现在，我们将通过具体的交互示例来验证代理的长期记忆能力。关键在于使用 configurable 字典来区分不同的用户 (user_id) 和不同的对话会话 (thread_id) 。   

- configurable 字典的角色:
    - thread_id: 由检查点（MemorySaver）使用，用于区分和持久化**单个对话会话**的状态（短期记忆）。不同的 thread_id 代表独立的对话历史。
    - user_id: 由我们自定义的长期记忆逻辑（load_memories_node 和 execute_tools_node 中的 _search_memory, _upsert_memory）使用，用于在向量存储中隔离和查找特定用户的持久化记忆（长期记忆）。

这种 user_id 和 thread_id 的分离是实现跨会话用户记忆的核心机制 。   

#### 交互 1：存储信息 (用户 user_123, 会话 A)
让第一个用户告诉代理一些信息，并要求它记住。

**代码片段：第一次交互**

In [224]:
# 定义配置，指定用户 ID 和会话 ID
config_user1_sessionA = {"configurable": {"user_id": "user_123", "thread_id": "session_A"}}

print("\n--- 交互 1: 用户 user_123, 会话 A ---")
print(f"配置: {config_user1_sessionA}")

# 第一次与代理交互，提供信息
initial_message_content = "你好，我叫 Bob，我住在巴黎。请记住这些信息。"
print(f"用户输入: {initial_message_content}")

# 使用 stream 方法查看详细步骤
print("\n--- 开始执行 graph.stream ---")
final_state = None # 用于存储最终状态

for event in graph.stream(
    {"messages": [HumanMessage(content=initial_message_content)]}, # 输入必须是 AgentState 格式
    config=config_user1_sessionA,
    stream_mode="values" # 获取每个节点后的完整状态
):
    # event 结构: {node_name: full_state_after_node}
    if not event:
        print("接收到空事件，跳过。")
        continue
    print()
    # 获取节点名称和该节点运行后的状态
    current_state = event
    final_state = current_state # 保留最后的状态

    print(f"\n--- 事件: 节点 '{node_name}' 完成 ---")
    # 打印当前状态的部分信息用于调试
    print(f"当前消息数量: {len(current_state.get('messages',))}")
    print(f"当前加载的记忆: {current_state.get('recall_memories',)}")

    # 检查并打印最后一条消息
    if isinstance(current_state, dict) and "messages" in current_state and current_state["messages"]:
        last_message = current_state["messages"][-1]
        print(f"节点 '{node_name}' 之后的最后消息:")
        last_message.pretty_print() # 使用 pretty_print() 以获得更好的格式
    else:
        print(f"节点 '{node_name}' 之后，状态格式不符合预期或无消息。")
        print(f"当前状态类型: {type(current_state)}")
        print(f"当前状态内容: {current_state}") # 打印整个状态以供调试

print("\n--- graph.stream 执行完毕 ---")

# 打印最终的 AI 回复 (如果存在)
if final_state and final_state.get("messages"):
    print("\n最终 AI 回复:")
    final_state["messages"][-1].pretty_print()
else:
    print("\n未能获取最终 AI 回复。")

# 检查向量存储中是否添加了数据 (调试用)
print("\n调试：检查向量存储内容 (搜索 'Bob')...")
try:
     results = vector_store.similarity_search("Bob", k=5) # 尝试搜索验证
     print(f"搜索 'Bob' 找到 {len(results)} 条结果:")
     for i, doc in enumerate(results):
         # 只显示与 user_123 相关的内容
         if doc.metadata.get("user_id") == "user_123":
             print(f"  - 相关结果 {i+1}: Content='{doc.page_content}', Metadata={doc.metadata}")
         else:
             print(f"  - 无关结果 {i+1}: Metadata={doc.metadata}")
except Exception as e:
     print(f"无法检查 InMemoryVectorStore 内容: {e}")


--- 交互 1: 用户 user_123, 会话 A ---
配置: {'configurable': {'user_id': 'user_123', 'thread_id': 'session_A'}}
用户输入: 你好，我叫 Bob，我住在巴黎。请记住这些信息。

--- 开始执行 graph.stream ---


--- 事件: 节点 'messages' 完成 ---
当前消息数量: 8
当前加载的记忆: []
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你好，我叫 Bob，我住在巴黎。请记住这些信息。
为用户 user_123 加载记忆，基于查询: '你好，我叫 Bob，我住在巴黎。请记住这些信息。'
为用户 user_123 找到相关记忆:
- 用户的名字是 Bob，住在巴黎。
加载到状态的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字是 Bob，住在巴黎。']


--- 事件: 节点 'messages' 完成 ---
当前消息数量: 8
当前加载的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字是 Bob，住在巴黎。']
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你好，我叫 Bob，我住在巴黎。请记住这些信息。
调用 LLM (用户 user_123)...
LLM 响应: 
LLM 请求调用工具: [{'name': 'upsert_memory', 'args': {'memory_content': '用户的名字是 Bob，住在巴黎。'}, 'id': 'call_0_9c3218c6-997e-4447-bec1-63ece538030b', 'type': 'tool_call'}]
路由决策：调用工具 (tool_node)


--- 事件: 节点 'messages' 完成 ---
当前消息数量: 9
当前加载的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字

预期输出分析:

1. `load_memories_node` 运行，但因为是第一次交互，找不到相关记忆。
2. `agent_node` 接收到用户消息，LLM 理解了“请记住这些信息”的指令。
3. LLM 决定调用 `upsert_memory` 工具，生成包含 `tool_calls` 的 AIMessage。
4. `should_continue` 路由到 tool_node (即 `execute_tools_node`)。
5. `execute_tools_node` 执行 upsert_memory 调用，它内部调用 `_upsert_memory` 函数，将 "我叫 Bob，我住在巴黎"（或类似内容）连同 `user_id="user_123"` 存入向量存储。并生成 ToolMessage 确认操作。
6. 流程回到 agent_node，LLM 看到工具执行成功的 ToolMessage。
7. LLM 生成最终回复，例如 "好的，我已经记住了：'我叫 Bob，我住在巴黎。'"。
8. should_continue 路由到 END。

#### 交互 2：检索信息 (用户 user_123, 会话 B - 新会话)
现在，模拟用户关闭了之前的对话窗口，重新打开一个新的对话（不同的 thread_id），并询问之前提供的信息。

**代码片段：第二次交互 (新会话)**

In [225]:
# 定义配置，使用相同的 user_id，但不同的 thread_id
config_user1_sessionB = {"configurable": {"user_id": "user_123", "thread_id": "session_B"}}

print("\n\n--- 交互 2: 用户 user_123, 会话 B (新会话) ---")
print(f"配置: {config_user1_sessionB}")

# 用户在新会话中提问
question_content = "你知道我叫什么名字吗？我住在哪里？"
print(f"用户输入: {question_content}")

# 再次调用 stream
print("\n--- 开始执行 graph.stream ---")
final_state_b = None
for event in graph.stream(
    {"messages": [HumanMessage(content=question_content)]},
    config=config_user1_sessionB,
    stream_mode="values"
):
    if not event: continue
    current_state = event
    final_state_b = current_state

    print(f"\n--- 事件: 节点 '{node_name}' 完成 ---")
    print(f"当前消息数量: {len(current_state.get('messages',))}")
    print(f"当前加载的记忆: {current_state.get('recall_memories',)}")

    if isinstance(current_state, dict) and "messages" in current_state and current_state["messages"]:
        last_message = current_state["messages"][-1]
        print(f"节点 '{node_name}' 之后的最后消息:")
        last_message.pretty_print()
    else:
        print(f"节点 '{node_name}' 之后，状态格式不符合预期或无消息。")
        print(f"当前状态类型: {type(current_state)}")
        print(f"当前状态内容: {current_state}")

print("\n--- graph.stream 执行完毕 ---")

# 打印最终的 AI 回复
if final_state_b and final_state_b.get("messages"):
    if isinstance(final_state_b["messages"][-1], AIMessage):
        print("\n最终 AI 回复:")
        final_state_b["messages"][-1].pretty_print()
    else:
        print("\n最终状态的最后一条消息不是 AI 回复。")
else:
    print("\n未能获取最终状态或最终状态无消息。")



--- 交互 2: 用户 user_123, 会话 B (新会话) ---
配置: {'configurable': {'user_id': 'user_123', 'thread_id': 'session_B'}}
用户输入: 你知道我叫什么名字吗？我住在哪里？

--- 开始执行 graph.stream ---

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 2
当前加载的记忆: None
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你知道我叫什么名字吗？我住在哪里？
为用户 user_123 加载记忆，基于查询: '你知道我叫什么名字吗？我住在哪里？'
为用户 user_123 找到相关记忆:
- 用户的名字是 Bob，住在巴黎。
- 用户的名字是 Bob，住在巴黎。
加载到状态的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字是 Bob，住在巴黎。\n- 用户的名字是 Bob，住在巴黎。']

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 2
当前加载的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字是 Bob，住在巴黎。\n- 用户的名字是 Bob，住在巴黎。']
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你知道我叫什么名字吗？我住在哪里？
调用 LLM (用户 user_123)...
LLM 响应: 你叫 Bob，住在巴黎。
路由决策：结束 (END)

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 3
当前加载的记忆: ['在您的记忆中找到以下相关信息：\n- 用户的名字是 Bob，住在巴黎。\n- 用户的名字是 Bob，住在巴黎。']
节点 'messages' 之后的最后消息:
================================== Ai Message ==================

预期输出分析:

- `load_memories_node` 运行，打印日志。这次它使用 `user_id="user_123"` 和查询（用户问题）调用 `_search_memory`。
- `_search_memory` 在向量存储中找到之前存储的关于 Bob 和巴黎的记忆，并返回格式化的字符串。
- `load_memories_node` 将检索到的记忆放入 `state['recall_memories']`。
- `agent_node` 运行，打印包含已加载记忆的系统提示和用户问题。LLM 结合问题和记忆生成回答。
- `LLM` 不需要调用工具，AIMessage 不包含 `tool_calls`。
- `should_continue` 运行，打印决策，路由到 END。
- Stream 结束，打印最终 AI 回复，其中应包含 Bob 和巴黎的信息 。
   
这个交互证明了代理能够**跨越不同的会话 (thread_id)**，为**同一个用户 (user_id)** 检索并使用长期记忆。

#### 交互 3：不同用户 (用户 user_456, 会话 C)
为了验证记忆是用户隔离的，我们模拟一个全新的用户进行交互。

**代码片段：第三次交互 (新用户)**

In [226]:
# 定义配置，使用新的 user_id 和 thread_id
config_user2_sessionC = {"configurable": {"user_id": "user_456", "thread_id": "session_C"}}

print("\n\n--- 交互 3: 用户 user_456, 会话 C (新用户) ---")
print(f"配置: {config_user2_sessionC}")

# 新用户提问
question_new_user_content = "你知道我叫什么名字吗？"
print(f"用户输入: {question_new_user_content}")

# 调用 stream
print("\n--- 开始执行 graph.stream ---")
final_state_c = None
for event in graph.stream(
    {"messages": [HumanMessage(content=question_new_user_content)]},
    config=config_user2_sessionC,
    stream_mode="values"
):
    if not event: continue
    current_state = event
    final_state_c = current_state

    print(f"\n--- 事件: 节点 '{node_name}' 完成 ---")
    print(f"当前消息数量: {len(current_state.get('messages',))}")
    print(f"当前加载的记忆: {current_state.get('recall_memories',)}")

    if isinstance(current_state, dict) and "messages" in current_state and current_state["messages"]:
        last_message = current_state["messages"][-1]
        print(f"节点 '{node_name}' 之后的最后消息:")
        last_message.pretty_print()
    else:
        print(f"节点 '{node_name}' 之后，状态格式不符合预期或无消息。")
        print(f"当前状态类型: {type(current_state)}")
        print(f"当前状态内容: {current_state}")

print("\n--- graph.stream 执行完毕 ---")

# 打印最终的 AI 回复
if final_state_c and final_state_c.get("messages"):
    if isinstance(final_state_c["messages"][-1], AIMessage):
        print("\n最终 AI 回复:")
        final_state_c["messages"][-1].pretty_print()
    else:
        print("\n最终状态的最后一条消息不是 AI 回复。")
else:
    print("\n未能获取最终状态或最终状态无消息。")



--- 交互 3: 用户 user_456, 会话 C (新用户) ---
配置: {'configurable': {'user_id': 'user_456', 'thread_id': 'session_C'}}
用户输入: 你知道我叫什么名字吗？

--- 开始执行 graph.stream ---

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 1
当前加载的记忆: None
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你知道我叫什么名字吗？
为用户 user_456 加载记忆，基于查询: '你知道我叫什么名字吗？'
未找到相关记忆或发生错误。

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 1
当前加载的记忆: []
节点 'messages' 之后的最后消息:
================================ Human Message =================================

你知道我叫什么名字吗？
调用 LLM (用户 user_456)...
LLM 响应: 
LLM 请求调用工具: [{'name': 'search_memory', 'args': {'search_query': '用户的名字'}, 'id': 'call_0_6aa4bd3d-e8c2-414a-8d94-42305f06b2d0', 'type': 'tool_call'}]
路由决策：调用工具 (tool_node)

--- 事件: 节点 'messages' 完成 ---
当前消息数量: 2
当前加载的记忆: []
节点 'messages' 之后的最后消息:
================================== Ai Message ==================================
Tool Calls:
  search_memory (call_0_6aa4bd3d-e8c2-414a-8d94-42305f06b2d0)
 Call ID: call_0_

预期输出分析:

- `load_memories_node` 运行，打印日志，使用 `user_id="user_456"` 进行搜索。
- `_search_memory` 找不到任何与 `user_456` 相关的记忆，返回 "没有找到..." 的消息。
- `state['recall_memories']` 为空。
- `agent_node` 运行，系统提示中显示“无相关记忆被加载”。
- `LLM` 根据当前对话（只有用户的问题）生成回答，例如 "抱歉，我目前还不知道你的名字。"
- `should_continue` 运行，打印决策，路由到 END。
- `Stream` 结束，打印最终 AI 回复。

这个交互证明了长期记忆是基于 user_id 进行隔离的，一个用户的记忆不会泄露给另一个用户。

### 高级考量与最佳实践

虽然一个基础的长期记忆代理实现是功能性的，但在实际应用中还需要考虑一些更高级的方面。

#### 记忆管理策略

* **工具调用 vs. 手动管理**:
    * **工具调用**: 将记忆操作（存储/检索）暴露为工具，由大型语言模型 (LLM) 根据上下文决定何时调用。优点是记忆管理与代理的核心推理逻辑结合紧密，实现相对简单。缺点是依赖 LLM 的判断，可能调用不准确或遗漏，且增加了 LLM 的推理负担。
    * **手动管理**: 在代理的代码逻辑中（例如在特定节点或边上）显式地决定何时以及如何存储或检索记忆。优点是开发者对记忆操作有更精确的控制。缺点是可能需要更复杂的代码逻辑来判断时机和内容，灵活性可能稍差。

* **热路径 vs. 后台处理**:
    * **热路径 (Hot Path)**: 记忆更新（存储）作为对话流程的一部分实时发生。优点是记忆可以立即在后续对话中使用，对用户透明。缺点是可能会增加用户交互的延迟，因为需要等待记忆存储操作完成。
    * **后台处理 (Background / Subconscious Formation)**: 记忆更新在对话结束后或系统空闲时异步进行。优点是不会增加用户交互的延迟，可以进行更复杂的记忆处理或“反思”（Reflection）以提高记忆质量。缺点是记忆更新不是即时的，需要额外的后台处理资源和架构。

* **记忆摘要与整合**:
    * **短期记忆管理**: 对于非常长的对话，仅靠保存完整的消息历史可能会导致上下文窗口溢出或效率低下。可以引入一个摘要机制，定期将早期的对话消息压缩成摘要，以控制短期记忆的大小。
    * **长期记忆维护**: 随着时间的推移，长期记忆库可能会变得庞大、冗余甚至包含过时信息。可能需要实现机制来定期回顾、整合、提炼或删除旧的记忆，以保持其相关性和有效性。

#### 扩展记忆：选择合适的存储方案

内存中的向量存储（如 InMemoryVectorStore）仅适用于演示和原型开发。对于需要持久化、可扩展和高性能的生产应用，需要选择更强大的存储后端。

* **Redis**: 一个非常流行的高性能内存数据存储，常被用作缓存、消息队列和数据库。`langgraph-redis` 包提供了对 Redis 的原生支持，可用作 LangGraph 的检查点（`RedisSaver`）和长期记忆存储（`RedisStore`），后者还支持向量搜索功能。Redis 提供了低延迟读写 (<1ms) 和灵活的数据结构（如 JSON），非常适合存储代理状态和记忆。
* **Elasticsearch**: 一个强大的分布式搜索和分析引擎，也具备向量搜索能力。如果系统中已经在使用 Elasticsearch，将其用作记忆存储是一个自然的选择。
* **其他向量数据库**: 市面上有许多专门为向量搜索优化的数据库，如 Pinecone, Weaviate, Chroma, Milvus, Qdrant 等。它们通常提供更高级的向量索引和搜索功能。
* **PostgreSQL (pgvector)**: 如果已经在使用 PostgreSQL 数据库，可以通过 `pgvector` 扩展为其增加向量存储和搜索能力。
* **专用记忆平台**: 如 Zep，提供了专门为 AI 代理设计的长期记忆解决方案，可能包含更丰富的功能，如自动摘要、实体提取等。

**表 : 记忆存储选项比较 (简要)**

| 存储类型           | 持久性       | 可扩展性   | 向量搜索     | 设置复杂度   | 主要用例                     |
| :----------------- | :----------- | :--------- | :----------- | :----------- | :--------------------------- |
| InMemoryVectorStore | 会话/进程    | 低         | 内建 (基本)  | 低           | 演示, 快速原型               |
| Redis (w/ Search)  | 磁盘/服务器  | 高         | 内建 (通过模块)| 中           | 高性能缓存, 状态管理, 生产级 LTM |
| Elasticsearch      | 磁盘/服务器  | 非常高     | 内建         | 中高         | 企业搜索, 日志分析, 生产级 LTM |
| PostgreSQL (pgvector)| 磁盘/服务器  | 高         | 通过扩展     | 中           | 已使用 PG 的系统, 结构化数据+向量|
| 专用向量数据库     | 磁盘/服务器  | 高         | 核心功能 (高级)| 中           | 大规模向量搜索应用           |
| Zep                | 磁盘/服务器  | 高         | 内建 (平台特性)| 中           | 完整的 AI 代理记忆平台       |

选择哪种存储取决于具体的应用需求、预期的负载、团队的技术栈以及对持久性、可扩展性和高级功能的要求。

#### 有效地结构化记忆数据

仅仅存储原始文本可能不是最高效的方式。结构化地存储记忆可以带来更多好处。

* **JSON 格式**: 将记忆存储为 JSON 文档，而不仅仅是纯文本。这允许在记忆条目中包含多个字段。
* **元数据 (Metadata)**: 在存储记忆时附加丰富的元数据，例如：
    * `user_id` (必需，用于隔离)
    * `timestamp` (记忆创建或更新的时间)
    * `thread_id` (可选，关联到特定对话)
    * `memory_type` (例如 'preference', 'fact', 'instruction', 'experience')
    * `importance_score` (由 LLM 评估或基于规则确定)
    这使得在检索时可以进行更精细的过滤和排序。
* **结构化信息提取**: 对于某些信息，可以考虑将其提取为结构化格式，例如知识三元组（主语, 谓语, 宾语），或者 Pydantic 模型定义的结构。这有助于进行更精确的查询或与其他知识库集成。
* **多向量索引**: 对于复杂的记忆条目，可以考虑为不同的方面创建不同的向量表示（例如，为记忆内容本身创建一个向量，为相关的情感或上下文创建另一个向量），并分别进行索引和搜索，以实现更细致的语义匹配。

#### 安全与隐私

当代理存储用户信息，尤其是个人身份信息 (PII) 时，必须高度重视安全和隐私。

* **数据最小化**: 只存储真正必要的信息。
* **访问控制**: 确保只有授权的组件（例如特定用户的代理实例）才能访问其对应的记忆。`user_id` 隔离是基础。
* **存储安全**: 对记忆存储本身进行适当的安全配置（例如，网络访问控制、认证、加密）。
* **合规性**: 遵守相关的隐私法规（如 GDPR, CCPA）。
* **透明度**: 考虑让用户了解代理存储了哪些关于他们的信息，并提供管理（例如查看、删除）这些信息的途径。

### 结论：回顾与展望

#### 主要学习点总结

构建具有长期记忆能力的 AI 代理涉及多个核心概念和技术。主要学习点包括：

* 理解 LangGraph 的基本构建块：状态（State）、节点（Nodes）、边（Edges）、图（StateGraph）和检查点（Checkpointers）。
* 区分短期记忆（通常由检查点管理的会话内上下文）和长期记忆（跨会话持久化的用户信息）。
* 利用向量存储和嵌入模型实现长期记忆的语义存储和检索。
* 通过定义工具并让 LLM 调用它们来管理长期记忆操作（如存储和搜索）。
* 在代理工作流中设计关键节点和条件边，以整合记忆加载、代理推理和工具执行。
* 理解并利用配置参数（如 `user_id` 和 `thread_id`）来实现用户级长期记忆隔离和会话级短期记忆持久化。

#### 价值重申

为 AI 代理添加长期记忆能力极大地增强了其实用性和用户体验。它使得代理能够：

* **个性化交互**: 根据用户的历史偏好和信息定制响应。
* **保持连续性**: 在跨越多个会话的长时间交互中保持上下文理解。
* **从经验中学习**: 记住过去的成功或失败，并可能据此调整行为（尽管更高级的学习机制可能需要额外实现）。
* **建立用户信任**: 通过展现出对用户的了解和记忆，建立更自然、更值得信赖的人机关系。

#### 未来方向

构建的代理可以作为一个坚实的基础，在此之上进行许多扩展和探索：

* **实现后台记忆处理**: 探索异步更新记忆的模式，以减少交互延迟并进行更复杂的记忆整合。
* **尝试不同的向量存储**: 将简单的内存存储替换为生产级的存储方案，如 Redis、Elasticsearch 或其他向量数据库，并评估其性能和特性。
* **实现记忆摘要**: 为短期记忆（对话历史）添加摘要机制，以处理超长对话。
* **探索更复杂的记忆结构**: 尝试存储结构化记忆（如知识三元组）或实现多向量表示。
* **集成知识图谱**: 将向量存储与知识图谱结合，实现更丰富的知识表示和推理能力。
* **记忆反思与提炼**: 实现更高级的机制，让代理能够“反思”其记忆，进行提炼、泛化或纠错。

## 使用Gradio构建生成式人工智能应用程序

### 基础：面向生成式AI开发的Gradio

#### 理解Gradio：在生成式AI生态系统中的目标与优势

Gradio是一个开源Python库，旨在简化为机器学习模型、API或任何Python函数创建用户友好的Web界面的过程。其主要关注点在于快速原型设计和使机器学习模型易于访问。

对于生成式人工智能（GenAI）而言，Gradio提供了一个关键优势。GenAI模型通常需要迭代式的提示工程和参数调整。Gradio通过快速创建UI来促进这一过程，允许开发者和用户实时与模型交互。这对于文本生成、图像合成和聊天机器人交互等任务尤其有价值。GenAI模型的开发本质上是迭代的，例如需要反复调整提示词或模型参数。传统的UI开发可能会成为这个迭代过程中的瓶颈。然而，Gradio允许开发者仅用“几行代码”就能构建一个界面，这极大地加速了测试、评估和改进GenAI模型的周期。通过抽象化前端的复杂性，开发者可以将精力集中在模型的核心逻辑和性能上，这一点在文本摘要、图像生成和聊天机器人等应用中得到了体现。因此，Gradio成为了加速GenAI迭代过程的重要工具。

由DeepLearning.AI和Hugging Face联合推出、吴恩达（Andrew Ng）和Apolinário Passos参与的“使用Gradio构建生成式AI应用”课程，特别强调了Gradio在为文本摘要、图像描述、图像生成和大型语言模型（LLM）聊天机器人等GenAI应用创建演示方面的实用性。该课程着重于构建友好的Web界面，而无需前端编码专业知识。

Gradio的易用性降低了展示复杂GenAI模型的门槛，使其能够被非技术背景的相关者接触，并促进了更快的反馈循环。GenAI模型虽然功能强大，但如果没有易于访问的界面，其内部工作原理可能难以理解。Gradio允许通过公共链接共享交互式演示，使得更广泛的受众（包括非编码人员、利益相关者、测试用户）能够体验GenAI应用并提供反馈。这使得Gradio在推动GenAI能力的普及化和理解方面发挥了作用，将其从专业研究实验室带向更广泛的实际应用和审视。

此外，Gradio能够与TensorFlow、PyTorch和Hugging Face Transformers等流行的机器学习框架无缝集成，这些框架在GenAI开发中被广泛使用。

#### 环境配置：Gradio安装与设置

安装Gradio的过程非常直接，可以通过pip完成：`pip install gradio`。推荐在虚拟环境中使用Gradio。

基本的导入语句是 `import gradio as gr`，这已成为广泛采用的标准约定。

对于GenAI应用，可能还需要安装额外的库。例如，要使用Ollama，需要安装ollama库 (`pip install ollama`)。要使用DeepSeek API（或其他符合OpenAI规范的API），需要安装openai库 (`pip install openai`)。如果仍需使用如Stable Diffusion等模型（例如用于图像生成），则可能还需要`diffusers`和`torch`。这些库通常也通过pip安装。

核心的Gradio库可以通过一个简单的pip命令安装，为现有Python函数创建简单的界面只需要这个核心库。这种创建UI的低门槛意味着开发者可以快速可视化函数输出，而无需为UI部分本身进行复杂的设置。复杂性随后转移到安装和配置机器学习/GenAI模型及其特定依赖项（如 ollama, openai, diffusers），而不是Gradio本身。

使用Ollama时，需要确保Ollama服务正在本地运行，并且已经拉取了所需的模型（例如 `ollama pull llama3.2` 或 `ollama pull llava`）。使用DeepSeek API时，需要获取API密钥并进行配置（例如，作为环境变量）。

### 核心Gradio构造：用户界面设计

#### gr.Interface：快速UI原型设计

`gr.Interface` 类是Gradio的高级API，用于通过为Python函数提供输入和输出列表来快速创建Web演示。它非常适合单函数应用以及对布局定制要求最低的场景。

许多Python函数，尤其是在机器学习领域，接受特定数据类型作为输入并产生特定数据类型作为输出。`gr.Interface` 允许开发者使用简单的字符串快捷方式（例如，“text”、“image”）或组件实例（例如 `gr.Textbox()`）来指定这些输入/输出类型。Gradio随后会自动生成相应的Web UI元素（文本框、图像上传字段等），并处理Web格式和Python类型之间的数据转换。这种抽象消除了开发者为基本UI编写任何HTML、CSS或JavaScript的需求，使得将模型或函数“上线”变得极其迅速。这是贯穿研究材料的核心价值主张。

1.  基本参数：
    * `fn`：需要包装UI的Python函数。此函数处理输入并返回输出。
    * `inputs`：用于用户输入的Gradio组件或组件列表。组件数量应与函数的参数数量匹配。
    * `outputs`：用于显示函数返回值的Gradio组件或组件列表。
    * 其他常用参数包括用于提供文本上下文的 `title`、`description`、`article`，用于预填充输入的 `examples`，以及用于实时更新的 `live`。
    * `launch()` 方法启动Web服务器。设置 `share=True` 会生成一个公共链接。

2.  GenAI应用的关键输入/输出组件（参见表2）：
    Gradio提供了超过30个内置组件。这些组件处理将用户通过浏览器提交的数据转换为Python函数可用的格式（预处理），以及将Python函数返回的值转换为可在浏览器中显示的格式（后处理）。
    * `gr.Textbox`：用于文本提示（输入）或生成的文本（输出）。可通过 `lines`、`placeholder`、`label`、`interactive` 进行配置。
    * `gr.Image`：用于上传源图像（输入）或显示生成的图像（输出）。可通过 `type`（例如，“pil”、“filepath”）、`sources`（例如，“upload”、“webcam”）进行配置。
    * `gr.Audio`：用于语音命令（输入）或合成语音（输出）。可通过 `sources`、`type` 进行配置。
    * `gr.Slider`：用于数值参数，如温度、令牌数量、引导比例（输入）。可通过 `minimum`、`maximum`、`step`、`value`、`label` 进行配置。
    * `gr.Dropdown`：用于选择模型或选项（输入）。可通过 `choices`、`value`、`label` 进行配置。
    * `gr.Button`：用于触发操作（输入）。可通过 `value`（标签）、`variant` 进行配置。
    * `gr.Dataframe`、`gr.JSON`、`gr.Label`、`gr.Plot` 是其他用于显示各种类型输出的有用组件。

3.  示例：基本文本处理应用
    一个简单的函数，例如 `def greet(name): return "Hello " + name + "!"`，可以用 `gr.Interface(fn=greet, inputs="text", outputs="text")` 来包装。这展示了将Python函数映射到交互式UI元素的基本方法。

**表2：生成式AI应用的基本Gradio组件**

| 组件 (`gr.*`)      | GenAI中的典型用途                                  | 关键配置参数                                            |
| :----------------- | :------------------------------------------------- | :------------------------------------------------------ |
| Textbox            | 输入文本提示；显示生成的文本、摘要、标题               | `lines`, `placeholder`, `label`, `interactive`, `show_copy_button` |
| Image              | 上传输入图像；显示生成的图像                         | `type` ("pil", "numpy", "filepath"), `sources` ("upload", "webcam"), `width`, `height`, `interactive` |
| Slider             | 调整数值参数（温度、步数、引导比例、最大令牌数）     | `minimum`, `maximum`, `step`, `value`, `label`, `interactive` |
| Dropdown           | 选择模型、风格或其他离散选项                         | `choices`, `value`, `label`, `interactive`              |
| Checkbox           | 切换布尔选项（例如，“使用负面提示”）                 | `value`, `label`, `interactive`                       |
| Radio              | 从一组选项中单选（例如，选择操作模式）                 | `choices`, `value`, `label`, `interactive`              |
| Button             | 触发生成、提交或其他操作                             | `value` (label), `variant` ("primary", "secondary", "stop") |
| Audio              | 输入语音命令；播放生成的音频                         | `sources` ("upload", "microphone"), `type` ("numpy", "filepath"), `streaming` |
| Label              | 显示分类结果（例如，内容过滤器标签）                 | `value`, `num_top_classes`, `confidences`             |
| JSON / Dataframe   | 显示结构化输出或调试信息                             | `value`, `label`                                        |
| Chatbot            | 显示聊天对话历史（通常在`gr.Blocks`或`gr.ChatInterface`中使用） | `value`, `label`, `height`, `placeholder`, `likeable` |

注：此表总结了最常见的组件及其在GenAI场景下的应用，更多组件和详细信息请查阅Gradio官方文档。

#### gr.Blocks：高级UI定制与控制

`gr.Blocks` 提供了一个较低级别的API，用于对组件布局、数据流和事件驱动的交互进行更精细的控制。当 `gr.Interface` 的自动布局不足以满足需求时，应使用 Blocks。

GenAI应用很少是简单的单次函数调用。它们通常涉及模型链（例如，图像描述后根据描述生成图像）、条件逻辑或动态UI更新。`gr.Interface` 对于此类复杂流程来说过于受限。`gr.Blocks` 允许定义任意数据流，其中一个组件/函数的输出可以作为另一个组件/函数的输入。这种组合能力对于构建复杂的GenAI演示至关重要，这些演示不仅展示单个模型，还展示整个系统或流程。精确控制布局的能力也意味着开发者可以设计出直观引导用户完成这些多步骤过程的UI。

1.  使用 Blocks 的理由：
    * 将相关的演示分组到不同的选项卡中。
    * 使用行、列等自定义布局（例如，指定输入/输出的位置）。
    * 创建多步骤界面，其中一个模型/函数的输出成为下一个模型的输入。
    * 根据用户输入动态更改组件属性（例如，可见性、下拉列表中的选项）。

2.  布局管理：
    * 组件在 `with gr.Blocks() as demo:` 上下文中实例化。
    * `gr.Row()`：水平排列组件。可以控制子元素的 `scale` 和 `min_width`。
    * `gr.Column()`：垂直排列组件。通常嵌套在 `gr.Row()` 中以实现复杂布局。`scale` 和 `min_width` 也适用。
    * `gr.Tab("tab_name")`：将组件组织到可选择的选项卡中。
    * `gr.Accordion("label", open=False)`：创建可折叠的组件部分。
    * `gr.Group()`：在视觉上对组件进行分组。
    * 布局元素和组件的可见性可以通过 `visible` 参数控制。

3.  通过事件监听器实现交互性：
    用户与GenAI模型的交互本质上是事件驱动的（例如，用户输入提示并点击“生成”，用户上传图像，用户调整参数）。`gr.Blocks` 的事件监听器系统（例如 `.click()`、`.change()`）直接将这些用户操作映射到Python函数调用。这使得应用程序的逻辑更加明确，并且比可能需要更多事件处理样板代码的框架更容易管理。它允许开发者以“当X发生时，执行Y”的方式思考，这对于UI设计来说很自然。
    * 事件监听器通过将组件交互（例如，按钮点击、文本框更改）链接到处理输入并更新输出组件的Python函数来定义数据流。
    * 常见事件：按钮的 `.click()`，输入组件的 `.change()`，文本输入的 `.submit()`。每个组件支持的事件的完整列表可在Gradio文档中找到。
    * 事件监听器方法接受 `fn`（要调用的函数）、`inputs`（输入组件列表）和 `outputs`（输出组件列表）。
    * 输入可以作为列表（映射到函数参数）或集合（映射到函数中的单个字典参数）传递。输出可以作为列表或字典返回。
    * Gradio会根据事件触发器自动确定组件是否应具有交互性，但这可以通过 `interactive=True`/`False` 覆盖。

4.  示例：使用 Blocks 的多组件UI
    一个示例可以包含一个输入文本框、一个按钮和一个输出文本框，其中按钮的 `.click()` 事件触发一个函数，该函数处理输入文本并更新输出文本。一个多步骤过程的典型例子是：上传图像 -> 调用描述器函数 -> 输出文本 -> 调用生成器函数 -> 输出图像。

**表1：gr.Interface 与 gr.Blocks 对比**

| 特性         | gr.Interface            | gr.Blocks                           |
| :----------- | :---------------------- | :---------------------------------- |
| 主要用途     | 快速为单个函数创建简单Demo | 构建复杂、多步骤、自定义布局的应用     |
| 易用性       | 非常高，代码量最少      | 较高，但比Interface更复杂             |
| 布局控制     | 有限（自动布局）        | 高度灵活（Rows, Columns, Tabs等）     |
| 数据流       | 单一：输入 -> 函数 -> 输出 | 灵活：允许多个、链式或条件数据流      |
| 事件处理     | 隐式（基于live参数或提交按钮） | 显式（通过.click(), .change()等事件监听器） |
| 动态UI       | 有限                    | 支持根据交互动态更新组件属性（可见性、值等） |
| 适用场景     | 快速原型、简单模型展示    | 需要自定义布局、多模型交互、复杂工作流的应用 |

#### gr.ChatInterface：对话式AI的专用解决方案

`gr.ChatInterface` 是一个专门为快速构建聊天机器人UI而设计的高级类。

聊天机器人可以说是大型语言模型和GenAI最广泛的应用。从头开始构建一个好的聊天UI涉及到处理消息历史记录、用户输入、显示对话轮次、流式传输等，这可能很复杂。`gr.ChatInterface` 抽象了大部分这种复杂性。通过提供一个专门的类，Gradio显著降低了创建交互式聊天机器人演示的工作量，使开发者能够专注于LLM交互逻辑（fn函数），而不是UI的底层实现。这符合Gradio快速构建演示的理念。

1.  参数与功能概述：
    * 需要一个聊天函数 `fn`，该函数接受 `message`（用户输入）和 `history`（过去的消息列表）作为参数，并返回聊天机器人的响应。
    * 推荐使用 `type="messages"` 作为历史记录格式（包含“role”和“content”键的字典列表）。
    * 通过在聊天函数中使用 `yield` 支持流式响应。
    * 自定义选项包括 `chatbot`（用于自定义 `gr.Chatbot` 实例）、`textbox`（用于自定义 `gr.Textbox` 或 `gr.MultimodalTextbox`）、`title`、`description`、`examples`、`theme`。
    * `multimodal=True` 允许与文本消息一起上传文件。
    * `additional_inputs` 可用于添加其他UI元素，如用于温度的滑块等。

2.  与语言模型的集成：
    * 聊天函数 `fn` 是调用LLM（例如，通过Ollama或DeepSeek API调用模型）逻辑所在的位置。
    * 管理对话历史记录并为LLM提示正确格式化至关重要。
    * 诸如 `stop_sequences` 和系统消息之类的功能可以在聊天函数中实现，以控制LLM的行为。

### 构建生成式AI应用：实践实现

本节将介绍具体的GenAI应用示例，并讨论如何使用Gradio构建其用户界面。

#### 应用焦点1：文本摘要系统

1.  模型考量：
    * 可以使用在Ollama本地运行的通用LLM（如 llama3.2, mistral, qwen2.5 等）或通过DeepSeek API访问的模型（如 deepseek-chat, deepseek-reasoner）来进行文本摘要。
    * 需要构建合适的提示词来指导模型执行摘要任务。

2.  UI实现细节：
    * 通常涉及一个用于输入长文本的 `gr.Textbox` 和另一个用于输出摘要的 `gr.Textbox`。
    * `gr.Interface` 非常适合这种直接的输入/输出结构。
    * 可以调整 `gr.Textbox` 的 `lines` 等参数以更好地显示多行文本。
    * 使用标签、标题和描述可以增强可用性。
    * Gradio使得对不同摘要模型（通过Ollama或DeepSeek API）进行快速实验成为可能。核心UI结构保持不变，而模型调用可以在后端函数中轻松修改。
3. 带注释的代码示例 (使用Ollama)：

In [234]:
# %pip install gradio
# %pip install langchain langchain-community langchain-core

In [238]:
import gradio as gr
from langchain_community.chat_models import ChatOllama # LangChain Ollama集成
from langchain.chains.summarize import load_summarize_chain # LangChain摘要链
from langchain.docstore.document import Document # LangChain文档对象
import os

# --- LangChain Setup ---
# 确保Ollama服务正在运行，并且已拉取所需模型，例如 'ollama pull llama3.2'
# 初始化ChatOllama，指定模型和温度
llm = ChatOllama(model="llama3.2", temperature=0) # 使用你已拉取的Ollama模型

# 加载摘要链 (stuff类型适用于较短文本)
summarize_chain = load_summarize_chain(llm, chain_type="stuff") #

# --- Gradio Function ---
def summarize_with_langchain_ollama(text_to_summarize):
    """使用LangChain和Ollama进行文本摘要"""
    if not summarize_chain:
        return "Error: Summarization chain not initialized. Check Ollama setup."
    try:
        # LangChain摘要链需要Document对象列表作为输入
        docs = [Document(page_content=text_to_summarize)]
        # 调用链进行摘要
        summary_result = summarize_chain.invoke(docs) #
        # 提取输出文本
        return summary_result.get("output_text", "Sorry, could not generate summary.")
    except Exception as e:
        # 处理可能的API或LangChain错误
        if "connect" in str(e).lower():
             return f"Summarization Error: Could not connect to Ollama service. Is it running?"
        return f"Error generating summary: {e}"

# --- Gradio Interface ---
demo = gr.Interface(
    fn=summarize_with_langchain_ollama, # 指向新的LangChain函数
    inputs=gr.Textbox(
        lines=10,
        label="Input Text to Summarize",
        placeholder="Paste long text here..."
    ),
    outputs=gr.Textbox(
        label="Generated Summary (LangChain + Ollama)",
        lines=5
    ),
    title="Text Summarization (LangChain + Ollama)",
    description="Enter text to get a summary using LangChain and a local Ollama model.",
    allow_flagging="never"
)

# 启动 Gradio 应用
if __name__ == "__main__":
    print("Ensure Ollama service is running and the required model (e.g., 'llama3.2') is pulled.")
    demo.launch()

C:\Users\iSPACE\AppData\Local\Temp\ipykernel_24808\3329463669.py:11: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2", temperature=0) # 使用你已拉取的Ollama模型
D:\Users\iSPACE\anaconda3\envs\rag\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Ensure Ollama service is running and the required model (e.g., 'llama3.2') is pulled.
* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


#### 应用焦点2：图像描述界面
1. 模型考量：

- 可以使用在Ollama本地运行的多模态模型，如LLaVA (llava:7b, llava:13b 等) 。   
- DeepSeek API也提供多模态模型，可以通过其API进行调用。

2. 处理图像输入和生成文本输出：

- 输入：gr.Image() 组件，允许上传图像 。   
- 后端函数需要接收图像数据。对于Ollama LLaVA，ollama Python库可以直接处理图像文件路径或base64编码的图像数据 。Gradio的 -`gr.Image(type="filepath")` 或 `gr.Image(type="pil")` 后转换为base64是可行的。   
- 输出：`gr.Textbox()` 或 "text" 用于显示生成的标题 。   

3. 带注释的代码示例 (使用Ollama LLaVA)：

In [241]:
import gradio as gr
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from PIL import Image
import base64
from io import BytesIO
import os

# --- LangChain Setup ---
# 确保Ollama服务正在运行，并且已拉取LLaVA模型，例如 'ollama pull llava:7b'
try:
    llm = ChatOllama(model="llava:7b")
    print("ChatOllama 初始化成功.")
except Exception as e:
    llm = None
    print(f"ChatOllama 初始化失败: {e}. 请检查 Ollama 服务和模型.")

# --- Helper Function ---
def pil_to_base64(image: Image.Image) -> str:
    """将PIL图像转换为Base64编码的字符串"""
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

# --- Gradio Function ---
def caption_image_ollama(input_image_pil):
    """使用LangChain ChatOllama和LLaVA生成中文图像描述"""
    if llm is None:
        return "错误: LLM 未初始化. 请检查 Ollama 服务和模型."
    if input_image_pil is None:
         return "错误: 没有上传图片."

    try:
        image_b64 = pil_to_base64(input_image_pil.convert("RGB"))

        # 构建包含中文文本提示和图像的消息
        message = HumanMessage(
            content=[
                {"type": "text", "text": "用中文详细描述这张图片。"}, # 要求中文描述
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}},
            ]
        )

        response = llm.invoke([message])
        return response.content.strip()

    except Exception as e:
        error_message = str(e).lower()
        if "connect" in error_message or "connection refused" in error_message:
             return "描述错误: 无法连接到 Ollama 服务. 请确保它正在运行."
        if "model" in error_message and ("not found" in error_message or "pull" in error_message):
             return "描述错误: LLaVA 模型未找到. 请运行 'ollama pull llava:7b'."
        return f"发生未知错误: {e}"

# --- Gradio Interface ---
# 检查示例图片文件夹是否存在
example_dir = "images"
examples_list = [os.path.join(example_dir, f) for f in os.listdir(example_dir)] if os.path.exists(example_dir) else []

demo = gr.Interface(
    fn=caption_image_ollama,
    inputs=gr.Image(type="pil", label="上传图片"),
    outputs=gr.Textbox(label="生成的中文描述"),
    title="图片描述 (LangChain + Ollama LLaVA)",
    description="上传图片，使用 Ollama 中的 LLaVA 模型生成中文描述。",
    examples=examples_list, # 使用找到的示例图片列表
    allow_flagging="never"
)

# --- Launch ---
if __name__ == "__main__":
    if not os.path.exists(example_dir):
        print(f"'{example_dir}' 文件夹不存在. 请创建并放入示例图片.")

    print("\n确保 Ollama 服务正在运行并已拉取 'llava:7b' 模型.")
    demo.launch()

ChatOllama 初始化成功.


D:\Users\iSPACE\anaconda3\envs\rag\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(



确保 Ollama 服务正在运行并已拉取 'llava:7b' 模型.
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


#### 应用焦点3：通过扩散模型进行图像生成 (LangChain提示 + Stable Diffusion)

1. 模型考量：

Ollama本身不直接提供Stable Diffusion。我们将继续使用LangChain + Ollama来生成或优化提示，然后将提示传递给本地的Stable Diffusion（通过diffusers库）。

2. 构建UI（提示生成 + 图像生成）：

UI结构保持不变：`gr.Textbox` 输入基本概念，`gr.Slider` 等控制SD参数，`gr.Image` 输出最终图像，`gr.Textbox` 显示生成的详细提示 。   
后端函数将使用`LangChain ChatOllama` 生成提示，然后调用diffusers。

3. 带注释的代码示例 (LangChain提示 + Diffusers生成图像)：

In [7]:
# %pip install --upgrade diffusers[torch]
# %git lfs install
# %pip install accelerate
# %git clone git@hf.co:stable-diffusion-v1-5/stable-diffusion-v1-5

In [1]:
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from langchain_community.chat_models import ChatOllama # LangChain Ollama集成
from langchain_core.messages import HumanMessage, SystemMessage # LangChain消息类型
import os

# --- Stable Diffusion Setup ---
# 尝试使用 GPU，如果不可用则回退到 CPU
sd_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Stable Diffusion 使用设备: {sd_device}")
sd_pipe = None # SD pipeline 缓存

# --- LangChain Setup ---
# 确保Ollama服务正在运行，并且已拉取所需模型，例如 'ollama pull llama3'
# 使用 Llama3 模型进行提示生成
try:
    llm = ChatOllama(model="llama3", temperature=0.7)
    # 尝试进行一次简单的调用，检查 Ollama 是否正常工作
    # print(llm.invoke([HumanMessage(content="hi")]).content)
    print("LangChain Ollama 初始化成功 (模型: llama3).")
except Exception as e:
    llm = None # 如果初始化失败，则设为 None
    print(f"LangChain Ollama 初始化失败: {e}. 请确保 Ollama 服务运行且 'llama3' 模型已拉取.")


# --- LangChain Prompt Generation Function ---
def generate_detailed_prompt_langchain(basic_prompt):
    """使用LangChain和Ollama将基本概念扩展为详细的图像生成提示。"""
    if llm is None:
        print("用于提示生成的 LLM 未初始化. 返回基本提示.")
        return basic_prompt
    try:
        system_message_content = "You are a creative assistant skilled at expanding simple ideas into detailed, vivid image generation prompts suitable for models like Stable Diffusion. Focus on visual details, style, mood, and composition. Output only the prompt itself, without any introductory text."
        user_message_content = f"Based on the following concept, generate a detailed English image generation prompt: '{basic_prompt}'"

        # 修正: 完善 messages 列表
        messages = [
            SystemMessage(content=system_message_content),
            HumanMessage(content=user_message_content),
        ]

        response = llm.invoke(messages)
        detailed_prompt = response.content.strip()
        # 简单的清理，以防模型输出额外文本
        if "Here is a detailed prompt:" in detailed_prompt:
             detailed_prompt = detailed_prompt.split(":")[-1].strip()
        if detailed_prompt.startswith('"') and detailed_prompt.endswith('"'):
             detailed_prompt = detailed_prompt[1:-1]
        return detailed_prompt
    except Exception as e:
        print(f"使用 LangChain/Ollama 生成提示出错: {e}")
        # 回退到基本提示
        return basic_prompt

# --- Image Generation Function (LangChain Prompt + Stable Diffusion) ---
def generate_image_langchain_prompt(basic_concept, neg_prompt, steps, guidance, seed):
    """先用LangChain+Ollama生成详细提示，再用Stable Diffusion生成图像。"""
    global sd_pipe
    try:
        # 步骤 1: 使用LangChain+Ollama生成详细提示
        print(f"为概念 '{basic_concept}' 生成详细提示...")
        detailed_prompt = generate_detailed_prompt_langchain(basic_concept)
        print(f"生成的详细提示: {detailed_prompt}")

        # 步骤 2: 加载或使用缓存的Stable Diffusion模型
        if sd_pipe is None:
            print("正在加载 Stable Diffusion 模型... (可能需要一些时间)")
            # 注意: 建议使用更高效的模型或调整 torch_dtype 根据硬件优化
            sd_pipe = StableDiffusionPipeline.from_pretrained(
                "stable-diffusion-v1-5/stable-diffusion-v1-5", # 使用 SD 1.5 模型
                torch_dtype=torch.float16 if sd_device == "cuda" else torch.float32
            )
            sd_pipe = sd_pipe.to(sd_device)
            print("Stable Diffusion 模型加载完成.")

        # 设置种子
        generator = torch.Generator(device=sd_device)
        if seed == -1: # 如果种子为 -1，则随机生成
            seed = torch.randint(0, 2**32 - 1, (1,)).item()
        generator = generator.manual_seed(int(seed))
        print(f"使用种子: {seed} 生成图片")

        # 步骤 3: 使用生成的详细提示调用Stable Diffusion
        # 使用 autocast 提升 CUDA 性能 (如果可用)
        with torch.autocast(sd_device if sd_device == "cuda" else "cpu"):
            output = sd_pipe(
                prompt=detailed_prompt, # 使用 LangChain 生成的提示
                negative_prompt=neg_prompt,
                num_inference_steps=int(steps),
                guidance_scale=float(guidance),
                generator=generator
            )
            generated_image = output.images[0] # 获取生成的第一个图像

        # 返回图像和生成的提示
        return generated_image, detailed_prompt, seed # 返回实际使用的种子

    except Exception as e:
        print(f"生成图片出错: {e}")
        # 创建一个红色图像表示错误
        error_img = Image.new('RGB', (512, 512), color = (255, 0, 0))
        # 返回错误图像、错误信息和 -1 作为种子
        return error_img, f"生成图片出错: {e}", -1

# --- Gradio Interface ---
demo = gr.Interface(
    fn=generate_image_langchain_prompt, # 指向图像生成函数
    # 修正: 完善 inputs 列表
    inputs=[
        gr.Textbox(label="基本概念 (用于生成详细提示)", placeholder="例如: 一只在月球上弹吉他的猫"),
        gr.Textbox(label="负面提示 (不希望出现在图片中的内容)", placeholder="例如: 模糊, 水印, 低质量"),
        gr.Slider(minimum=10, maximum=100, value=30, step=1, label="推理步数"),
        gr.Slider(minimum=1.0, maximum=20.0, value=7.5, step=0.1, label="引导强度"),
        gr.Number(label="随机种子 (-1 表示随机)", value=-1, step=1),
    ],
    # 修正: 完善 outputs 列表
    outputs=[
        gr.Image(type="pil", label="生成的图片"),
        gr.Textbox(label="LangChain 生成的详细提示"),
        gr.Number(label="使用的种子"), # 显示实际使用的种子
    ],
    title="AI 协同绘图 (LangChain 提示增强 + Stable Diffusion)",
    description="输入一个基本概念，由 Ollama (llama3) 通过 LangChain 生成详细的英文提示，然后使用 Stable Diffusion 模型根据详细提示生成图片。",
    # 示例格式应与 inputs 列表对应
    examples=[
        ["一只宇航员骑着马的照片", "低质量, 模糊", 30, 7.5, 12345],
        ["戴眼镜编程的猫的油画", "卡通, 素描, 草图", 50, 9.0, 54321],
    ],
    flagging_mode="never"
)

# 启动 Gradio 应用
if __name__ == "__main__":
    print("\n--- 启动前检查 ---")
    print("1. 确保 Ollama 服务正在运行.")
    print("2. 确保已拉取 'llama3' 模型 (或其他你在代码中指定的用于生成提示的模型).")
    print("3. 确保已安装 pytorch 和 diffusers 库.")
    print("--------------------\n")
    demo.launch()

Stable Diffusion 使用设备: cpu
LangChain Ollama 初始化成功 (模型: llama3).

--- 启动前检查 ---
1. 确保 Ollama 服务正在运行.
2. 确保已拉取 'llama3' 模型 (或其他你在代码中指定的用于生成提示的模型).
3. 确保已安装 pytorch 和 diffusers 库.
--------------------



C:\Users\iSPACE\AppData\Local\Temp\ipykernel_23052\3107235259.py:19: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3", temperature=0.7)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## 评估和调试生成型人工智能

## 微调大型语言模型

## RAG基础篇

## MCP 详解篇

## A2A 详解篇

## Embedding篇

## 多模态大模型篇